Iran Twitter / X Data Analysis Project

 # italicized text*POI Scraper (Wikipedia Categories → CSV)

In [ ]:
# ============================================
# File: step1_collect_wikipedia_categories.py (UPDATED)
#
# Description:
#   Recursively collects people/pages from Wikipedia categories (including subcategories),
#   and saves a CSV per category with columns: Name, Wikipedia_Link.
#   Output structure (per course spec):
#       <IRAN_DIR>/
#         POIs/
#           <slug>/
#             <slug>_wikipedia.csv
#   Also produces a summary table and a ZIP with all CSVs.
# ============================================

import requests, time, os, zipfile
import pandas as pd
from urllib.parse import quote
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# ---------------- IRAN_DIR resolution ----------------
# Set the Iran directory path (adjust if needed)
IRAN_DIR = r"C:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran"

if not os.path.isdir(IRAN_DIR):
    raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran' בנתיב: {IRAN_DIR}")

# Base POIs directory (as per course structure)
POIS_DIR = os.path.join(IRAN_DIR, "POIs")
os.makedirs(POIS_DIR, exist_ok=True)

# ---------------- API config ----------------
BASE_API = "https://en.wikipedia.org/w/api.php"
HEADERS = {"User-Agent": "SCE-DataScience-FinalProject/1.0 (contact: student@example.com)"}

session = requests.Session()
retries = Retry(total=5, backoff_factor=0.8, status_forcelist=[403,429,500,502,503,504])
session.mount("https://", HTTPAdapter(max_retries=retries))

def api_get(params):
    """Thin wrapper around MediaWiki API GET with a fallback UA on 403."""
    r = session.get(BASE_API, params=params, headers=HEADERS, timeout=30)
    if r.status_code == 403:
        time.sleep(1)
        r = session.get(BASE_API, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    return r.json()

def get_category_members(category_title, cmtype="page"):
    """
    Fetches up to all members of a category, paging through 'continue'.
    cmtype can be "page" (actual pages) or "subcat" (subcategories).
    """
    members, params = [], {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": category_title,
        "cmlimit": "500",
        "cmtype": cmtype,
        "format": "json"
    }
    while True:
        data = api_get(params)
        members.extend(data.get("query", {}).get("categorymembers", []))
        if "continue" in data:
            params["cmcontinue"] = data["continue"]["cmcontinue"]
        else:
            break
    return members

def collect_people_from_category(root_category):
    """
    Breadth-first traversal from a root category:
      - collects 'page' members as rows {Name, Wikipedia_Link}
      - enqueues 'subcat' members for further traversal
    Returns a deduplicated DataFrame by Name.
    """
    seen, pages, queue = set(), [], [root_category]
    while queue:
        cat = queue.pop(0)
        if cat in seen:
            continue
        seen.add(cat)
        print(f"📂 סורק: {cat}")
        # pages
        for m in get_category_members(cat, "page"):
            title = m["title"]
            link  = "https://en.wikipedia.org/wiki/" + quote(title.replace(" ", "_"))
            pages.append({"Name": title, "Wikipedia_Link": link})
        # subcategories
        for sc in get_category_members(cat, "subcat"):
            queue.append(sc["title"])
        time.sleep(0.15)  # be polite
    return pd.DataFrame(pages).drop_duplicates(subset=["Name"]).reset_index(drop=True)

def safe_slug(cat):
    """
    Converts a category title to a folder/filename-safe slug.
    Example:
      "Category:Iranian physicians" -> "iranian_physicians"
    """
    name = cat.replace("Category:", "").strip()
    name = name.replace(" ", "_")
    for bad in ['"', "'", "'", """, """, "(", ")", "/", "\\", ":", "*", "?", "<", ">", "|", ",", ";", "—", "–"]:
        name = name.replace(bad, "")
    # compress consecutive underscores
    while "__" in name:
        name = name.replace("__", "_")
    return name.lower()

# ---------------- Categories to collect (expanded incl. healthcare) ----------------
categories = [
    # Existing:
    "Category:Government_ministers_of_Iran",
    "Category:Presidents_of_Iran",
    "Category:Vice_presidents_of_Iran",
    "Category:Iranian_ayatollahs",
    "Category:Iranian_actors",
    "Category:Iranian_singers",
    "Category:Iranian_scientists",
    "Category:Iranian_economists",
    "Category:Iranian_writers",
    "Category:Iranian_football_managers",

    # NEW — Healthcare related:
    "Category:Hospitals_in_Iran",
    "Category:Private_hospitals_in_Iran",
    "Category:Teaching_hospitals_in_Iran",
    "Category:Iranian_physicians",
    "Category:Iranian_cardiologists",
    "Category:Iranian_women_physicians",
    "Category:21st-century_Iranian_physicians",
    "Category:19th-century_Iranian_physicians",
    "Category:Medical_and_health_organisations_based_in_Iran",
    "Category:Healthcare_in_Iran",
    "Category:Medicine_in_Iran",

    # NEW — Journalists and Media:
    "Category:Iranian_journalists",
    "Category:Iranian_women_journalists",
    "Category:Iranian_reporters",
    "Category:Iranian_correspondents",
    "Category:Iranian_editors",
    "Category:Iranian_news_readers",

    # NEW — Activists and Political:
    "Category:Iranian_activists",
    "Category:Iranian_political_activists",
    "Category:Iranian_human_rights_activists",
    "Category:Iranian_women_activists",
    "Category:Iranian_feminists",
    "Category:Iranian_dissidents",
    "Category:Iranian_bloggers",

    # NEW — Sports:
    "Category:Iranian_footballers",
    "Category:Iranian_athletes",
    "Category:Iranian_wrestlers",
    "Category:Iranian_taekwondo_practitioners",
    "Category:Iranian_volleyball_players",
    "Category:Iranian_weightlifters",

    # NEW — Entertainment:
    "Category:Iranian_television_actors",
    "Category:Iranian_film_actors",
    "Category:Iranian_pop_singers",
    "Category:Iranian_rappers",
    "Category:Iranian_comedians",
]

# ---------------- Main loop: collect, save per-spec, summarize ----------------
csv_files = []
summary_rows = []

for cat in categories:
    try:
        slug = safe_slug(cat)
        cat_dir = os.path.join(POIS_DIR, slug)
        os.makedirs(cat_dir, exist_ok=True)

        csv_name = f"{slug}_wikipedia.csv"
        csv_path = os.path.join(cat_dir, csv_name)

        # Skip if file already exists and is not empty
        if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
            print(f"⏩ Skipping {slug}: file already exists.")
            summary_rows.append({"Category": cat, "Slug": slug, "SavedRows": "SKIPPED", "CSV": csv_path})
            continue

        df = collect_people_from_category(cat)
        n = len(df)

        if n == 0:
            print(f"⚠️ אין נתונים עבור {cat}")
            summary_rows.append({"Category": cat, "Slug": slug, "SavedRows": 0, "CSV": None})
            continue

        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        csv_files.append(csv_path)

        print(f"✅ נשמר: {csv_path} | רשומות: {n}")
        summary_rows.append({"Category": cat, "Slug": slug, "SavedRows": n, "CSV": csv_path})

    except Exception as e:
        print(f"❌ שגיאה בקטגוריה {cat}: {e}")
        summary_rows.append({"Category": cat, "Slug": safe_slug(cat), "SavedRows": 0, "CSV": None})

# --- Summary table ---
summary_df = pd.DataFrame(summary_rows)

# Add numeric column for sorting (convert SKIPPED to -1 for sorting purposes)
summary_df['SavedRows_Numeric'] = summary_df['SavedRows'].apply(
    lambda x: -1 if x == "SKIPPED" else (int(x) if isinstance(x, (int, float)) else 0)
)

# Sort by the numeric column and drop it
summary_df = summary_df.sort_values("SavedRows_Numeric", ascending=False).reset_index(drop=True)
summary_df = summary_df.drop(columns=['SavedRows_Numeric'])

# Calculate total (only numeric values)
total_saved = sum(x for x in summary_df["SavedRows"] if isinstance(x, (int, float)))

print("\n================= Summary =================")
for _, r in summary_df.iterrows():
    base = os.path.basename(r['CSV']) if (pd.notna(r['CSV']) and r['CSV']) else "-"
    saved_str = str(r['SavedRows'])
    print(f"• {r['Category']} -> {r['Slug']}: {saved_str} רשומות  |  קובץ: {base}")
print(f"\n📊 Total across files: {total_saved} rows")
print("===========================================\n")

# Display summary table
display(summary_df)

# ---------------- ZIP all CSVs ----------------
if csv_files:
    zip_path = os.path.join(IRAN_DIR, "Iran_POIs_Wikipedia_Categories.zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in csv_files:
            # Save relative path in zip
            arcname = os.path.relpath(file, start=IRAN_DIR)
            zipf.write(file, arcname)
    print(f"💾 Created ZIP with all categories: {zip_path}")
else:
    print("ℹ️ No CSV files were created → no ZIP.")

 **Step 3: Wikidata Enrichment for ALL POI Folders**

In [ ]:
# ============================================
# Step 3: Wikidata Enrichment for ALL POI Folders (UPDATED)
# ============================================
# For each subfolder under POIs:
#  1) Locate *_wikipedia.csv (or the first .csv)
#  2) Detect the Wikipedia link column (fallback to Name)
#  3) Resolve wikidata_qid (+ wikidata_url)
#  4) Fetch details via SPARQL in batches
#  5) Save:
#     a) *_with_wikidata_ids_and_links.csv
#     b) *_with_wikidata_ids_and_links_wikidata_detailed.csv
# ============================================

import os, re, time, glob, json, math, random
import pandas as pd
import requests
from pathlib import Path

# ---------------- Dynamic path detection ----------------
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

ROOT_DIR = os.path.join(IRAN_DIR, "POIs")
os.makedirs(ROOT_DIR, exist_ok=True)
print(f"📁 ROOT_DIR: {ROOT_DIR}")

# ---------------- HTTP / Endpoints ----------------
UA = {"User-Agent": "SCE-DS-FinalProject/1.0 (contact: student@example.com)"}
MEDIAWIKI_API = "https://en.wikipedia.org/w/api.php"
SPARQL = "https://query.wikidata.org/sparql"

# ---------------- Helpers: retries ----------------
def http_get(url, params=None, headers=None, timeout=30, max_tries=4, sleep_base=0.8):
    """
    Simple GET with retries/backoff. Jitters a bit to be polite.
    """
    headers = headers or UA
    for attempt in range(1, max_tries + 1):
        try:
            r = requests.get(url, params=params, headers=headers, timeout=timeout)
            if r.status_code == 403 and headers is UA:
                # fallback UA
                r = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"}, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == max_tries:
                # last attempt: raise
                raise
            sleep_s = sleep_base * attempt + random.uniform(0, 0.3)
            time.sleep(sleep_s)
    # Shouldn't reach here
    raise RuntimeError("GET retries exhausted")

# ---------------- Column detection ----------------
def find_wikipedia_column(df: pd.DataFrame) -> str:
    # First pass: headers with hints
    for name in df.columns:
        n = str(name).strip().lower()
        if any(k in n for k in ["wikipedia", "wiki", "link", "url"]):
            return name
    # Second pass: sample content
    for name in df.columns:
        sample = " ".join(map(str, df[name].dropna().astype(str).head(20).tolist())).lower()
        if "wikipedia.org" in sample or sample.startswith("http"):
            return name
    # Fallback
    if "Wikipedia_Link" in df.columns:
        return "Wikipedia_Link"
    # As a last resort: if Name exists, we'll construct enwiki URLs from it
    if "Name" in df.columns:
        return None  # signal to construct from Name
    raise ValueError("לא מצאתי עמודת קישור לוויקיפדיה, ואין גם עמודת 'Name' לבניית קישורים.")

def name_to_enwiki_url(name: str) -> str:
    from urllib.parse import quote
    if not isinstance(name, str) or not name.strip():
        return None
    title = name.strip().replace(" ", "_")
    return f"https://en.wikipedia.org/wiki/{quote(title)}"

def wikipedia_url_to_title(url: str) -> str | None:
    if not isinstance(url, str) or not url:
        return None
    try:
        url = url.split("?")[0].split("#")[0]
        title = url.rstrip("/").split("/")[-1]
        return title if title else None
    except Exception:
        return None

# ---------------- QID resolution (with cache) ----------------
qid_cache = {}

def get_qid_from_wikipedia_url(url: str) -> str | None:
    """
    Resolve QID for a single Wikipedia page using the pageprops endpoint.
    """
    title = wikipedia_url_to_title(url)
    if not title:
        return None
    if title in qid_cache:
        return qid_cache[title]
    params = {
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "redirects": 1,
        "titles": title,
    }
    try:
        r = http_get(MEDIAWIKI_API, params=params, headers=UA, timeout=25)
        data = r.json()
        page = next(iter(data["query"]["pages"].values()))
        qid = page.get("pageprops", {}).get("wikibase_item")
        if qid:
            qid_cache[title] = qid
        return qid
    except Exception:
        return None

# ---------------- SPARQL batch fetch ----------------
def batch_fetch_wikidata_details(qids: list[str], batch_size: int = 50) -> dict:
    """
    Fetch properties for many QIDs via SPARQL using VALUES batching.
    Returns dict: { QID: {fields...}, ... }
    """
    results = {}
    # Normalize & unique
    qids = [q for q in qids if isinstance(q, str) and q.strip()]
    uniq = sorted(set(qids))
    if not uniq:
        return results

    def run_batch(subset):
        values = " ".join(f"wd:{q}" for q in subset)
        query = f"""
        SELECT ?item ?genderLabel ?occupationLabel ?countryLabel ?placeOfBirthLabel ?dateOfBirth WHERE {{
          VALUES ?item {{ {values} }}
          OPTIONAL {{ ?item wdt:P21 ?gender. }}
          OPTIONAL {{ ?item wdt:P106 ?occupation. }}
          OPTIONAL {{ ?item wdt:P27 ?country. }}
          OPTIONAL {{ ?item wdt:P19 ?placeOfBirth. }}
          OPTIONAL {{ ?item wdt:P569 ?dateOfBirth. }}
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,fa,en-gb". }}
        }}
        """
        r = http_get(SPARQL, params={"query": query, "format": "json"}, headers=UA, timeout=45)
        rows = r.json().get("results", {}).get("bindings", [])
        # collect multiple rows per item
        tmp = {}
        for b in rows:
            uri = b.get("item", {}).get("value", "")
            q = uri.rsplit("/", 1)[-1] if uri else None
            if not q:
                continue
            cur = tmp.setdefault(q, {"wikidata_gender": set(),
                                     "wikidata_occupation": set(),
                                     "wikidata_country_of_citizenship": set(),
                                     "wikidata_place_of_birth": None,
                                     "wikidata_date_of_birth": None})
            if "genderLabel" in b:
                cur["wikidata_gender"].add(b["genderLabel"]["value"])
            if "occupationLabel" in b:
                cur["wikidata_occupation"].add(b["occupationLabel"]["value"])
            if "countryLabel" in b:
                cur["wikidata_country_of_citizenship"].add(b["countryLabel"]["value"])
            if "placeOfBirthLabel" in b and not cur["wikidata_place_of_birth"]:
                cur["wikidata_place_of_birth"] = b["placeOfBirthLabel"]["value"]
            if "dateOfBirth" in b and not cur["wikidata_date_of_birth"]:
                cur["wikidata_date_of_birth"] = b["dateOfBirth"]["value"]
        # flatten sets
        for q, d in tmp.items():
            results[q] = {
                "wikidata_gender": "; ".join(sorted(d["wikidata_gender"])) or None,
                "wikidata_occupation": "; ".join(sorted(d["wikidata_occupation"])) or None,
                "wikidata_country_of_citizenship": "; ".join(sorted(d["wikidata_country_of_citizenship"])) or None,
                "wikidata_place_of_birth": d["wikidata_place_of_birth"],
                "wikidata_date_of_birth": d["wikidata_date_of_birth"],
            }

    for i in range(0, len(uniq), batch_size):
        subset = uniq[i:i+batch_size]
        # polite pacing
        try:
            run_batch(subset)
        except Exception as e:
            print(f"⚠️ SPARQL batch failed ({subset[0]}..): {e}. מנסה שוב בקבוצות קטנות יותר...")
            # fallback: try half batch to circumvent transient errors
            mid = len(subset)//2 or 1
            for chunk in (subset[:mid], subset[mid:]):
                try:
                    run_batch(chunk)
                except Exception as ee:
                    print(f"❌ SPARQL sub-batch failed ({chunk[0]}..): {ee}")
        time.sleep(0.5)

    return results

# ---------------- Main processing ----------------
folders = [f for f in sorted(os.listdir(ROOT_DIR)) if os.path.isdir(os.path.join(ROOT_DIR, f))]
print(f"🗂️ נמצאו {len(folders)} תיקיות POI לעיבוד.")

for folder in folders:
    FOLDER_PATH = os.path.join(ROOT_DIR, folder)
    print(f"\n📂 מעבד תיקייה: {folder}")

    # Locate input CSV (prefer *_wikipedia.csv)
    candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*_wikipedia.csv")))
    if not candidates:
        candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*.csv")))
    if not candidates:
        print("⚠️ לא נמצא קובץ CSV בתיקייה הזו, מדלג.")
        continue

    INPUT_CSV = candidates[0]
    base_name = re.sub(r"\.csv$", "", os.path.basename(INPUT_CSV))
    
    # ⚡ SMART SKIP: Check if Wikidata enrichment already complete
    detailed_path = os.path.join(FOLDER_PATH, f"{base_name}_with_wikidata_ids_and_links_wikidata_detailed.csv")
    if os.path.exists(detailed_path):
        try:
            df_check = pd.read_csv(detailed_path, nrows=5)
            wd_cols = [c for c in df_check.columns if c.startswith('wikidata_')]
            if wd_cols and df_check[wd_cols].notna().any().any():
                print(f"✅ SKIP: Wikidata enrichment already complete")
                print(f"   File: {os.path.basename(detailed_path)}")
                print(f"   Found {len(wd_cols)} Wikidata columns with data")
                continue
        except Exception:
            pass

    candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*_wikipedia.csv")))
    if not candidates:
        candidates = sorted(glob.glob(os.path.join(FOLDER_PATH, "*.csv")))
    if not candidates:
        print("⚠️ לא נמצא קובץ CSV בתיקייה הזו, מדלג.")
        continue

    INPUT_CSV = candidates[0]
    try:
        df = pd.read_csv(INPUT_CSV)
    except Exception as e:
        print(f"❌ לא הצלחתי לקרוא את הקובץ {os.path.basename(INPUT_CSV)}: {e} — מדלג.")
        continue

    # Detect / construct Wikipedia links
    try:
        wiki_col = find_wikipedia_column(df)
    except Exception as e:
        print(f"⚠️ {e} — מנסה לבנות קישורי ויקיפדיה מ־'Name'...")
        wiki_col = None

    if wiki_col is None:
        if "Name" in df.columns:
            df["Wikipedia_Link"] = df["Name"].apply(name_to_enwiki_url)
            wiki_col = "Wikipedia_Link"
        else:
            print("❌ אין עמודת קישור/Name — מדלג על התיקייה.")
            continue

    print("🧩 עמודת ויקיפדיה:", wiki_col)

    # ---- A) Resolve QIDs (with simple in-memory cache) ----
    qids = []
    for url in df[wiki_col].fillna(""):
        qids.append(get_qid_from_wikipedia_url(url))
        time.sleep(0.08)  # polite throttle

    df["wikidata_qid"] = qids
    df["wikidata_url"] = df["wikidata_qid"].apply(lambda q: f"https://www.wikidata.org/wiki/{q}" if isinstance(q, str) and q else None)

    # Save mid file
    mid_path = os.path.join(
        FOLDER_PATH,
        re.sub(r"\.csv$", "", os.path.basename(INPUT_CSV)) + "_with_wikidata_ids_and_links.csv"
    )
    try:
        df.to_csv(mid_path, index=False, encoding="utf-8-sig")
        print(f"💾 נשמר ביניים: {os.path.basename(mid_path)}  (שורות: {len(df)})")
    except Exception as e:
        print(f"❌ שגיאה בשמירת קובץ ביניים: {e}")

    # ---- B) Fetch details via SPARQL in batches ----
    uniq_qids = [q for q in pd.Series(df["wikidata_qid"]).dropna().astype(str).unique().tolist() if q]
    details_map = batch_fetch_wikidata_details(uniq_qids, batch_size=50)

    details_rows = []
    for q in df["wikidata_qid"]:
        if isinstance(q, str) and q in details_map:
            details_rows.append(details_map[q])
        else:
            details_rows.append({
                "wikidata_gender": None,
                "wikidata_occupation": None,
                "wikidata_country_of_citizenship": None,
                "wikidata_place_of_birth": None,
                "wikidata_date_of_birth": None,
            })

    details_df = pd.DataFrame(details_rows)
    enriched = pd.concat([df, details_df], axis=1)

    out_path = os.path.join(
        FOLDER_PATH,
        re.sub(r"\.csv$", "", os.path.basename(INPUT_CSV)) + "_with_wikidata_ids_and_links_wikidata_detailed.csv"
    )
    try:
        enriched.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"✅ נשמר פלט סופי: {os.path.basename(out_path)}  (שורות: {len(enriched)})")
    except Exception as e:
        print(f"❌ שגיאה בשמירת הפלט הסופי: {e}")

print("\n🎉 הושלם עיבוד לכל התיקיות!")


**Step 4: Find Twitter handles for ALL POI folders**

In [ ]:
# ============================================
# Step 4: Find Twitter handles for ALL POI folders
# Adds only: Twitter_username + Twitter_url (no twitter_source)
# ============================================

import os, re, time, json, glob, requests, pandas as pd, random
from pathlib import Path
from urllib.parse import urlparse, unquote
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---- Dynamic path detection ----
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

# ירוץ על כל התיקיות תחת POIs
REL_ROOT = "POIs"
ROOT = os.path.join(IRAN_DIR, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
print("📁 ROOT:", ROOT)

# ---------- HTTP session with retries ----------
UA = {"User-Agent": "SCE-DS-FinalProject/1.0 (contact: student@example.com)"}
session = requests.Session()
session.headers.update(UA)

# urllib3 v2: allowed_methods (not method_whitelist). עדיף frozenset
retry_cfg = Retry(
    total=5,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset({"GET"}),
)
session.mount("https://", HTTPAdapter(max_retries=retry_cfg))
session.mount("http://", HTTPAdapter(max_retries=retry_cfg))

SPARQL = "https://query.wikidata.org/sparql"
MEDIAWIKI_API = "https://en.wikipedia.org/w/api.php"

# ---------- helpers ----------
def pick_csv(folder):
    for pat in ["*_with_wikidata_ids_and_links_wikidata_detailed*.csv",
                "*_with_wikidata_ids_and_links*.csv",
                "*_wikipedia.csv",
                "*.csv"]:
        cand = sorted(glob.glob(os.path.join(folder, pat)))
        # סנן קבצים שכבר מכילים _with_twitter (אלה הם פלטים, לא קלטים)
        cand = [c for c in cand if not c.endswith("_with_twitter.csv")]
        if cand: return cand[0]
    return None

def clean_handle(h):
    if not isinstance(h, str):
        return None
    h = h.strip().lstrip("@")
    h = re.sub(r"[/?#].*$", "", h)
    m = re.match(r"^[A-Za-z0-9_]{1,15}$", h)
    return m.group(0) if m else None

def extract_handle_from_url(url):
    """
    מחלץ ידית מכתובת twitter/x אם קיימת. לינקים מסוג /i/user/12345 לא מכילים ידית => נחזיר None.
    """
    try:
        u = url.split("?")[0].split("#")[0]
        host = urlparse(u).netloc.lower()
        if any(d in host for d in ["twitter.com","x.com","mobile.twitter.com","www.twitter.com","www.x.com"]):
            parts = urlparse(u).path.strip("/").split("/")
            if parts and parts[0].lower() not in {"i","intent","share","home"}:
                return clean_handle(parts[0])
    except:
        pass
    return None

def build_twitter_url(handle):
    if not isinstance(handle, str) or not handle.strip():
        return None
    return f"https://x.com/{handle.strip().lstrip('@')}"

def find_wikipedia_column(df):
    # ראשית: לפי שם עמודה
    for c in df.columns:
        n = str(c).lower()
        if any(k in n for k in ["wikipedia", "wiki", "link", "url"]):
            return c
    # שנית: לפי תוכן
    for c in df.columns:
        vals = " ".join(map(str, df[c].dropna().astype(str).head(15).tolist())).lower()
        if "wikipedia.org" in vals or "https://" in vals or "http://" in vals:
            return c
    return None

def http_get(url, params=None, headers=None, timeout=30, tries=3, backoff=0.7):
    headers = headers or UA
    for attempt in range(1, tries+1):
        try:
            r = session.get(url, params=params, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == tries:
                raise
            time.sleep(backoff * attempt + random.uniform(0, 0.2))

def guess_twitter_from_wiki(title_or_url):
    """
    נסה לחלץ ידית מהדף בוויקיפדיה:
    - External links
    - טקסט המקור (wikitext) עם regex
    """
    try:
        if isinstance(title_or_url, str) and title_or_url.startswith(("http://", "https://")):
            title = unquote(title_or_url.split("?")[0].split("#")[0].rstrip("/").split("/")[-1])
            title = title.replace("_", " ")
        else:
            title = str(title_or_url).strip().replace("_", " ")
        if not title:
            return None

        r = http_get(MEDIAWIKI_API, params={
            "action": "parse",
            "format": "json",
            "page": title,
            "prop": "externallinks|wikitext",
            "redirects": 1
        }, timeout=30, tries=3)
        data = r.json()

        links = data.get("parse", {}).get("externallinks", []) or []
        for ln in links:
            h = extract_handle_from_url(ln)
            if h:
                return h

        wt = data.get("parse", {}).get("wikitext", {}).get("*", "")
        # תופס גם twitter וגם x.com
        for m in re.finditer(r"(?:https?://)?(?:www\.)?(?:twitter|x)\.com/([A-Za-z0-9_]{1,15})", wt, re.I):
            h = clean_handle(m.group(1))
            if h:
                return h
    except:
        pass
    return None

def batch_p2002(qids):
    """
    מחזיר מפה { QID: handle or None } עם retry.
    """
    qids = [q for q in qids if isinstance(q, str) and q]
    if not qids:
        return {}
    values = " ".join(f"(wd:{q})" for q in qids)
    q = f"""
    SELECT ?item ?twitter WHERE {{
      VALUES (?item) {{ {values} }}
      OPTIONAL {{ ?item wdt:P2002 ?twitter. }}
    }}
    """
    # retry דרך http_get
    r = http_get(SPARQL, params={"query": q, "format": "json"}, timeout=45, tries=3, backoff=0.9)
    rows = r.json().get("results", {}).get("bindings", [])
    out = {}
    for b in rows:
        qid = b["item"]["value"].rsplit("/",1)[-1]
        tw = b.get("twitter", {}).get("value")
        out[qid] = clean_handle(tw) if tw else None
    return out

# cache (QID -> handle)
cache_file = os.path.join(ROOT, "_twitter_cache.json")
twitter_cache = {}
if os.path.exists(cache_file):
    try:
        twitter_cache = json.load(open(cache_file, "r", encoding="utf-8"))
    except:
        twitter_cache = {}

# ---------- process all subfolders ----------
folders = [os.path.join(ROOT, d) for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))]
folders.sort()

for FOLDER in folders:
    in_csv = pick_csv(FOLDER)
    if not in_csv:
        print(f"⚠️ אין CSV בתיקייה {os.path.basename(FOLDER)} — דילוג")
        continue

    # ⚡ SMART SKIP: Check if Twitter enrichment already complete
    # בדיקת כל הפורמטים האפשריים של קבצי Twitter
    base_name = os.path.splitext(os.path.basename(in_csv))[0]
    possible_outputs = [
        os.path.join(FOLDER, base_name + "_with_twitter.csv"),
        os.path.join(FOLDER, base_name.replace("_wikidata_detailed", "") + "_with_twitter.csv"),
        # אם הקובץ נקרא *_detailed.csv, נסה גם ללא detailed
        os.path.join(FOLDER, base_name.replace("_detailed", "") + "_with_twitter.csv")
    ]
    
    skip = False
    for out_path in possible_outputs:
        if os.path.exists(out_path):
            try:
                df_check = pd.read_csv(out_path, nrows=5)
                if 'Twitter_username' in df_check.columns:
                    twitter_count = df_check['Twitter_username'].notna().sum()
                    if twitter_count > 0:
                        print(f"\n📂 {os.path.basename(FOLDER)}")
                        print(f"✅ SKIP: Twitter enrichment already complete")
                        print(f"   File: {os.path.basename(out_path)} ({twitter_count}+ handles)")
                        skip = True
                        break
            except Exception:
                pass
    
    if skip:
        continue

    print(f"\n📂 {os.path.basename(FOLDER)}")
    try:
        df = pd.read_csv(in_csv)
    except Exception as e:
        print(f"❌ לא הצלחתי לקרוא את הקובץ: {e} — דילוג")
        continue

    wiki_col = find_wikipedia_column(df)
    if "wikidata_qid" not in df.columns:
        df["wikidata_qid"] = None

    if "Twitter_username" not in df.columns:
        df["Twitter_username"] = None

    # 1) Wikidata P2002 (with cache)
    qids = [q for q in df["wikidata_qid"].dropna().astype(str).unique() if q]
    p2002_map = {}
    # קודם מה-cache
    for q in qids:
        if q in twitter_cache:
            p2002_map[q] = twitter_cache[q]
    # מה שחסר — לשאילתא (בקבוצות)
    to_query = [q for q in qids if q not in p2002_map]
    for i in range(0, len(to_query), 60):
        part = to_query[i:i+60]
        try:
            m = batch_p2002(part)
            p2002_map.update(m)
            # לעדכן cache רק כשיש ידית (כדי לא “לקבע” None)
            twitter_cache.update({k: v for k, v in m.items() if v})
            time.sleep(0.25)
        except Exception as e:
            print("⚠️ SPARQL batch failed:", e)

    # כתיבה לפי QID
    for idx, row in df.iterrows():
        qid = row.get("wikidata_qid")
        if isinstance(qid, str) and qid in p2002_map and p2002_map[qid]:
            if not df.at[idx, "Twitter_username"]:
                df.at[idx, "Twitter_username"] = p2002_map[qid]

    # 2) Wikipedia fallback
    if wiki_col:
        missing = df["Twitter_username"].isna()
        for idx, row in df[missing].iterrows():
            url = row[wiki_col]
            if isinstance(url, str) and url.strip():
                h = guess_twitter_from_wiki(url)
                if h:
                    df.at[idx, "Twitter_username"] = h
                    # אם יש גם QID — עדכן cache (נחסוך בהפעלות עתידיות)
                    qid = row.get("wikidata_qid")
                    if isinstance(qid, str) and qid:
                        twitter_cache[qid] = h

    # ניקוי + URL
    df["Twitter_username"] = df["Twitter_username"].apply(lambda h: clean_handle(h) if isinstance(h, str) else None)
    df["Twitter_url"] = df["Twitter_username"].apply(build_twitter_url)

    # save (UTF-8-SIG to be Excel-friendly)
    out_path = os.path.join(FOLDER, os.path.splitext(os.path.basename(in_csv))[0] + "_with_twitter.csv")
    try:
        df.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"✅ Saved: {os.path.basename(out_path)} | found {df['Twitter_username'].notna().sum()} / {len(df)}")
    except Exception as e:
        print(f"❌ שמירה נכשלה: {e}")

# save cache
try:
    with open(cache_file, "w", encoding="utf-8") as f:
        json.dump(twitter_cache, f, ensure_ascii=False, indent=2)
    print("\n💾 cache saved:", cache_file)
except Exception as e:
    print("⚠️ couldn't save cache:", e)

print("\n🎉 Done — Step 4 completed for all POI folders (username + url only).")


In [ ]:
# ============================================
# Build Manual_Search_POIs.csv from all *_with_twitter.csv
# ממלא אוטומטית את הקובץ המרכזי מכל התיקיות (מהעמודה Twitter_username)
# בהתאם להנחיות הקורס: שם הקובץ Manual_Search_POIs.csv ושמירה תחת POIs/
# ============================================

import os, re, glob, pandas as pd
from pathlib import Path

# ---- Dynamic path detection ----
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

REL_ROOT = "POIs"
ROOT = os.path.join(IRAN_DIR, REL_ROOT)
assert os.path.isdir(ROOT), f"❌ לא קיימת התיקייה: {ROOT}"
print("📁 ROOT:", ROOT)

# שם הקובץ לפי ההנחיות (שלב 5)
OUTPUT_NAME = "Manual_Search_POIs.csv"
OUTPUT_PATH = os.path.join(ROOT, OUTPUT_NAME)

# -------- Keyword map (folder_slug -> keyword) --------
# הערה: השמות כאן ב-lowercase כי התיקיות נוצרו עם slug קטן (safe_slug)
KEYWORD_MAP = {
    # Existing
    "government_ministers_of_iran": "Iran minister",
    "presidents_of_iran": "Iran president",
    "vice_presidents_of_iran": "Iran vice president",
    "iranian_ayatollahs": "Iran ayatollah",
    "iranian_actors": "Iran actor",
    "iranian_singers": "Iran singer",
    "iranian_scientists": "Iran scientist",
    "iranian_economists": "Iran economist",
    "iranian_writers": "Iran writer",
    "iranian_football_managers": "Iran football manager",

    # Healthcare related
    "hospitals_in_iran": "Iran hospital",
    "private_hospitals_in_iran": "Iran hospital",
    "teaching_hospitals_in_iran": "Iran teaching hospital",
    "iranian_physicians": "Iran physician",
    "iranian_cardiologists": "Iran cardiologist",
    "iranian_women_physicians": "Iran physician",
    "21st-century_iranian_physicians": "Iran physician",
    "19th-century_iranian_physicians": "Iran physician",
    "medical_and_health_organisations_based_in_iran": "Iran health org",
    "healthcare_in_iran": "Iran healthcare",
    "medicine_in_iran": "Iran medicine",
}

CENTURY_PREFIX = re.compile(r"^\d{1,2}(st|nd|rd|th)-century_")

def base_slug(folder_slug: str) -> str:
    """מסיר קידומת של מאה מהסלאג (e.g., 19th-century_...)"""
    return CENTURY_PREFIX.sub("", folder_slug)

def infer_keyword_from_folder(folder_slug: str) -> str:
    """מפיק מילת חיפוש הגיונית מהסלאג, עם מפה ידנית לקייסים חשובים."""
    slug = folder_slug.lower().strip()
    if slug in KEYWORD_MAP:
        return KEYWORD_MAP[slug]
    slug = base_slug(slug)
    if slug in KEYWORD_MAP:
        return KEYWORD_MAP[slug]
    # כלל ברירת מחדל: הורד "iranian_" וה־s הסופית, והחלף _ ברווח
    token = slug.replace("iranian_", "").replace("_", " ").strip()
    token = token[:-1] if token.endswith("s") else token
    return f"Iran {token}".strip()

def detect_name_col(df: pd.DataFrame) -> str:
    """מאתר עמודת שם סבירה."""
    for c in df.columns:
        n = str(c).strip().lower()
        if n in {"name", "poi name", "poi_name", "full_name", "full name"}:
            return c
    # fallback: העמודה הראשונה
    return df.columns[0]

def detect_twitter_col(df: pd.DataFrame) -> str | None:
    """מאתר עמודת Twitter_username (או דומה)."""
    for c in df.columns:
        n = str(c).strip().lower()
        if n in {"twitter_username", "twitter", "handle", "username"}:
            return c
    # חפש עמודה שיש בה הרבה ערכים שנראים כמו ידיות
    for c in df.columns:
        vals = df[c].dropna().astype(str).head(20).tolist()
        hits = sum(1 for v in vals if re.match(r"^@?[A-Za-z0-9_]{1,15}$", v))
        if hits >= max(3, len(vals)//3):
            return c
    return None

def build_twitter_url(handle: str) -> str | None:
    if not isinstance(handle, str) or not handle.strip():
        return None
    h = handle.strip()
    h = h[1:] if h.startswith("@") else h
    return f"https://x.com/{h}"

rows = []
folders = [d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))]
folders.sort()

for folder in folders:
    folder_path = os.path.join(ROOT, folder)
    # חפש את קובץ היעד שנוצר בשלב 4
    cands = sorted(glob.glob(os.path.join(folder_path, "*_with_twitter.csv")))
    if not cands:
        continue
    path = cands[0]

    try:
        df = pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="utf-8-sig")
    except Exception as e:
        print("⚠️ can't read:", path, e)
        continue

    twitter_col = detect_twitter_col(df)
    if not twitter_col:
        continue

    name_col = detect_name_col(df)
    kw = infer_keyword_from_folder(folder)

    sub = df[[name_col, twitter_col]].copy()
    sub.rename(columns={name_col: "poi_name", twitter_col: "Twitter_username"}, inplace=True)

    # ניקוי, סינון, והשלמת URL
    sub["Twitter_username"] = sub["Twitter_username"].astype(str).str.strip()
    sub = sub[sub["Twitter_username"].str.len() > 0]
    sub["Twitter_username"] = sub["Twitter_username"].str.lstrip("@")
    sub["Twitter_url"] = sub["Twitter_username"].apply(build_twitter_url)
    sub["keyword"] = kw
    sub["source_folder"] = folder  # מאיזה קטגוריה הגיע

    rows.append(sub[["keyword", "poi_name", "Twitter_username", "Twitter_url", "source_folder"]])

# איחוד ושמירה
if rows:
    out = pd.concat(rows, ignore_index=True)
    # סינון כפילויות על בסיס שלישייה (שם משתמש + מילת חיפוש + שם POI)
    out.drop_duplicates(subset=["Twitter_username", "keyword", "poi_name"], inplace=True, ignore_index=True)
    # וריאנט נוסף: אם אותו handle הופיע בכמה תיקיות — נשמור את ההופעה הראשונה בלבד
    out.sort_values(["Twitter_username", "keyword", "poi_name"], inplace=True, ignore_index=True)
    out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
    print(f"✅ Saved: {OUTPUT_PATH} | rows: {len(out)}")
    try:
        from IPython.display import display
        display(out.head(20))
    except Exception:
        pass
else:
    print("ℹ️ לא נמצאו *_with_twitter.csv בתיקיות, או שאין ידיות למלא.")


In [ ]:
# ============================================
# Add "Twitter_status" column to mark suspended/invalid/missing users
# Works on Manual_Search_POIs.csv
# ============================================

import os, re, pandas as pd
from pathlib import Path
from IPython.display import display

# ---- Dynamic path detection ----
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

# ---- Config ----
REL_ROOT  = "POIs"
FILE_NAME = "Manual_Search_POIs.csv"   # <-- מעודכן לשם הקובץ החדש

ROOT = os.path.join(IRAN_DIR, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
path = os.path.join(ROOT, FILE_NAME)
assert os.path.exists(path), f"❌ File not found: {path}"
print("📁 ROOT:", ROOT)

# ---- Load ----
try:
    df = pd.read_csv(path, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding="utf-8-sig")

# ---- Helpers ----
HANDLE_RE = re.compile(r"^[A-Za-z0-9_]{1,15}$")

def classify_status(username: str) -> str:
    if pd.isna(username):
        return "missing"
    s = str(username).strip().lstrip("@")
    if not s:
        return "missing"
    # מספר בלבד: לעיתים מצביע על חשבון מושעה/מומר למזהה
    if s.isdigit():
        return "suspended"
    # ידית לא חוקית (אורך >15 או תווים לא תקינים/רווחים)
    if (len(s) > 15) or (HANDLE_RE.fullmatch(s) is None):
        return "invalid"
    return "active"

# ---- Compute & Save ----
df["Twitter_status"] = df["Twitter_username"].apply(classify_status)

df.to_csv(path, index=False, encoding="utf-8-sig")
print(f"✅ Updated file: {path} ({len(df)} rows)")
try:
    display(df.head(20))
except Exception:
    pass


In [ ]:
# ============================================
# 🧮 Step 6 – Enhanced Statistics & Visuals (UPDATED)
# Builds robust stats table + coverage + charts (portable)
# ============================================

from IPython.display import display
import os, glob, re, pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# ---------- Dynamic path detection ----------
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

REL_ROOT     = "POIs"
OUT_BASENAME = "POI_statistics.csv"             # basic
OUT_ENHANCED = "POI_statistics_enhanced.csv"    # enhanced with coverage
FIG_DIR      = "figures"

ROOT = os.path.join(IRAN_DIR, REL_ROOT)
os.makedirs(ROOT, exist_ok=True)
print("📁 ROOT:", ROOT)

# ---------- Helpers ----------
def safe_read_csv(path, nrows=None):
    for enc in (None, "utf-8", "utf-8-sig"):
        try:
            return pd.read_csv(path, nrows=nrows, encoding=enc)
        except Exception:
            pass
    # ניסיון אחרון עם engine=python ו-skip
    try:
        return pd.read_csv(path, nrows=nrows, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
    except Exception:
        return None

def is_wikipedia_csv(path):
    """Detect Wikipedia CSV by content (columns/values), not filename alone."""
    name = os.path.basename(path).lower()
    if any(k in name for k in ["wikidata", "with_twitter"]):
        return False
    df = safe_read_csv(path, nrows=10)
    if df is None or df.empty:
        return False
    cols = [str(c).strip().lower() for c in df.columns]
    if any(any(k in c for k in ["wikipedia", "wiki", "link", "url"]) for c in cols):
        return True
    for c in df.columns:
        try:
            sample = " ".join(map(str, df[c].dropna().astype(str).head(10).tolist())).lower()
            if "wikipedia.org" in sample or sample.startswith("http"):
                return True
        except Exception:
            pass
    return False

def detect_twitter_col(df):
    candidates = {"twitter_username","twitter user","twitter_user","twitter handle","handle","twitter"}
    for c in df.columns:
        if str(c).strip().lower() in candidates:
            return c
    for c in df.columns:
        norm = re.sub(r'[^a-z]', '', str(c).lower())
        if "twitter" in norm and ("username" in norm or "handle" in norm or "user" in norm or "name" in norm):
            return c
    return None

def detect_qid_col(df):
    for c in df.columns:
        if str(c).strip().lower() in {"wikidata_qid","qid","wikidata id","wikidataid"}:
            return c
    return None

# ---------- Build basic stats ----------
rows = []
folders = sorted([d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))])

for folder in folders:
    fpath = os.path.join(ROOT, folder)
    csvs = sorted(glob.glob(os.path.join(fpath, "*.csv")))
    if not csvs:
        continue

    wiki_file = None
    wikidata_file = None
    twitter_file = None

    # Twitter file by name
    for p in csvs:
        if re.search(r"with_twitter", os.path.basename(p), re.I):
            twitter_file = p
            break

    # Wikidata: prefer detailed
    for p in csvs:
        if re.search(r"wikidata.*detailed", os.path.basename(p), re.I):
            wikidata_file = p
            break
    if wikidata_file is None:
        for p in csvs:
            if re.search(r"with_wikidata_ids_and_links", os.path.basename(p), re.I):
                wikidata_file = p
                break

    # Wikipedia by content (fallback to "<folder>.csv")
    for p in csvs:
        if is_wikipedia_csv(p):
            wiki_file = p
            break
    if wiki_file is None:
        fallback = os.path.join(fpath, f"{folder}.csv")
        if os.path.exists(fallback):
            wiki_file = fallback

    wikipedia_count = 0
    wikidata_count  = 0
    twitter_count   = 0

    # Wikipedia count
    if wiki_file:
        dfw = safe_read_csv(wiki_file)
        if dfw is not None:
            wikipedia_count = len(dfw)

    # Wikidata count (count non-empty QIDs if column exists)
    if wikidata_file:
        dfd = safe_read_csv(wikidata_file)
        if dfd is not None:
            qcol = detect_qid_col(dfd)
            wikidata_count = dfd[qcol].notna().sum() if qcol else len(dfd)

    # Twitter count (valid handles only)
    if twitter_file:
        dft = safe_read_csv(twitter_file)
        if dft is not None:
            tw_col = detect_twitter_col(dft)
            if tw_col:
                s = dft[tw_col].astype(str).str.strip()
                valid = s[(s != "") &
                          (~s.str.fullmatch(r"\d+")) &
                          (~s.str.lower().isin({"nan","none","null","suspended"}))]
                twitter_count = valid.shape[0]

    rows.append({
        "Category": folder,
        "Wikipedia count": wikipedia_count,
        "Wikidata count": wikidata_count,
        "Twitter user count": twitter_count
    })

basic = pd.DataFrame(rows).sort_values("Category").reset_index(drop=True)
basic_path = os.path.join(ROOT, OUT_BASENAME)
basic.to_csv(basic_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved basic table: {basic_path}")
display(basic)

# ---------- Enhanced: coverage, gaps, totals ----------
enh = basic.copy()
with pd.option_context('mode.use_inf_as_na', True):
    enh["Wikidata coverage (%)"] = (100 * enh["Wikidata count"] / enh["Wikipedia count"]).fillna(0).round(1)
    enh["Twitter coverage (%)"]  = (100 * enh["Twitter user count"] / enh["Wikipedia count"]).fillna(0).round(1)

enh["Wikidata gap"] = (enh["Wikipedia count"] - enh["Wikidata count"]).clip(lower=0)
enh["Twitter gap"]  = (enh["Wikipedia count"] - enh["Twitter user count"]).clip(lower=0)

tot = pd.DataFrame([{
    "Category": "TOTAL",
    "Wikipedia count": enh["Wikipedia count"].sum(),
    "Wikidata count": enh["Wikidata count"].sum(),
    "Twitter user count": enh["Twitter user count"].sum(),
}])
tot["Wikidata coverage (%)"] = (100 * tot["Wikidata count"] / tot["Wikipedia count"]).round(1) if tot["Wikipedia count"].iloc[0] else 0.0
tot["Twitter coverage (%)"]  = (100 * tot["Twitter user count"] / tot["Wikipedia count"]).round(1) if tot["Wikipedia count"].iloc[0] else 0.0
tot["Wikidata gap"] = tot["Wikipedia count"] - tot["Wikidata count"]
tot["Twitter gap"]  = tot["Wikipedia count"] - tot["Twitter user count"]

enhanced = pd.concat([enh, tot], ignore_index=True)
enhanced_path = os.path.join(ROOT, OUT_ENHANCED)
enhanced.to_csv(enhanced_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved enhanced table: {enhanced_path}")
display(enhanced)

# ---------- Charts ----------
fig_dir = os.path.join(ROOT, FIG_DIR)
os.makedirs(fig_dir, exist_ok=True)

# מסננים את TOTAL עבור גרפים פר-קטגוריה
per_cat = enhanced[enhanced["Category"] != "TOTAL"].reset_index(drop=True)

# 1) Counts by category (lines) — ללא TOTAL
plt.figure(figsize=(12,5))
for col in ["Wikipedia count", "Wikidata count", "Twitter user count"]:
    plt.plot(per_cat["Category"], per_cat[col], marker='o', label=col)
plt.xticks(rotation=45, ha='right')
plt.title("POIs – Counts by Source")
plt.xlabel("Category")
plt.ylabel("Count")
plt.legend()
plt.tight_layout()
fig1_path = os.path.join(fig_dir, "counts_by_category.png")
plt.savefig(fig1_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig1_path}")

# 2) Coverage by category (%) — ללא TOTAL
plt.figure(figsize=(12,5))
plt.bar(per_cat["Category"], per_cat["Wikidata coverage (%)"], alpha=0.8, label="Wikidata coverage (%)")
plt.bar(per_cat["Category"], per_cat["Twitter coverage (%)"], alpha=0.6, label="Twitter coverage (%)")
plt.xticks(rotation=45, ha='right')
plt.title("POIs – Coverage by Category (%)")
plt.xlabel("Category")
plt.ylabel("Coverage (%)")
plt.legend()
plt.tight_layout()
fig2_path = os.path.join(fig_dir, "coverage_by_category.png")
plt.savefig(fig2_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig2_path}")

# 3) Stacked bars: gaps to Wikipedia — ללא TOTAL
plt.figure(figsize=(12,5))
bar_x = range(len(per_cat))
plt.bar(bar_x, per_cat["Wikidata count"], label="Wikidata count")
plt.bar(bar_x, per_cat["Wikidata gap"], bottom=per_cat["Wikidata count"], label="Wikidata gap")
plt.xticks(bar_x, per_cat["Category"], rotation=45, ha='right')
plt.title("Wikidata Count + Gap to Wikipedia")
plt.xlabel("Category")
plt.ylabel("Entities")
plt.legend()
plt.tight_layout()
fig3_path = os.path.join(fig_dir, "wikidata_gap_stacked.png")
plt.savefig(fig3_path, dpi=150)
plt.show()
print(f"📈 Saved: {fig3_path}")

print("\n✅ Done. Files written to:")
print(f"- {basic_path}")
print(f"- {enhanced_path}")
print(f"- {fig1_path}")
print(f"- {fig2_path}")
print(f"- {fig3_path}")


**API Twitter**

In [ ]:
# ============================================
# Step 10 — Candidates user metadata (built on your twitter_selenium scraper)
# Output: /content/drive/MyDrive/Iran/POIs/Candidates/Candidates_user_data.csv
# ============================================

# --- אם צריך התקנות בסיס (בטל הערה והריץ פעם אחת) ---
# !apt-get update -y && apt-get install -y chromium-browser chromium-chromedriver
# !pip install -U selenium tqdm pandas

# ---------- CONFIG (Dynamic path detection) ----------
import os, json, time, shutil, pandas as pd, sys, importlib.util, re
from pathlib import Path
from tqdm.auto import tqdm

# Auto-detect project root
IRAN_DIR = os.getcwd()
if not os.path.basename(IRAN_DIR) == 'Iran':
    for parent in Path(IRAN_DIR).parents:
        if parent.name == 'Iran' and os.path.isdir(os.path.join(parent, 'POIs')):
            IRAN_DIR = str(parent)
            break
    else:
        raise FileNotFoundError(f"❌ לא נמצאה תיקיית 'Iran'. נא להריץ מתוך תיקיית הפרויקט.")

BASE_DRIVE_DIR = os.path.join(IRAN_DIR, 'POIs')
CAND_DIR       = os.path.join(BASE_DRIVE_DIR, 'Candidates')

USERS_CSV      = os.path.join(BASE_DRIVE_DIR, 'POI_twitter_users_data.csv')     # קלט: ה-POIs המקוריים (עמודה: username)
CONN_CSV       = os.path.join(CAND_DIR, 'POIs_candidate_connections.csv')       # קלט: שלב 9 (עמודות: target_username, other_username)
OUT_CSV        = os.path.join(CAND_DIR, 'Candidates_user_data.csv')             # פלט: שלב 10

# 🔹 קובץ ה-scraper שלך
SCRAPER_PATH = os.path.join(BASE_DRIVE_DIR, 'tools', 'twitter_selenium.py')

# שליטה בהרצה
SAMPLE_LIMIT      = 4000     # ← בדיקת עשן על 10; שנה ל-None כדי לרוץ על כולם
BATCH_SIZE        = 40      # כמה משתמשים בכל מנה
RETRIES_PER_USER  = 2       # ניסיונות לכל משתמש
OPEN_PAUSE        = 1.4     # שניות המתנה אחרי פתיחת פרופיל

# מיפוי עמודות: מהסקרולר שלך אל הסכימה התקנית של הפרויקט
COLUMNS = [
    "username","Full_Name","Bio","Location","External_URL",
    "Followers_Count","Following_Count","Is_Verified","Is_Protected",
    "Joined_Date","Profile_URL","Profile_Image"
]

Path(CAND_DIR).mkdir(parents=True, exist_ok=True)
print(f"📁 Working directory: {IRAN_DIR}")

# ---------- LOAD YOUR SCRAPER MODULE ----------
def load_scraper(scraper_path: str):
    if not os.path.exists(scraper_path):
        raise FileNotFoundError(f"twitter_selenium.py not found at: {scraper_path}")
    spec = importlib.util.spec_from_file_location("twitter_selenium", scraper_path)
    mod  = importlib.util.module_from_spec(spec)
    sys.modules["twitter_selenium"] = mod
    spec.loader.exec_module(mod)
    return mod

ts = load_scraper(SCRAPER_PATH)  # מכיל scrape_twitter_profile(username)

# ---------- HELPERS ----------
def normalize_row_from_scraper(d: dict):
    """
    ממפה את פלט הסקרולר שלך לשדות של הפרויקט.
    scraper keys (your file): user_name, name, bio, location, url, joined_date,
                              followers, following, verified, profile_image, profile_url
    """
    return {
        "username":       (d.get("user_name") or "").lstrip("@"),
        "Full_Name":      d.get("name"),
        "Bio":            d.get("bio"),
        "Location":       d.get("location"),
        "External_URL":   d.get("url"),
        "Followers_Count":d.get("followers"),
        "Following_Count":d.get("following"),
        "Is_Verified":    bool(d.get("verified")),
        "Is_Protected":   None,  # הסקרולר שלך לא מחלץ; אפשר להשאיר None (לא נדרש בסעיף)
        "Joined_Date":    d.get("joined_date"),
        "Profile_URL":    d.get("profile_url"),
        "Profile_Image":  d.get("profile_image")
    }

# ---------- BUILD USER SET (שלב 9 + המקור, ללא כפילויות) ----------
user_set = set()

if os.path.exists(CONN_CSV):
    df_conn = pd.read_csv(CONN_CSV)
    if "target_username" in df_conn.columns:
        user_set.update(df_conn["target_username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())
    if "other_username" in df_conn.columns:
        user_set.update(df_conn["other_username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())

if os.path.exists(USERS_CSV):
    df_users = pd.read_csv(USERS_CSV)
    if "username" in df_users.columns:
        user_set.update(df_users["username"].dropna().astype(str).str.replace("@","", regex=False).str.strip())

users_all = [u for u in dict.fromkeys([u for u in user_set if u])]  # ייחוד + שמירת סדר
if SAMPLE_LIMIT:
    users_all = users_all[:SAMPLE_LIMIT]

print(f"Users to enrich: {len(users_all)} → {users_all[:5]}")

# ---------- HEADER BOOTSTRAP (מבטיח כותרות) ----------
if not os.path.exists(OUT_CSV):
    pd.DataFrame(columns=COLUMNS).to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# ---------- RESUME (דלג על מי שכבר נשמר) ----------
done = set()
try:
    prev = pd.read_csv(OUT_CSV, usecols=["username"])
    done = set(prev["username"].dropna().astype(str))
    print(f"↻ Resume: skipping {len(done)} already saved.")
except Exception as e:
    print("Resume read issue:", e)

# ---------- RUN (במנות) ----------
from math import ceil
total   = len(users_all)
batches = ceil(total / BATCH_SIZE)
idx = 0

for b in range(batches):
    chunk = [u for u in users_all[idx: idx+BATCH_SIZE] if u not in done]
    idx += BATCH_SIZE
    if not chunk:
        continue

    with tqdm(total=len(chunk), desc=f"Batch {b+1}/{batches}", unit="user") as pbar:
        for uname in chunk:
            row = None
            # ניסיונות חוזרים לכל משתמש (הסקרולר יפתח/יסגור כרום בעצמו)
            for attempt in range(RETRIES_PER_USER + 1):
                try:
                    raw = ts.scrape_twitter_profile(uname)   # ⚠️ מתוך twitter_selenium.py שלך
                    row = normalize_row_from_scraper(raw)
                    break
                except Exception as e:
                    if attempt < RETRIES_PER_USER:
                        time.sleep(0.8)
                    else:
                        # במקרה כשל — נשמור שורה "ריקה" עם השם וה-URL בלבד, כדי לא לחסום התקדמות
                        row = {
                            "username": uname, "Full_Name": None, "Bio": None, "Location": None,
                            "External_URL": None, "Followers_Count": None, "Following_Count": None,
                            "Is_Verified": None, "Is_Protected": None, "Joined_Date": None,
                            "Profile_URL": f"https://twitter.com/{uname}", "Profile_Image": None
                        }

            pd.DataFrame([row], columns=COLUMNS).to_csv(
                OUT_CSV, mode="a", header=False, index=False, encoding="utf-8-sig"
            )
            done.add(uname)
            pbar.update(1)

# ---------- דה-דופליקציה (ליתר ביטחון) ----------
try:
    df_out = pd.read_csv(OUT_CSV)
    df_out = df_out.drop_duplicates(subset=["username"], keep="first")
    df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print("✅ Saved:", OUT_CSV)
    print(df_out.head())
except Exception as e:
    print("Compact error:", e)


## Step 6 – POI Statistics Summary
Generate comprehensive statistics for all POI categories: Wikipedia count, Wikidata count, and Twitter count.

In [ ]:
# ============================================
# STEP 6 – POI Statistics Summary (COMPLETE)
# Counts UNIQUE Wikipedia, Wikidata, and Twitter entries per category
# Includes: Statistics table, Bar chart, and Venn diagram
# ============================================

import os
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# ---------- Auto-detect Iran project root ----------
def find_iran_root():
    """Find the Iran project root (folder named 'Iran' with 'POIs' subfolder)"""
    current = Path.cwd()
    
    # Check current directory
    if current.name == 'Iran' and (current / 'POIs').is_dir():
        return str(current)
    
    # Check parent directories
    for parent in current.parents:
        if parent.name == 'Iran' and (parent / 'POIs').is_dir():
            return str(parent)
    
    raise FileNotFoundError("❌ Could not find Iran project root (folder named 'Iran' with 'POIs' subfolder)")

IRAN_DIR = find_iran_root()
POIS_DIR = os.path.join(IRAN_DIR, "POIs")
print(f"✅ Found Iran project root: {IRAN_DIR}")
print(f"📁 POIs directory: {POIS_DIR}\n")

# ---------- Helper functions ----------
def safe_read_csv(filepath):
    """Safely read CSV with multiple encoding attempts"""
    if not os.path.exists(filepath):
        return None
    
    for encoding in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            df = pd.read_csv(filepath, encoding=encoding)
            return df
        except Exception:
            continue
    
    print(f"⚠️  Could not read: {os.path.basename(filepath)}")
    return None

def find_file_by_pattern(folder_path, patterns):
    """Find first file matching any of the given patterns"""
    for pattern in patterns:
        matches = list(Path(folder_path).glob(pattern))
        if matches:
            return str(matches[0])
    return None

def find_name_column(df):
    """Find the main name/title column in a DataFrame"""
    priority_cols = ['name', 'title', 'poi_name', 'person_name', 'entity_name']
    cols_lower = {c.lower(): c for c in df.columns}
    
    for priority in priority_cols:
        if priority in cols_lower:
            return cols_lower[priority]
    
    # Return first column as fallback
    return df.columns[0] if len(df.columns) > 0 else None

def find_twitter_column(df):
    """Find Twitter username column (case-insensitive)"""
    for col in df.columns:
        col_lower = col.lower()
        if 'twitter' in col_lower and ('username' in col_lower or 'user' in col_lower or 'handle' in col_lower):
            return col
    return None

def find_qid_column(df):
    """Find Wikidata QID column (case-insensitive)"""
    for col in df.columns:
        col_lower = col.lower()
        if 'wikidata' in col_lower and 'qid' in col_lower:
            return col
    return None

# ---------- Process all POIs folders ----------
results = []
all_entities = []  # For Venn diagram

folders = sorted([f for f in os.listdir(POIS_DIR) 
                  if os.path.isdir(os.path.join(POIS_DIR, f)) 
                  and not f.startswith('.')
                  and f not in ['figures', 'tools', 'Candidates']])

print(f"🔍 Processing {len(folders)} categories...\n")

for idx, folder_name in enumerate(folders, 1):
    folder_path = os.path.join(POIS_DIR, folder_name)
    print(f"[{idx}/{len(folders)}] Processing: {folder_name}")
    
    # Detect files
    wikipedia_file = find_file_by_pattern(folder_path, [
        f"{folder_name}_wikipedia.csv",
        "*_wikipedia.csv",
        f"{folder_name}.csv"
    ])
    
    wikidata_file = find_file_by_pattern(folder_path, [
        "*_wikidata_detailed.csv",
        "*_with_wikidata_ids_and_links_wikidata_detailed.csv",
        "*_with_wikidata*.csv"
    ])
    
    twitter_file = find_file_by_pattern(folder_path, [
        "*_with_twitter.csv",
        "*twitter*.csv"
    ])
    
    # Track entities for this category
    category_entities = set()
    
    # ========== 1. Count UNIQUE Wikipedia entries ==========
    wikipedia_count = 0
    if wikipedia_file:
        df_wiki = safe_read_csv(wikipedia_file)
        if df_wiki is not None:
            name_col = find_name_column(df_wiki)
            if name_col:
                # Remove duplicates by name
                unique_names = df_wiki[name_col].dropna().drop_duplicates()
                wikipedia_count = len(unique_names)
                category_entities.update(unique_names.str.lower().str.strip())
            else:
                wikipedia_count = len(df_wiki)
            print(f"   📖 Wikipedia: {wikipedia_count} unique entries")
    
    # ========== 2. Count UNIQUE Wikidata entries ==========
    wikidata_count = 0
    if wikidata_file:
        df_wikidata = safe_read_csv(wikidata_file)
        if df_wikidata is not None:
            qid_col = find_qid_column(df_wikidata)
            if qid_col:
                # Count unique non-null QIDs
                unique_qids = df_wikidata[qid_col].dropna().drop_duplicates()
                wikidata_count = len(unique_qids)
            else:
                # No QID column found, use unique names
                name_col = find_name_column(df_wikidata)
                if name_col:
                    unique_names = df_wikidata[name_col].dropna().drop_duplicates()
                    wikidata_count = len(unique_names)
                else:
                    wikidata_count = len(df_wikidata)
            print(f"   🔗 Wikidata: {wikidata_count} unique entries")
    
    # ========== 3. Count UNIQUE Twitter entries ==========
    twitter_count = 0
    if twitter_file:
        df_twitter = safe_read_csv(twitter_file)
        if df_twitter is not None:
            twitter_col = find_twitter_column(df_twitter)
            if twitter_col:
                # Remove duplicates by Twitter username
                valid_twitter = df_twitter[twitter_col].dropna()
                valid_twitter = valid_twitter[valid_twitter.astype(str).str.strip() != '']
                valid_twitter = valid_twitter[~valid_twitter.astype(str).str.lower().isin(['nan', 'none', 'null', 'suspended'])]
                unique_twitter = valid_twitter.drop_duplicates()
                twitter_count = len(unique_twitter)
                print(f"   🐦 Twitter: {twitter_count} unique users")
    
    # Store results for this category
    results.append({
        'folder': folder_name,
        'category_name': folder_name.replace('_', ' ').title(),
        'wikipedia_count': wikipedia_count,
        'wikidata_count': wikidata_count,
        'twitter_count': twitter_count
    })
    
    # Add to global entities list for Venn diagram
    all_entities.append({
        'has_wikipedia': wikipedia_count > 0,
        'has_wikidata': wikidata_count > 0,
        'has_twitter': twitter_count > 0,
        'wiki_count': wikipedia_count,
        'wd_count': wikidata_count,
        'tw_count': twitter_count
    })

# ---------- Create summary DataFrame ----------
df_stats = pd.DataFrame(results)

# Calculate success rate (Twitter coverage)
df_stats['success_rate'] = np.where(
    df_stats['wikipedia_count'] > 0,
    (df_stats['twitter_count'] / df_stats['wikipedia_count'] * 100).round(1),
    0
)

# Add totals row
totals = {
    'folder': 'TOTAL',
    'category_name': 'TOTAL',
    'wikipedia_count': df_stats['wikipedia_count'].sum(),
    'wikidata_count': df_stats['wikidata_count'].sum(),
    'twitter_count': df_stats['twitter_count'].sum(),
    'success_rate': 0  # Will calculate below
}
if totals['wikipedia_count'] > 0:
    totals['success_rate'] = round(totals['twitter_count'] / totals['wikipedia_count'] * 100, 1)

df_stats = pd.concat([df_stats, pd.DataFrame([totals])], ignore_index=True)

# ---------- Save to CSV ----------
output_path = os.path.join(POIS_DIR, "POI_statistics.csv")
df_stats.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*80}")
print(f"✅ Statistics saved to: {output_path}")
print(f"📊 Total categories: {len(results)}")
print(f"📖 Total Wikipedia entries: {totals['wikipedia_count']}")
print(f"🔗 Total Wikidata entries: {totals['wikidata_count']}")
print(f"🐦 Total Twitter users: {totals['twitter_count']}")
print(f"📈 Overall success rate: {totals['success_rate']}%")
print(f"{'='*80}\n")

# ---------- Display summary table ----------
display(df_stats)

# ---------- VISUALIZATION 1: Success Rate Bar Chart ----------
print("\n📊 Generating success rate bar chart...")

# Exclude TOTAL row for visualization
df_viz = df_stats[df_stats['folder'] != 'TOTAL'].copy()
df_viz = df_viz.sort_values('success_rate', ascending=False)

plt.figure(figsize=(14, 8))
bars = plt.bar(range(len(df_viz)), df_viz['success_rate'], 
               color='steelblue', alpha=0.8, edgecolor='black')

# Add value labels on bars
for i, (idx, row) in enumerate(df_viz.iterrows()):
    plt.text(i, row['success_rate'] + 1, f"{row['success_rate']:.1f}%", 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.xlabel('Category', fontsize=12, fontweight='bold')
plt.ylabel('Twitter Success Rate (%)', fontsize=12, fontweight='bold')
plt.title('Twitter Account Discovery Success Rate by Category\n(Twitter users / Wikipedia entries)', 
          fontsize=14, fontweight='bold', pad=20)
plt.xticks(range(len(df_viz)), df_viz['category_name'], rotation=45, ha='right', fontsize=10)
plt.ylim(0, max(df_viz['success_rate']) * 1.15)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()

chart_path = os.path.join(POIS_DIR, "figures")
os.makedirs(chart_path, exist_ok=True)
chart_file = os.path.join(chart_path, "twitter_success_rate.png")
plt.savefig(chart_file, dpi=150, bbox_inches='tight')
plt.show()
print(f"✅ Bar chart saved: {chart_file}")

# ---------- VISUALIZATION 2: Venn Diagram ----------
print("\n📊 Generating Venn diagram...")

# Calculate totals for Venn diagram
total_wiki = sum(e['wiki_count'] for e in all_entities)
total_wikidata = sum(e['wd_count'] for e in all_entities)
total_twitter = sum(e['tw_count'] for e in all_entities)

# Overlaps (simplified - using minimum counts as proxy)
wiki_wd = min(total_wiki, total_wikidata)
wiki_tw = min(total_wiki, total_twitter)
wd_tw = min(total_wikidata, total_twitter)
wiki_wd_tw = min(total_wiki, total_wikidata, total_twitter)

plt.figure(figsize=(10, 8))
venn_diagram = venn3(
    subsets=(
        total_wiki - wiki_wd - wiki_tw + wiki_wd_tw,  # Only Wikipedia
        total_wikidata - wiki_wd - wd_tw + wiki_wd_tw,  # Only Wikidata
        wiki_wd - wiki_wd_tw,  # Wiki + Wikidata
        total_twitter - wiki_tw - wd_tw + wiki_wd_tw,  # Only Twitter
        wiki_tw - wiki_wd_tw,  # Wiki + Twitter
        wd_tw - wiki_wd_tw,  # Wikidata + Twitter
        wiki_wd_tw  # All three
    ),
    set_labels=('Wikipedia', 'Wikidata', 'Twitter')
)

plt.title('Overlap Between Wikipedia, Wikidata, and Twitter Coverage\n(Total POI Entities)', 
          fontsize=14, fontweight='bold', pad=20)

venn_file = os.path.join(chart_path, "wikipedia_wikidata_twitter_venn.png")
plt.savefig(venn_file, dpi=150, bbox_inches='tight')
plt.show()
print(f"✅ Venn diagram saved: {venn_file}")

print(f"\n{'='*80}")
print("🎉 STEP 6 COMPLETE!")
print(f"{'='*80}")
print(f"📄 Summary table: {output_path}")
print(f"📊 Bar chart: {chart_file}")
print(f"📊 Venn diagram: {venn_file}")
print(f"{'='*80}")

## STEP 7: Scrape Twitter User Metadata Using Selenium

In this step, we scrape detailed metadata for each Twitter username found in our POI dataset using Selenium WebDriver. This approach directly accesses Twitter profile pages without API rate limits.

In [ ]:
"""
===============================================================================
STEP 7: Selenium-Based Twitter User Metadata Scraper
===============================================================================

This script scrapes Twitter user profiles using Selenium WebDriver to extract:
- Username, Display Name, Description (Bio)
- Followers, Following, Tweets count
- Location, Profile Image, Verification status
- Account status (active/suspended/protected/not found)

Features:
- Headless Chrome for faster execution
- Automatic retry logic with exponential backoff
- Random delays to avoid detection
- Robust error handling for missing elements
- Progress tracking with intermediate saves
- Works without Twitter API credentials

Author: Iran DS Project - STEP 7
Date: December 2024
===============================================================================
"""

import os
import re
import time
import random
import pandas as pd
from pathlib import Path
from datetime import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    TimeoutException, 
    NoSuchElementException, 
    WebDriverException
)
from webdriver_manager.chrome import ChromeDriverManager

# snscrape import for fallback (statuses_count and created_at)
try:
    import snscrape.modules.twitter as sntwitter
    SNSCRAPE_AVAILABLE = True
    print("✅ snscrape available for fallback data collection")
except ImportError:
    SNSCRAPE_AVAILABLE = False
    print("⚠️  snscrape not available - will skip statuses_count and created_at")

print("=" * 80)
print("🚀 TWITTER USER METADATA SCRAPER - SELENIUM")
print("=" * 80)
print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# ============================================================================
# CONFIGURATION
# ============================================================================

# Find Iran project root directory
def find_iran_root():
    """Locate the Iran directory containing POIs folder"""
    cwd = Path(os.getcwd())
    if (cwd / 'POIs').exists():
        return cwd
    for parent in cwd.parents:
        if (parent / 'POIs').exists():
            return parent
    raise FileNotFoundError("Could not find Iran directory with POIs folder")

IRAN_ROOT = find_iran_root()
POIS_DIR = IRAN_ROOT / 'POIs'
INPUT_FILE = POIS_DIR / 'Manual_Search_POIs_Unique.csv'
OUTPUT_FILE = POIS_DIR / 'POI_twitter_users_data.csv'
TEMP_FILE = POIS_DIR / 'POI_twitter_users_data_temp.csv'
FAILED_FILE = POIS_DIR / 'POI_twitter_scraping_failed.txt'

# Scraping parameters
MAX_RETRIES = 3
PAGE_LOAD_TIMEOUT = 15  # seconds
ELEMENT_WAIT_TIMEOUT = 10  # seconds
MIN_DELAY = 2  # minimum seconds between requests
MAX_DELAY = 5  # maximum seconds between requests
SAVE_INTERVAL = 25  # save progress every N users

print(f"📂 Project Root: {IRAN_ROOT}")
print(f"📂 POIs Directory: {POIS_DIR}")
print(f"📄 Input File: {INPUT_FILE.name}")
print(f"📄 Output File: {OUTPUT_FILE.name}\n")

# ============================================================================
# LOAD USERNAMES
# ============================================================================

def load_unique_usernames():
    """
    Load Twitter usernames from CSV and prepare for scraping
    
    Returns:
        list: Unique, cleaned Twitter usernames
    """
    if not INPUT_FILE.exists():
        raise FileNotFoundError(f"Input file not found: {INPUT_FILE}")
    
    print(f"📖 Reading usernames from: {INPUT_FILE.name}")
    df = pd.read_csv(INPUT_FILE, encoding='utf-8-sig')
    
    if 'Twitter_username' not in df.columns:
        raise ValueError(f"Column 'Twitter_username' not found in {INPUT_FILE.name}")
    
    # Extract usernames and clean
    usernames = df['Twitter_username'].dropna().astype(str)
    
    # Remove year-only entries (e.g., 2015, 2023)
    usernames = usernames[~usernames.str.fullmatch(r'\d{4}')]
    
    # Remove purely numeric entries
    usernames = usernames[~usernames.str.fullmatch(r'\d+')]
    
    # Remove duplicates and sort
    usernames = usernames.drop_duplicates().sort_values().reset_index(drop=True)
    
    print(f"✅ Loaded {len(usernames)} unique Twitter usernames\n")
    return usernames.tolist()

# ============================================================================
# SELENIUM SETUP
# ============================================================================

def setup_chrome_driver():
    """
    Initialize Chrome WebDriver with optimized options
    
    Returns:
        webdriver.Chrome: Configured Chrome driver instance
    """
    print("🔧 Setting up Chrome WebDriver...")
    
    chrome_options = Options()
    
    # Headless mode for faster execution
    chrome_options.add_argument('--headless=new')
    
    # Performance optimizations
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    
    # Disable images for faster loading
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')
    
    # Set user agent to avoid detection
    chrome_options.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/120.0.0.0 Safari/537.36'
    )
    
    # Additional preferences
    prefs = {
        'profile.default_content_setting_values': {
            'images': 2,  # Disable images
            'javascript': 1  # Enable JavaScript (required for Twitter)
        }
    }
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
        print("✅ Chrome WebDriver initialized successfully\n")
        return driver
    except Exception as e:
        print(f"❌ Failed to initialize Chrome WebDriver: {e}")
        raise

# ============================================================================
# SCRAPING FUNCTIONS
# ============================================================================

def check_driver_alive(driver):
    """
    Check if WebDriver session is still alive
    
    Args:
        driver: Selenium WebDriver instance
        
    Returns:
        bool: True if driver is alive, False otherwise
    """
    try:
        _ = driver.current_url
        return True
    except:
        return False

def extract_number_from_text(text):
    """
    Extract numeric value from text like '1,234' or '5.2K' or '1.2M'
    
    Args:
        text (str): Text containing a number
        
    Returns:
        int: Extracted number, or 0 if extraction fails
    """
    if not text:
        return 0
    
    try:
        # Remove commas
        text = text.replace(',', '').strip()
        
        # Handle K (thousands) and M (millions)
        if 'K' in text.upper():
            return int(float(text.upper().replace('K', '')) * 1000)
        elif 'M' in text.upper():
            return int(float(text.upper().replace('M', '')) * 1000000)
        else:
            # Extract first number found
            match = re.search(r'[\d.]+', text)
            if match:
                return int(float(match.group()))
    except:
        pass
    
    return 0

def scrape_twitter_user(driver, username, retry_count=0):
    """
    Scrape a single Twitter user's profile
    
    Args:
        driver: Selenium WebDriver instance
        username (str): Twitter username (without @)
        retry_count (int): Current retry attempt
        
    Returns:
        dict: User data dictionary, or None if failed
    """
    url = f"https://twitter.com/{username}"
    
    try:
        # Navigate to profile
        driver.get(url)
        
        # Wait for page to load and scroll to ensure all content loads
        time.sleep(random.uniform(2.5, 4.0))
        
        # Scroll down a bit to trigger lazy loading
        driver.execute_script("window.scrollTo(0, 500);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.5)
        
        # Initialize result dictionary
        result = {
            'username': username,
            'display_name': '',
            'description': '',
            'followers_count': 0,
            'following_count': 0,
            'statuses_count': 0,
            'location': '',
            'profile_image_url': '',
            'verified': False,
            'account_status': 'active'
        }
        
        # Get page source for status checks
        page_source_lower = driver.page_source.lower()
        
        # Check for suspended account (be more specific)
        if 'account suspended' in page_source_lower or 'has been suspended' in page_source_lower:
            result['account_status'] = 'suspended'
            return result
        
        # Check for not found (before protected, as it's terminal)
        if "this account doesn't exist" in page_source_lower or "this account doesn't exist" in page_source_lower:
            result['account_status'] = 'not_found'
            return result
        
        # Check for deactivated account
        if 'account deactivated' in page_source_lower or 'has been deactivated' in page_source_lower:
            result['account_status'] = 'deactivated'
            return result
        
        # Check for protected account (tweets are protected)
        if 'these tweets are protected' in page_source_lower or 'protected tweets' in page_source_lower:
            result['account_status'] = 'protected'
            # Continue to extract available data (name, bio, etc.)
        
        wait = WebDriverWait(driver, ELEMENT_WAIT_TIMEOUT)
        
        # Extract display name
        try:
            # Multiple selectors for display name
            selectors = [
                "//div[@data-testid='UserName']//span[1]",
                "//div[contains(@class, 'profile')]//span[contains(text(), '@')]/../preceding-sibling::span",
                "//h2[@role='heading']//span"
            ]
            for selector in selectors:
                try:
                    name_elem = driver.find_element(By.XPATH, selector)
                    if name_elem.text and not name_elem.text.startswith('@'):
                        result['display_name'] = name_elem.text.strip()
                        break
                except:
                    continue
        except:
            pass
        
        # Extract description/bio
        try:
            bio_elem = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']")
            result['description'] = bio_elem.text.strip()
        except:
            pass
        
        # Extract followers count
        try:
            followers_elem = driver.find_element(By.XPATH, "//a[contains(@href, '/verified_followers') or contains(@href, '/followers')]//span[1]")
            result['followers_count'] = extract_number_from_text(followers_elem.text)
        except:
            # Alternative selector
            try:
                followers_text = driver.find_element(By.XPATH, "//span[contains(text(), 'Followers')]/../span[1]").text
                result['followers_count'] = extract_number_from_text(followers_text)
            except:
                pass
        
        # Extract following count
        try:
            following_elem = driver.find_element(By.XPATH, "//a[contains(@href, '/following')]//span[1]")
            result['following_count'] = extract_number_from_text(following_elem.text)
        except:
            # Alternative selector
            try:
                following_text = driver.find_element(By.XPATH, "//span[contains(text(), 'Following')]/../span[1]").text
                result['following_count'] = extract_number_from_text(following_text)
            except:
                pass
        
        # Extract tweets/statuses count - Twitter API v2 style
        try:
            # Method 1: Look in the page source for JSON data
            page_source = driver.page_source
            
            # Twitter embeds user data in JSON format
            json_patterns = [
                r'"statuses_count"\s*:\s*(\d+)',
                r'"tweet_count"\s*:\s*(\d+)',
                r'\"statuses_count\\":(\d+)',
                r'\"tweet_count\\":(\d+)',
                r'"media_count"\s*:\s*(\d+)',  # Sometimes shown as media
            ]
            
            for pattern in json_patterns:
                match = re.search(pattern, page_source)
                if match:
                    count = int(match.group(1))
                    if count > 0:  # Make sure we found a real count
                        result['statuses_count'] = count
                        break
        except:
            pass
        
        # Method 2: If still 0, try scraping visible elements
        if result['statuses_count'] == 0:
            try:
                # Wait a bit more for content to load
                time.sleep(1)
                
                # Try to find any span with numbers that might be tweet count
                # Usually appears near followers/following
                all_spans = driver.find_elements(By.TAG_NAME, "span")
                
                # Look for pattern: number followed by "posts" or near followers
                for i, span in enumerate(all_spans):
                    text = span.text.strip()
                    if text and any(c.isdigit() for c in text):
                        # Check if next spans contain "Following" or "Followers"
                        context = ""
                        if i + 1 < len(all_spans):
                            context += all_spans[i + 1].text.lower()
                        if i + 2 < len(all_spans):
                            context += " " + all_spans[i + 2].text.lower()
                        
                        # If we see followers/following nearby, the first number might be posts
                        if "following" in context or "follower" in context:
                            potential_count = extract_number_from_text(text)
                            if potential_count > 0:
                                result['statuses_count'] = potential_count
                                break
            except:
                pass
        
        # Extract location
        try:
            location_elem = driver.find_element(By.XPATH, "//span[@data-testid='UserLocation']")
            result['location'] = location_elem.text.strip()
        except:
            pass
        
        # Extract profile image URL
        try:
            img_elem = driver.find_element(By.XPATH, "//div[@data-testid='UserAvatar-Container-unknown']//img | //a[contains(@href, '/photo')]//img")
            result['profile_image_url'] = img_elem.get_attribute('src')
        except:
            pass
        
        # Check for verification badge
        try:
            verified_elem = driver.find_element(By.XPATH, "//svg[@aria-label='Verified account']")
            result['verified'] = True
        except:
            pass
        
        return result
        
    except TimeoutException:
        if retry_count < MAX_RETRIES:
            print(f"      ⏱️  Timeout, retrying ({retry_count + 1}/{MAX_RETRIES})...")
            time.sleep(random.uniform(3, 6))
            return scrape_twitter_user(driver, username, retry_count + 1)
        else:
            print(f"      ❌ Failed after {MAX_RETRIES} retries (timeout)")
            return None
    
    except WebDriverException as e:
        # WebDriver crashed or session lost
        error_msg = str(e).lower()
        if 'invalid session' in error_msg or 'session' in error_msg:
            print(f"      🔄 WebDriver session lost - needs restart")
            return 'SESSION_LOST'
        else:
            print(f"      ❌ WebDriver error: {str(e)[:100]}")
            return None
    
    except Exception as e:
        print(f"      ❌ Error: {str(e)[:100]}")
        return None

# ============================================================================
# MAIN SCRAPING LOOP
# ============================================================================

def main():
    """Main execution function"""
    
    # Load usernames
    usernames = load_unique_usernames()
    total_users = len(usernames)
    
    # Initialize Chrome driver
    driver = setup_chrome_driver()
    
    # Storage for results
    results = []
    failed_usernames = []
    
    print("=" * 80)
    print(f"🚀 STARTING SCRAPING: {total_users} users to process")
    print("=" * 80)
    print()
    
    try:
        for idx, username in enumerate(usernames, 1):
            print(f"[{idx}/{total_users}] Scraping: @{username}")
            
            # Check if driver is still alive
            if not check_driver_alive(driver):
                print("   🔄 WebDriver session lost, restarting...")
                driver.quit()
                time.sleep(3)
                driver = setup_chrome_driver()
                print("   ✅ WebDriver restarted successfully")
            
            # Scrape user
            user_data = scrape_twitter_user(driver, username)
            
            # Handle session lost
            if user_data == 'SESSION_LOST':
                print("   🔄 Restarting WebDriver...")
                try:
                    driver.quit()
                except:
                    pass
                time.sleep(3)
                driver = setup_chrome_driver()
                print("   ✅ WebDriver restarted, retrying user...")
                
                # Retry scraping this user
                user_data = scrape_twitter_user(driver, username)
                if user_data == 'SESSION_LOST' or user_data is None:
                    print("   ❌ Failed again after restart")
                    failed_usernames.append(username)
                    continue
            
            if user_data:
                results.append(user_data)
                
                # Display status
                status = user_data['account_status']
                if status == 'active':
                    print(f"   ✅ {user_data['display_name']} | "
                          f"{user_data['followers_count']:,} followers | "
                          f"{user_data['following_count']:,} following")
                else:
                    print(f"   ⚠️  Account status: {status.upper()}")
            else:
                failed_usernames.append(username)
                print(f"   ❌ Failed to retrieve data")
            
            # Random delay between requests
            if idx < total_users:
                delay = random.uniform(MIN_DELAY, MAX_DELAY)
                time.sleep(delay)
            
            # Save intermediate results
            if idx % SAVE_INTERVAL == 0 and results:
                temp_df = pd.DataFrame(results)
                temp_df.to_csv(TEMP_FILE, index=False, encoding='utf-8-sig')
                print(f"   💾 Saved {len(results)} results to temp file")
                print()
        
        print("\n" + "=" * 80)
        print("📊 PROCESSING RESULTS")
        print("=" * 80)
        
        # Create final DataFrame
        if results:
            df_final = pd.DataFrame(results)
            
            # Sort by followers count (descending)
            df_final = df_final.sort_values('followers_count', ascending=False).reset_index(drop=True)
            
            # Save to CSV
            df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
            
            print(f"\n✅ Successfully scraped {len(df_final)} users")
            print(f"📁 Saved to: {OUTPUT_FILE}")
            
            # Print statistics
            print("\n" + "=" * 80)
            print("📈 SUMMARY STATISTICS")
            print("=" * 80)
            
            status_counts = df_final['account_status'].value_counts()
            print("\n🔍 Account Status Distribution:")
            for status, count in status_counts.items():
                print(f"   {status.upper()}: {count} ({count/len(df_final)*100:.1f}%)")
            
            active_users = df_final[df_final['account_status'] == 'active']
            if len(active_users) > 0:
                print(f"\n📊 Active Accounts Statistics:")
                print(f"   Total Followers: {active_users['followers_count'].sum():,}")
                print(f"   Average Followers: {active_users['followers_count'].mean():.0f}")
                print(f"   Median Followers: {active_users['followers_count'].median():.0f}")
                print(f"   Max Followers: {active_users['followers_count'].max():,}")
                print(f"   Verified Accounts: {active_users['verified'].sum()} ({active_users['verified'].sum()/len(active_users)*100:.1f}%)")
            
            # Display top 20 users
            print("\n" + "=" * 80)
            print("🏆 TOP 20 USERS BY FOLLOWERS")
            print("=" * 80)
            print()
            display(df_final[['username', 'display_name', 'followers_count', 'following_count', 'account_status']].head(20))
            
        else:
            print("\n❌ No user data collected")
        
        # Save failed usernames
        if failed_usernames:
            with open(FAILED_FILE, 'w', encoding='utf-8') as f:
                f.write(f"Failed to scrape {len(failed_usernames)} usernames:\n\n")
                for uname in failed_usernames:
                    f.write(f"{uname}\n")
            print(f"\n⚠️  Failed to scrape {len(failed_usernames)} usernames")
            print(f"📁 List saved to: {FAILED_FILE}")
        
        # Clean up temp file if exists
        if TEMP_FILE.exists():
            TEMP_FILE.unlink()
            print(f"\n🗑️  Cleaned up temporary file")
        
    finally:
        # Always close the driver
        driver.quit()
        print("\n🔒 Chrome WebDriver closed")
    
    print("\n" + "=" * 80)
    print("🎉 STEP 7 COMPLETE: Twitter User Metadata Collection")
    print(f"End Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

# Execute main function
if __name__ == "__main__" or True:  # Always run in notebook
    main()

## 7.1 Update Statuses Count with New Authentication

Now that we have a logged-in Twitter account, we can extract the posts count that appears below the username on profile pages.

In [ ]:
"""
Update Statuses Count (Posts) for all POIs
Using Selenium with authenticated session
"""

import os
import json
import time
import random
import re
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

print("=" * 80)
print("📊 UPDATING STATUSES COUNT FOR ALL POIs")
print("=" * 80)

# Paths
IRAN_ROOT = Path(r"c:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran")
POIS_DIR = IRAN_ROOT / 'POIs'
INPUT_CSV = POIS_DIR / 'POI_twitter_users_data.csv'
COOKIES_FILE = POIS_DIR / 'x_cookies.json'

print(f"📄 Input: {INPUT_CSV}")
print(f"🔐 Cookies: {COOKIES_FILE} ({'✅ Found' if COOKIES_FILE.exists() else '❌ Missing'})\n")

# Load data
df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
print(f"✅ Loaded {len(df)} users")
print(f"   Users with statuses_count = 0: {(df['statuses_count'] == 0).sum()}\n")

# Setup Selenium
print("🌐 Setting up Chrome WebDriver...")
options = Options()
options.add_argument('--headless=new')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(20)

# Load cookies
if COOKIES_FILE.exists():
    driver.get("https://twitter.com")
    time.sleep(2)
    
    with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
        cookies = json.load(f)
    
    for cookie in cookies:
        try:
            cookie.pop('sameSite', None)
            cookie.pop('storeId', None)
            driver.add_cookie(cookie)
        except:
            continue
    
    print("✅ Cookies loaded\n")
else:
    print("⚠️  No cookies found - results may be limited\n")

# Function to extract posts count from profile header
def extract_posts_count(driver, username, debug=False):
    """
    Extract posts count directly from Twitter profile header DOM element.
    The post count appears in a div with text ending in 'posts' (e.g., '45.8K posts')
    """
    try:
        url = f"https://twitter.com/{username}"
        driver.get(url)
        
        # Wait for page to load - looking specifically for the posts element
        time.sleep(5)
        
        if debug:
            print(f"\n[DEBUG] Loading profile: {url}")
        
        # Check if we hit a login/captcha page (Twitter blocking detection)
        page_source = driver.page_source.lower()
        if 'log in' in page_source and 'sign up' in page_source and 'posts' not in page_source:
            if debug:
                print(f"[DEBUG] ⚠️ Twitter login page detected - might be blocked!")
            return "BLOCKED"
        
        # Method 1: Find div element containing "posts" text in profile header
        # XPath: //div[contains(text(), 'posts')] or //div[contains(text(), 'Posts')]
        try:
            # Look for elements with "posts" text (case-insensitive by checking both)
            posts_elements = driver.find_elements(By.XPATH, 
                "//div[contains(translate(text(), 'POSTS', 'posts'), 'posts')]")
            
            if debug:
                print(f"[DEBUG] Found {len(posts_elements)} elements with 'posts' text")
            
            for elem in posts_elements:
                text = elem.text.strip()
                if debug:
                    print(f"[DEBUG] Checking element text: '{text}'")
                
                # Look for pattern: "45.8K posts" or "3,368 posts" or "123 posts"
                # Text should end with "posts" or "Posts"
                if text.lower().endswith('posts') or text.lower().endswith('post'):
                    # Extract the numeric part (everything before "posts")
                    count_str = text.lower().replace('posts', '').replace('post', '').strip()
                    count_str = count_str.replace(',', '').upper()
                    
                    if debug:
                        print(f"[DEBUG] Extracted count string: '{count_str}'")
                    
                    # Handle K, M, B suffixes
                    try:
                        if 'K' in count_str:
                            count = int(float(count_str.replace('K', '')) * 1000)
                        elif 'M' in count_str:
                            count = int(float(count_str.replace('M', '')) * 1000000)
                        elif 'B' in count_str:
                            count = int(float(count_str.replace('B', '')) * 1000000000)
                        else:
                            count = int(count_str)
                        
                        if debug:
                            print(f"[DEBUG] Successfully parsed: {text} → {count:,}")
                        
                        return count
                    except ValueError:
                        if debug:
                            print(f"[DEBUG] Could not parse '{count_str}' as number")
                        continue
        
        except Exception as e:
            if debug:
                print(f"[DEBUG] XPath search error: {e}")
        
        # Method 2: Fallback - search in all divs with dir="ltr" attribute
        try:
            ltr_divs = driver.find_elements(By.XPATH, "//div[@dir='ltr']")
            for div in ltr_divs:
                text = div.text.strip()
                if text.lower().endswith('posts') or text.lower().endswith('post'):
                    count_str = text.lower().replace('posts', '').replace('post', '').strip()
                    count_str = count_str.replace(',', '').upper()
                    
                    try:
                        if 'K' in count_str:
                            count = int(float(count_str.replace('K', '')) * 1000)
                        elif 'M' in count_str:
                            count = int(float(count_str.replace('M', '')) * 1000000)
                        elif 'B' in count_str:
                            count = int(float(count_str.replace('B', '')) * 1000000000)
                        else:
                            count = int(count_str)
                        
                        return count
                    except ValueError:
                        continue
        except:
            pass
        
        if debug:
            print(f"[DEBUG] Could not find posts count on page")
        
        return None
        
    except Exception as e:
        print(f"❌ Error: {str(e)[:60]}")
        return None

# Function to reload cookies and refresh session
def reload_cookies(driver, cookies_file):
    """Reload cookies to maintain authenticated session"""
    try:
        driver.get("https://twitter.com")
        time.sleep(2)
        
        # Clear existing cookies
        driver.delete_all_cookies()
        
        # Load fresh cookies
        with open(cookies_file, 'r', encoding='utf-8') as f:
            cookies = json.load(f)
        
        for cookie in cookies:
            try:
                cookie.pop('sameSite', None)
                cookie.pop('storeId', None)
                driver.add_cookie(cookie)
            except:
                continue
        
        time.sleep(2)
        return True
    except Exception as e:
        print(f"⚠️ Cookie reload failed: {e}")
        return False

# Process users
print("🔄 Processing users...\n")
updated_count = 0
failed_count = 0
skipped_count = 0

# Only update users with statuses_count = 0 OR NaN
users_to_update = df[(df['statuses_count'] == 0) | (df['statuses_count'].isna())].copy()
users_with_data = (df['statuses_count'] > 0).sum()

print(f"📊 Statistics:")
print(f"   ✅ Already have data: {users_with_data} users")
print(f"   🔄 Need to update: {len(users_to_update)} users")
print(f"   📝 Total users: {len(df)}\n")

for idx, row in users_to_update.iterrows():
    username = row['username']
    current_count = row['statuses_count']
    
    # Enable debug for first 3 users to see what's happening
    debug = (updated_count + failed_count) < 3
    
    print(f"[{updated_count + failed_count + 1}/{len(users_to_update)}] {username}...", end=" ")
    
    # Reload cookies every 5 users (more frequent!) to maintain session
    if (updated_count + failed_count) > 0 and (updated_count + failed_count) % 5 == 0:
        print("\n🔄 Reloading cookies to maintain session...", end=" ")
        if reload_cookies(driver, COOKIES_FILE):
            print("✅")
            time.sleep(3)  # Extra wait after cookie reload
        else:
            print("⚠️")
    
    # Restart browser every 15 users (more frequent!) to avoid detection
    if (updated_count + failed_count) > 0 and (updated_count + failed_count) % 15 == 0:
        print("\n🔄 Restarting browser to avoid detection...")
        try:
            driver.quit()
        except:
            pass
        time.sleep(5)  # Wait before reopening
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(20)
        if COOKIES_FILE.exists():
            driver.get("https://twitter.com")
            time.sleep(3)
            with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
                cookies = json.load(f)
            for cookie in cookies:
                try:
                    cookie.pop('sameSite', None)
                    cookie.pop('storeId', None)
                    driver.add_cookie(cookie)
                except:
                    continue
        print("✅ Browser restarted\n")
        time.sleep(3)  # Extra wait after restart
    
    # Extra long pause every 10 users to "cool down"
    if (updated_count + failed_count) > 0 and (updated_count + failed_count) % 10 == 0:
        cooldown = random.uniform(15, 25)
        print(f"\n❄️ Cooling down for {cooldown:.1f} seconds...")
        time.sleep(cooldown)
    
    posts_count = extract_posts_count(driver, username, debug=debug)
    
    # Check if we got blocked
    if posts_count == "BLOCKED":
        print("🚫 BLOCKED - Restarting browser completely...")
        try:
            driver.quit()
        except:
            pass
        time.sleep(20)  # Long wait after block
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(20)
        reload_cookies(driver, COOKIES_FILE)
        time.sleep(10)
        posts_count = extract_posts_count(driver, username, debug=False)
    
    if posts_count is not None and posts_count != "BLOCKED" and posts_count > 0:
        df.loc[df['username'] == username, 'statuses_count'] = posts_count
        print(f"✅ {posts_count:,} posts")
        updated_count += 1
    elif posts_count == 0:
        df.loc[df['username'] == username, 'statuses_count'] = 0
        print(f"✅ 0 posts (new account)")
        updated_count += 1
    else:
        print("❌ Not found")
        failed_count += 1
    
    # Very long random delay between requests (8-15 seconds)
    delay = random.uniform(8, 15)
    time.sleep(delay)
    
    # Save progress every 25 users
    if (updated_count + failed_count) % 25 == 0:
        df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')
        print(f"  💾 Progress saved ({updated_count} updated)\n")

# Close browser
driver.quit()
print("\n🔚 Browser closed")

# Save final results
df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')

print(f"\n{'='*80}")
print("✅ UPDATE COMPLETE!")
print(f"{'='*80}")
print(f"Updated:  {updated_count}")
print(f"Failed:   {failed_count}")
print(f"Remaining zeros: {(df['statuses_count'] == 0).sum()}")
print(f"\n📁 Saved to: {INPUT_CSV}")

# Show sample
print("\n📋 Sample of updated data:")
display(df[df['statuses_count'] > 0][['username', 'display_name', 'followers_count', 'statuses_count']].head(10))

## 7.2 Manual Cookie Update Helper

If automatic cookie export doesn't work, use this helper to convert cookies to the correct format.

In [ ]:
"""
Quick way to check if your cookies file is valid and working
"""

import json
from pathlib import Path

POIS_DIR = Path(r"c:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran\POIs")
COOKIES_FILE = POIS_DIR / 'x_cookies.json'

print("🔍 Checking cookies file...\n")

if not COOKIES_FILE.exists():
    print(f"❌ Cookies file not found: {COOKIES_FILE}")
    print("\n📝 To create cookies manually:")
    print("1. Install Chrome extension: Cookie-Editor")
    print("   Link: https://chrome.google.com/webstore/detail/hlkenndednhfkekhgcdicdfddnkalmdm")
    print("2. Go to https://twitter.com and login")
    print("3. Click Cookie-Editor icon → Export → Export as JSON")
    print(f"4. Save the JSON to: {COOKIES_FILE}")
else:
    try:
        with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
            cookies = json.load(f)
        
        print(f"✅ Cookies file found: {COOKIES_FILE}")
        print(f"   Total cookies: {len(cookies)}")
        
        # Check for important Twitter cookies
        important_cookies = ['auth_token', 'ct0', 'guest_id']
        found_cookies = []
        
        for cookie in cookies:
            if cookie.get('name') in important_cookies:
                found_cookies.append(cookie.get('name'))
        
        print(f"\n🔑 Important cookies found: {', '.join(found_cookies)}")
        
        if 'auth_token' in found_cookies:
            print("✅ Authentication token present - you should be logged in!")
        else:
            print("⚠️  No auth_token - cookies might be outdated or incomplete")
            print("   Please re-export cookies from a logged-in Twitter session")
        
        # Show sample cookie
        if cookies:
            print(f"\n📋 Sample cookie:")
            print(f"   Name: {cookies[0].get('name')}")
            print(f"   Domain: {cookies[0].get('domain')}")
            
    except json.JSONDecodeError as e:
        print(f"❌ Error reading cookies file - invalid JSON format")
        print(f"   Error: {str(e)}")
        print("\n   The file might be corrupted. Please re-export cookies.")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

## 7.3 Clean Deleted/Suspended Users

Remove users with 0 followers, 0 following, and 0 posts (likely deleted or suspended accounts)

In [ ]:
"""
Clean POI_twitter_users_data.csv by removing deleted/suspended users
These are users with 0 followers, 0 following, AND 0 statuses
"""

import pandas as pd
from pathlib import Path

print("=" * 80)
print("🧹 CLEANING DELETED/SUSPENDED USERS")
print("=" * 80)

# Paths
IRAN_ROOT = Path(r"c:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran")
POIS_DIR = IRAN_ROOT / 'POIs'
INPUT_CSV = POIS_DIR / 'POI_twitter_users_data.csv'

# Load data
df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
original_count = len(df)

print(f"\n📊 Original data: {original_count} users")

# Identify deleted/suspended users (all three fields are 0)
deleted_mask = (
    (df['followers_count'] == 0) & 
    (df['following_count'] == 0) & 
    (df['statuses_count'] == 0)
)

deleted_users = df[deleted_mask]
deleted_count = len(deleted_users)

print(f"\n🗑️  Found {deleted_count} deleted/suspended users:")
if deleted_count > 0:
    print("\nDeleted users:")
    for idx, row in deleted_users.iterrows():
        print(f"   - {row['username']} (@{row['display_name']})")

# Remove deleted users
df_clean = df[~deleted_mask].copy()
clean_count = len(df_clean)

print(f"\n✅ Cleaned data: {clean_count} users")
print(f"🗑️  Removed: {deleted_count} users")
print(f"📉 Reduction: {(deleted_count/original_count*100):.1f}%")

# Save cleaned data
df_clean.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')
print(f"\n💾 Saved to: {INPUT_CSV}")

# Show statistics of cleaned data
print(f"\n{'='*80}")
print("📊 CLEANED DATA STATISTICS")
print(f"{'='*80}")
print(f"Total users: {clean_count}")
print(f"Average followers: {df_clean['followers_count'].mean():,.0f}")
print(f"Average following: {df_clean['following_count'].mean():,.0f}")
print(f"Average posts: {df_clean['statuses_count'].mean():,.0f}")

# Show sample
print("\n📋 Sample of cleaned data:")
display(df_clean[['username', 'display_name', 'followers_count', 'following_count', 'statuses_count']].head(10))

## 7.4 Fix Missing Data for Incomplete Profiles

Re-scrape followers, following, and location for users with inconsistent data (e.g., 0 followers/following but has posts)

In [ ]:
"""
Fix Missing Data for Users with Incomplete Profiles
Re-scrape followers and following for users where:
followers = 0 AND following = 0 BUT statuses > 0 (doesn't make sense - data collection failed)
"""

import os
import json
import time
import random
import re
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

print("=" * 80)
print("🔧 FIXING INCOMPLETE USER PROFILES (Followers & Following Only)")
print("=" * 80)

# Paths
IRAN_ROOT = Path(r"c:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran")
POIS_DIR = IRAN_ROOT / 'POIs'
INPUT_CSV = POIS_DIR / 'POI_twitter_users_data.csv'
COOKIES_FILE = POIS_DIR / 'x_cookies.json'

print(f"📄 Input: {INPUT_CSV}")
print(f"🔐 Cookies: {COOKIES_FILE} ({'✅ Found' if COOKIES_FILE.exists() else '❌ Missing'})\n")

# Load data
df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
print(f"✅ Loaded {len(df)} users")

# Identify problematic users: 0 followers AND 0 following BUT have posts
problematic_mask = (
    (df['followers_count'] == 0) & 
    (df['following_count'] == 0) & 
    (df['statuses_count'] > 0)
)

users_to_fix = df[problematic_mask].copy()
print(f"🔍 Found {len(users_to_fix)} users with 0 followers/following but have posts\n")

if len(users_to_fix) == 0:
    print("✅ No users need fixing!")
else:
    # Setup Selenium
    print("🌐 Setting up Chrome WebDriver...")
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_page_load_timeout(20)

    # Load cookies
    if COOKIES_FILE.exists():
        driver.get("https://twitter.com")
        time.sleep(2)
        
        with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
            cookies = json.load(f)
        
        for cookie in cookies:
            try:
                cookie.pop('sameSite', None)
                cookie.pop('storeId', None)
                driver.add_cookie(cookie)
            except:
                continue
        
        print("✅ Cookies loaded\n")
    else:
        print("⚠️  No cookies found - results may be limited\n")

    # Function to extract profile data using XPath selectors
    def extract_profile_data(driver, username):
        """
        Extract followers and following counts from profile header using XPath.
        Numbers appear in <span> elements right before the "Followers"/"Following" labels.
        """
        try:
            url = f"https://twitter.com/{username}"
            driver.get(url)
            time.sleep(5)
            
            # Extract followers count using XPath
            followers = None
            try:
                # Find the span containing "Followers" text, then get the preceding sibling span
                followers_elements = driver.find_elements(By.XPATH, 
                    "//span[contains(text(), 'Followers')]/preceding-sibling::span[1]")
                
                if followers_elements:
                    followers_text = followers_elements[0].text.strip()
                    # Clean and parse: "7", "12.3K", "1.5M", etc.
                    count_str = followers_text.replace(',', '').upper()
                    
                    if 'K' in count_str:
                        followers = int(float(count_str.replace('K', '')) * 1000)
                    elif 'M' in count_str:
                        followers = int(float(count_str.replace('M', '')) * 1000000)
                    elif 'B' in count_str:
                        followers = int(float(count_str.replace('B', '')) * 1000000000)
                    else:
                        followers = int(count_str)
            except Exception as e:
                pass
            
            # Extract following count using XPath
            following = None
            try:
                # Find the span containing "Following" text, then get the preceding sibling span
                following_elements = driver.find_elements(By.XPATH, 
                    "//span[contains(text(), 'Following')]/preceding-sibling::span[1]")
                
                if following_elements:
                    following_text = following_elements[0].text.strip()
                    # Clean and parse: "7", "12.3K", "1.5M", etc.
                    count_str = following_text.replace(',', '').upper()
                    
                    if 'K' in count_str:
                        following = int(float(count_str.replace('K', '')) * 1000)
                    elif 'M' in count_str:
                        following = int(float(count_str.replace('M', '')) * 1000000)
                    elif 'B' in count_str:
                        following = int(float(count_str.replace('B', '')) * 1000000000)
                    else:
                        following = int(count_str)
            except Exception as e:
                pass
            
            return {
                'followers': followers,
                'following': following
            }
            
        except Exception as e:
            print(f"❌ Error: {str(e)[:60]}")
            return None

    # Function to reload cookies
    def reload_cookies(driver, cookies_file):
        """Reload cookies to maintain authenticated session"""
        try:
            driver.get("https://twitter.com")
            time.sleep(2)
            driver.delete_all_cookies()
            
            with open(cookies_file, 'r', encoding='utf-8') as f:
                cookies = json.load(f)
            
            for cookie in cookies:
                try:
                    cookie.pop('sameSite', None)
                    cookie.pop('storeId', None)
                    driver.add_cookie(cookie)
                except:
                    continue
            
            time.sleep(2)
            return True
        except Exception as e:
            print(f"⚠️ Cookie reload failed: {e}")
            return False

    # Process users
    print("🔄 Processing users...\n")
    fixed_count = 0
    failed_count = 0

    for idx, row in users_to_fix.iterrows():
        username = row['username']
        
        print(f"[{fixed_count + failed_count + 1}/{len(users_to_fix)}] {username}...", end=" ")
        
        # Reload cookies every 5 users
        if (fixed_count + failed_count) > 0 and (fixed_count + failed_count) % 5 == 0:
            print("\n🔄 Reloading cookies...", end=" ")
            if reload_cookies(driver, COOKIES_FILE):
                print("✅")
            else:
                print("⚠️")
        
        # Restart browser every 15 users
        if (fixed_count + failed_count) > 0 and (fixed_count + failed_count) % 15 == 0:
            print("\n🔄 Restarting browser...")
            try:
                driver.quit()
            except:
                pass
            time.sleep(5)
            driver = webdriver.Chrome(service=service, options=options)
            driver.set_page_load_timeout(20)
            reload_cookies(driver, COOKIES_FILE)
            print("✅ Browser restarted\n")
        
        # Extract data
        data = extract_profile_data(driver, username)
        
        if data:
            updated_fields = []
            
            # Update followers if missing or 0
            if data['followers'] is not None and row['followers_count'] == 0:
                df.loc[df['username'] == username, 'followers_count'] = data['followers']
                updated_fields.append(f"followers={data['followers']:,}")
            
            # Update following if missing or 0
            if data['following'] is not None and row['following_count'] == 0:
                df.loc[df['username'] == username, 'following_count'] = data['following']
                updated_fields.append(f"following={data['following']:,}")
        
        if data:
            updated_fields = []
            
            # Update followers if missing or 0
            if data['followers'] is not None and row['followers_count'] == 0:
                df.loc[df['username'] == username, 'followers_count'] = data['followers']
                updated_fields.append(f"followers={data['followers']:,}")
            
            # Update following if missing or 0
            if data['following'] is not None and row['following_count'] == 0:
                df.loc[df['username'] == username, 'following_count'] = data['following']
                updated_fields.append(f"following={data['following']:,}")
            
            if updated_fields:
                print(f"✅ {', '.join(updated_fields)}")
                fixed_count += 1
            else:
                print("⚠️ No updates needed")
        else:
            print("❌ Failed")
            failed_count += 1
        
        # Random delay
        time.sleep(random.uniform(8, 15))
        
        # Save progress every 10 users
        if (fixed_count + failed_count) % 10 == 0:
            df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')
            print(f"  💾 Progress saved\n")

    # Close browser
    driver.quit()
    print("\n🔚 Browser closed")

    # Save final results
    df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')

    print(f"\n{'='*80}")
    print("✅ FIX COMPLETE!")
    print(f"{'='*80}")
    print(f"Fixed:  {fixed_count}")
    print(f"Failed: {failed_count}")
    print(f"\n📁 Saved to: {INPUT_CSV}")

    # Show sample
    print("\n📋 Sample of fixed data:")
    display(df[df['username'].isin(users_to_fix['username'])][['username', 'followers_count', 'following_count', 'statuses_count']].head(10))

## 7.5  Re-verify Last 28 Updated Users

Double-check the accuracy of follower/following data for the last 28 users that were updated

In [ ]:
"""
Re-verify the last 28 updated users to ensure data accuracy
Uses the improved XPath extraction method
"""

import os
import json
import time
import random
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

print("=" * 80)
print("🔍 RE-VERIFYING LAST 28 UPDATED USERS")
print("=" * 80)

# Paths
IRAN_ROOT = Path(r"c:\Users\nitib\OneDrive\שולחן העבודה\Iran\Iran")
POIS_DIR = IRAN_ROOT / 'POIs'
INPUT_CSV = POIS_DIR / 'POI_twitter_users_data.csv'
COOKIES_FILE = POIS_DIR / 'x_cookies.json'

# Load data
df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
print(f"✅ Loaded {len(df)} users\n")

# Get the last 28 users that were updated (have followers > 0 and following > 0)
# These are likely the ones from the last run
recently_updated = df[(df['followers_count'] > 0) & (df['following_count'] > 0)].tail(28).copy()

print(f"📋 Re-verifying {len(recently_updated)} recently updated users:\n")
for idx, row in recently_updated.iterrows():
    print(f"   - {row['username']}: {row['followers_count']:,} followers, {row['following_count']:,} following")

print("\n🌐 Setting up Chrome WebDriver...")
options = Options()
options.add_argument('--headless=new')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(20)

# Load cookies
if COOKIES_FILE.exists():
    driver.get("https://twitter.com")
    time.sleep(2)
    
    with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
        cookies = json.load(f)
    
    for cookie in cookies:
        try:
            cookie.pop('sameSite', None)
            cookie.pop('storeId', None)
            driver.add_cookie(cookie)
        except:
            continue
    
    print("✅ Cookies loaded\n")

# Function to extract with multiple methods (robust approach)
def extract_profile_data_xpath(driver, username, debug=False):
    """Extract followers and following using multiple fallback methods"""
    try:
        url = f"https://twitter.com/{username}"
        driver.get(url)
        time.sleep(6)
        
        if debug:
            print(f"\n[DEBUG] Loading: {url}")
        
        # Get page text for debugging and fallback
        page_text = driver.find_element(By.TAG_NAME, 'body').text
        
        if debug:
            print(f"[DEBUG] Page loaded, text length: {len(page_text)}")
        
        # METHOD 1: XPath with preceding-sibling
        followers = None
        following = None
        
        try:
            # Try to find using XPath
            followers_elements = driver.find_elements(By.XPATH, 
                "//span[contains(text(), 'Followers')]/preceding-sibling::span[1]")
            
            if followers_elements and followers_elements[0].text.strip():
                followers_text = followers_elements[0].text.strip().replace(',', '').upper()
                if debug:
                    print(f"[DEBUG] XPath Followers text: '{followers_text}'")
                
                if 'K' in followers_text:
                    followers = int(float(followers_text.replace('K', '')) * 1000)
                elif 'M' in followers_text:
                    followers = int(float(followers_text.replace('M', '')) * 1000000)
                elif 'B' in followers_text:
                    followers = int(float(followers_text.replace('B', '')) * 1000000000)
                else:
                    followers = int(followers_text)
        except Exception as e:
            if debug:
                print(f"[DEBUG] XPath followers failed: {e}")
        
        # METHOD 2: If XPath failed, try finding in all divs containing "Followers"
        if followers is None:
            try:
                import re
                match = re.search(r'([\d,\.]+[KMB]?)\s+Followers', page_text, re.IGNORECASE)
                if match:
                    followers_text = match.group(1).replace(',', '').upper()
                    if debug:
                        print(f"[DEBUG] Regex Followers text: '{followers_text}'")
                    
                    if 'K' in followers_text:
                        followers = int(float(followers_text.replace('K', '')) * 1000)
                    elif 'M' in followers_text:
                        followers = int(float(followers_text.replace('M', '')) * 1000000)
                    elif 'B' in followers_text:
                        followers = int(float(followers_text.replace('B', '')) * 1000000000)
                    else:
                        followers = int(followers_text)
            except Exception as e:
                if debug:
                    print(f"[DEBUG] Regex followers failed: {e}")
        
        # Extract following with same dual approach
        try:
            following_elements = driver.find_elements(By.XPATH, 
                "//span[contains(text(), 'Following')]/preceding-sibling::span[1]")
            
            if following_elements and following_elements[0].text.strip():
                following_text = following_elements[0].text.strip().replace(',', '').upper()
                if debug:
                    print(f"[DEBUG] XPath Following text: '{following_text}'")
                
                if 'K' in following_text:
                    following = int(float(following_text.replace('K', '')) * 1000)
                elif 'M' in following_text:
                    following = int(float(following_text.replace('M', '')) * 1000000)
                elif 'B' in following_text:
                    following = int(float(following_text.replace('B', '')) * 1000000000)
                else:
                    following = int(following_text)
        except Exception as e:
            if debug:
                print(f"[DEBUG] XPath following failed: {e}")
        
        # Fallback for following
        if following is None:
            try:
                import re
                match = re.search(r'([\d,\.]+[KMB]?)\s+Following', page_text, re.IGNORECASE)
                if match:
                    following_text = match.group(1).replace(',', '').upper()
                    if debug:
                        print(f"[DEBUG] Regex Following text: '{following_text}'")
                    
                    if 'K' in following_text:
                        following = int(float(following_text.replace('K', '')) * 1000)
                    elif 'M' in following_text:
                        following = int(float(following_text.replace('M', '')) * 1000000)
                    elif 'B' in following_text:
                        following = int(float(following_text.replace('B', '')) * 1000000000)
                    else:
                        following = int(following_text)
            except Exception as e:
                if debug:
                    print(f"[DEBUG] Regex following failed: {e}")
        
        if debug:
            print(f"[DEBUG] Final result: followers={followers}, following={following}")
        
        return {'followers': followers, 'following': following}
    except Exception as e:
        if debug:
            print(f"[DEBUG] Overall error: {e}")
        return None

# Reload cookies function
def reload_cookies(driver, cookies_file):
    try:
        driver.get("https://twitter.com")
        time.sleep(2)
        driver.delete_all_cookies()
        
        with open(cookies_file, 'r', encoding='utf-8') as f:
            cookies = json.load(f)
        
        for cookie in cookies:
            try:
                cookie.pop('sameSite', None)
                cookie.pop('storeId', None)
                driver.add_cookie(cookie)
            except:
                continue
        time.sleep(2)
        return True
    except:
        return False

# Re-verify users
print("🔄 Re-verifying users...\n")
verified_count = 0
updated_count = 0
unchanged_count = 0

for idx, row in recently_updated.iterrows():
    username = row['username']
    old_followers = row['followers_count']
    old_following = row['following_count']
    
    # Enable debug for first 2 users
    debug = verified_count < 2
    
    print(f"[{verified_count + 1}/{len(recently_updated)}] {username}...", end=" ")
    
    # Reload cookies every 5 users
    if verified_count > 0 and verified_count % 5 == 0:
        print("\n🔄 Reloading cookies...", end=" ")
        reload_cookies(driver, COOKIES_FILE)
        print("✅")
    
    # Extract new data
    data = extract_profile_data_xpath(driver, username, debug=debug)
    
    if data and data['followers'] is not None and data['following'] is not None:
        new_followers = data['followers']
        new_following = data['following']
        
        # Check if data changed
        if new_followers != old_followers or new_following != old_following:
            df.loc[df['username'] == username, 'followers_count'] = new_followers
            df.loc[df['username'] == username, 'following_count'] = new_following
            print(f"🔄 UPDATED: {old_followers:,}→{new_followers:,} followers, {old_following:,}→{new_following:,} following")
            updated_count += 1
        else:
            print(f"✅ Verified: {new_followers:,} followers, {new_following:,} following")
            unchanged_count += 1
        
        verified_count += 1
    else:
        print("❌ Failed to extract")
    
    # Delay
    time.sleep(random.uniform(8, 12))
    
    # Save progress every 10 users
    if verified_count % 10 == 0:
        df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')
        print(f"  💾 Progress saved\n")

# Close browser
driver.quit()
print("\n🔚 Browser closed")

# Save final results
df.to_csv(INPUT_CSV, index=False, encoding='utf-8-sig')

print(f"\n{'='*80}")
print("✅ RE-VERIFICATION COMPLETE!")
print(f"{'='*80}")
print(f"Verified: {verified_count}")
print(f"Updated:  {updated_count} (data changed)")
print(f"Unchanged: {unchanged_count} (data correct)")
print(f"\n📁 Saved to: {INPUT_CSV}")

## Stage 7.6: Re-categorize "Candidates" based on Occupation

The "Candidates" category contains 186 POIs from various fields. We'll re-categorize them based on their wikidata_occupation.

In [ ]:
# ============================================
# Stage 7.5: Re-categorize Candidates
# ============================================

import pandas as pd

# Load files
manual_pois = pd.read_csv(POIS_DIR / "Manual_Search_POIs_Unique.csv", encoding='utf-8')
print(f"Total POIs: {len(manual_pois)}")
print(f"Candidates before re-categorization: {len(manual_pois[manual_pois['source_folder'] == 'Candidates'])}")

# Load all wikidata files to get occupation data
occupation_mapping = {}

wikidata_files = list(POIS_DIR.glob('*/*_wikidata_detailed_with_twitter.csv'))
print(f"\nLoading occupation data from {len(wikidata_files)} wikidata files...")

for file in wikidata_files:
    try:
        df_wiki = pd.read_csv(file, encoding='utf-8')
        if 'Twitter_username' in df_wiki.columns and 'wikidata_occupation' in df_wiki.columns:
            for _, row in df_wiki.iterrows():
                username = row['Twitter_username']
                occupation = row['wikidata_occupation']
                if pd.notna(username) and pd.notna(occupation):
                    occupation_mapping[username] = occupation
    except Exception as e:
        continue

print(f"Loaded occupation data for {len(occupation_mapping)} users")

# Define occupation to category mapping
occupation_to_category = {
    'journalist': 'iranian_journalists',
    'reporter': 'iranian_journalists',
    'news reader': 'iranian_journalists',
    'correspondent': 'iranian_journalists',
    
    'politician': 'iranian_politicians',
    'political activist': 'iranian_political_activists',
    'human rights activist': 'iranian_human_rights_activists',
    'activist': 'iranian_activists',
    'feminist': 'iranian_feminists',
    'dissident': 'iranian_dissidents',
    
    'actor': 'iranian_actors',
    'film actor': 'iranian_film_actors',
    'television actor': 'iranian_television_actors',
    
    'singer': 'iranian_singers',
    'pop singer': 'iranian_pop_singers',
    'rapper': 'iranian_rappers',
    
    'athlete': 'iranian_athletes',
    'footballer': 'iranian_footballers',
    'football manager': 'iranian_football_managers',
    'volleyball player': 'iranian_volleyball_players',
    'wrestler': 'iranian_wrestlers',
    'weightlifter': 'iranian_weightlifters',
    'taekwondo practitioner': 'iranian_taekwondo_practitioners',
    
    'physician': 'iranian_physicians',
    'cardiologist': 'iranian_cardiologists',
    
    'scientist': 'iranian_scientists',
    'economist': 'iranian_economists',
    'writer': 'iranian_writers',
    'blogger': 'iranian_bloggers',
    'comedian': 'iranian_comedians',
    'editor': 'iranian_editors',
    'ayatollah': 'iranian_ayatollahs',
}

# Re-categorize Candidates
recategorized_count = 0
unchanged_count = 0
no_occupation_count = 0

for idx, row in manual_pois.iterrows():
    if row['source_folder'] == 'Candidates':
        username = row['Twitter_username']
        
        # Check if we have occupation data
        if username in occupation_mapping:
            occupations = occupation_mapping[username].lower()
            new_category = None
            
            # Try to match occupation to category
            for occ_keyword, category in occupation_to_category.items():
                if occ_keyword in occupations:
                    new_category = category
                    break
            
            if new_category:
                manual_pois.at[idx, 'source_folder'] = new_category
                recategorized_count += 1
            else:
                unchanged_count += 1
        else:
            no_occupation_count += 1

print(f"\n{'='*70}")
print("RE-CATEGORIZATION RESULTS")
print(f"{'='*70}")
print(f"Successfully re-categorized: {recategorized_count}")
print(f"No matching category found: {unchanged_count}")
print(f"No occupation data: {no_occupation_count}")
print(f"\nCandidates remaining: {len(manual_pois[manual_pois['source_folder'] == 'Candidates'])}")

# Save updated file
output_file = POIS_DIR / "Manual_Search_POIs_Unique_Recategorized.csv"
manual_pois.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n✓ Saved to: {output_file}")

# Show category distribution after re-categorization
print(f"\n{'='*70}")
print("CATEGORY DISTRIBUTION AFTER RE-CATEGORIZATION")
print(f"{'='*70}")
category_counts = manual_pois['source_folder'].value_counts().head(15)
display(category_counts.to_frame('Count'))

## Stage 8: Statistical & Descriptive Analysis of POIs

Compute overall statistics, category summaries, and occupation distribution

In [ ]:
# ============================================
# Stage 8: Statistical & Descriptive Analysis
# Input:  POI_twitter_users_data.csv + Manual_Search_POIs_Unique.csv + wikidata files
# Output: POI_twitter_users_statistics.csv
# ============================================

import pandas as pd
import numpy as np
from pathlib import Path

# File paths
twitter_data_csv = POIS_DIR / "POI_twitter_users_data.csv"
manual_pois_csv = POIS_DIR / "Manual_Search_POIs_Unique_Recategorized.csv"
output_csv = POIS_DIR / "POI_twitter_users_statistics.csv"

print("=" * 70)
print("STAGE 8: STATISTICAL & DESCRIPTIVE ANALYSIS OF POIs")
print("=" * 70)

# ============================================
# Step 1: Load and merge data with categories and occupations
# ============================================
print("\n1. Loading and merging data...")

# Load Twitter data
df_twitter = pd.read_csv(twitter_data_csv, encoding='utf-8')
print(f"   Loaded {len(df_twitter)} Twitter user records")

# Load Manual POIs with categories
df_manual = pd.read_csv(manual_pois_csv, encoding='utf-8')
print(f"   Loaded {len(df_manual)} manual POI records")

# Merge to get categories
df = df_twitter.merge(
    df_manual[['Twitter_username', 'source_folder']].drop_duplicates(),
    left_on='username',
    right_on='Twitter_username',
    how='left'
)
df.rename(columns={'source_folder': 'category'}, inplace=True)
df.drop(columns=['Twitter_username'], inplace=True, errors='ignore')

# Load occupation data from wikidata files
print("   Loading occupation data from wikidata files...")
occupation_data = []

# Find all wikidata_detailed_with_twitter.csv files
wikidata_files = list(POIS_DIR.glob('*/*_wikidata_detailed_with_twitter.csv'))
print(f"   Found {len(wikidata_files)} wikidata files")

for file in wikidata_files:
    try:
        df_wiki = pd.read_csv(file, encoding='utf-8')
        if 'Twitter_username' in df_wiki.columns and 'wikidata_occupation' in df_wiki.columns:
            occupation_data.append(df_wiki[['Twitter_username', 'wikidata_occupation']])
    except Exception as e:
        continue

if occupation_data:
    df_occupations = pd.concat(occupation_data, ignore_index=True)
    df_occupations = df_occupations.drop_duplicates(subset=['Twitter_username'])
    print(f"   Loaded {len(df_occupations)} occupation records")
    
    # Merge occupation data
    df = df.merge(
        df_occupations,
        left_on='username',
        right_on='Twitter_username',
        how='left'
    )
    df.drop(columns=['Twitter_username'], inplace=True, errors='ignore')
else:
    df['wikidata_occupation'] = None
    print("   No occupation data found")

# Remove duplicates
df = df.drop_duplicates(subset=['username'])
print(f"   After merging and deduplication: {len(df)} unique POIs")

# ============================================
# Step 2: Compute overall metrics
# ============================================
print("\n2. Computing overall metrics...")

total_pois = len(df)
avg_followers = df['followers_count'].mean()
avg_following = df['following_count'].mean()
avg_posts = df['statuses_count'].mean()

# Round to whole numbers
avg_followers = round(avg_followers) if pd.notna(avg_followers) else 0
avg_following = round(avg_following) if pd.notna(avg_following) else 0
avg_posts = round(avg_posts) if pd.notna(avg_posts) else 0

overall_stats = {
    'Total POIs': total_pois,
    'Average Followers': avg_followers,
    'Average Following': avg_following,
    'Average Posts': avg_posts
}

print(f"\n   Overall Statistics:")
print(f"   - Total POIs: {total_pois}")
print(f"   - Average Followers: {avg_followers}")
print(f"   - Average Following: {avg_following}")
print(f"   - Average Posts: {avg_posts}")

# ============================================
# Step 3: Summary Table 1 - By Category
# ============================================
print("\n3. Building Summary Table 1 (by category)...")

# Filter out rows without category
df_with_category = df[df['category'].notna()].copy()

summary_by_category = df_with_category.groupby('category').agg(
    Num_POIs=('username', 'count'),
    Avg_Followers=('followers_count', 'mean'),
    Avg_Following=('following_count', 'mean'),
    Avg_Posts=('statuses_count', 'mean')
).reset_index()

# Round averages to whole numbers
summary_by_category['Avg_Followers'] = summary_by_category['Avg_Followers'].round().astype(int)
summary_by_category['Avg_Following'] = summary_by_category['Avg_Following'].round().astype(int)
summary_by_category['Avg_Posts'] = summary_by_category['Avg_Posts'].round().astype(int)

# Sort by number of POIs descending
summary_by_category = summary_by_category.sort_values('Num_POIs', ascending=False)

print(f"   Created table with {len(summary_by_category)} categories")

# ============================================
# Step 4: Summary Table 2 - Occupation Distribution
# ============================================
print("\n4. Building Summary Table 2 (occupation distribution)...")

# Split occupations and count
occupation_counts = {}

for idx, row in df.iterrows():
    occupations = row['wikidata_occupation']
    
    if pd.isna(occupations) or occupations == '':
        continue
    
    # Split by semicolon and count each occupation
    for occupation in str(occupations).split(';'):
        occupation = occupation.strip()
        if occupation:
            occupation_counts[occupation] = occupation_counts.get(occupation, 0) + 1

# Create dataframe
occupation_data_list = []
for occupation, count in occupation_counts.items():
    percentage = round((count / total_pois) * 100, 1)
    occupation_data_list.append({
        'Occupation': occupation,
        'Count': count,
        'Percentage': percentage
    })

occupation_df = pd.DataFrame(occupation_data_list)

# Sort by count descending
occupation_df = occupation_df.sort_values('Count', ascending=False)

print(f"   Found {len(occupation_df)} unique occupations")

# ============================================
# Step 5: Save to CSV with sections
# ============================================
print("\n5. Saving results to CSV...")

with open(output_csv, 'w', encoding='utf-8') as f:
    # Overall Statistics Section
    f.write("=" * 70 + "\n")
    f.write("OVERALL STATISTICS\n")
    f.write("=" * 70 + "\n")
    for key, value in overall_stats.items():
        f.write(f"{key},{value}\n")
    f.write("\n")
    
    # Table 1: Summary by Category
    f.write("=" * 70 + "\n")
    f.write("TABLE 1: SUMMARY BY CATEGORY\n")
    f.write("=" * 70 + "\n")
    summary_by_category.to_csv(f, index=False)
    f.write("\n")
    
    # Table 2: Occupation Distribution
    f.write("=" * 70 + "\n")
    f.write("TABLE 2: OCCUPATION DISTRIBUTION\n")
    f.write("=" * 70 + "\n")
    occupation_df.to_csv(f, index=False)

print(f"   ✓ Saved to: {output_csv}")

# ============================================
# Step 6: Display Results
# ============================================
print("\n6. Displaying results...")
print("\n" + "=" * 70)
print("OVERALL STATISTICS")
print("=" * 70)
for key, value in overall_stats.items():
    print(f"{key:25s}: {value}")

print("\n" + "=" * 70)
print("TABLE 1: SUMMARY BY CATEGORY")
print("=" * 70)
display(summary_by_category)

print("\n" + "=" * 70)
print("TABLE 2: OCCUPATION DISTRIBUTION (Top 20)")
print("=" * 70)
display(occupation_df.head(20))

print("\n" + "=" * 70)
print("✓ STAGE 8 COMPLETED SUCCESSFULLY")
print("=" * 70)
print(f"\nOutput file: {output_csv}")
print(f"Total POIs analyzed: {total_pois}")
print(f"Categories found: {len(summary_by_category)}")
print(f"Unique occupations found: {len(occupation_df)}")

## Stage 9 - Collect Followers & Following Lists

**Using WORKING METHOD: Extract @usernames from span elements**

Test mode: 3 POIs only

In [ ]:
# ============================================
# Stage 9: Collect Followers & Following Lists
# Using WORKING METHOD: Extract @usernames from spans
# ============================================

import pandas as pd
import time
import random
import json
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

print("=" * 70)
print("STAGE 9: COLLECT FOLLOWERS & FOLLOWING LISTS")
print("=" * 70)

# Configuration
POI_DATA_CSV = POIS_DIR / "POI_twitter_users_data.csv"
CANDIDATES_DIR = POIS_DIR / "Candidates"
CONNECTIONS_CSV = CANDIDATES_DIR / "POIs_candidate_connections.csv"
STATISTICS_CSV = CANDIDATES_DIR / "Candidates_statistics.csv"
COOKIES_FILE = POIS_DIR / "x_cookies.json"

CANDIDATES_DIR.mkdir(exist_ok=True)

MAX_SCROLLS = 10  # Reduced for testing
SCROLL_PAUSE = 2
REQUEST_DELAY = (3, 5)

print(f"\n⚙️  Configuration:")
print(f"   Max scrolls per list: {MAX_SCROLLS}")
print(f"   Scroll pause: {SCROLL_PAUSE}s")
print(f"   Request delay: {REQUEST_DELAY}s")

# ============================================
# Helper Function - WORKING METHOD
# ============================================

def collect_usernames_from_page(driver, username, page_type="followers"):
    """
    Collect usernames using WORKING METHOD:
    Find all span elements with text starting with @
    """
    url = f"https://twitter.com/{username}/{page_type}"
    
    print(f"      🌐 URL: {url}")
    driver.get(url)
    time.sleep(3)
    
    # Check for redirect to login
    if "login" in driver.current_url.lower():
        print(f"      ❌ Redirected to login - authentication failed")
        return []
    
    collected_usernames = set()
    
    print(f"      📜 Scrolling and collecting...")
    for scroll_num in range(MAX_SCROLLS):
        # WORKING METHOD: Get all spans and filter for @username
        all_spans = driver.find_elements(By.TAG_NAME, "span")
        
        for span in all_spans:
            try:
                text = span.text.strip()
                if text.startswith('@') and len(text) > 1:
                    username_clean = text[1:].lower()
                    # Skip the target user themselves
                    if username_clean != username.lower():
                        collected_usernames.add(username_clean)
            except:
                pass
        
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE)
        
        # Show progress every 3 scrolls
        if (scroll_num + 1) % 3 == 0:
            print(f"      ... Scroll {scroll_num + 1}/{MAX_SCROLLS}, found {len(collected_usernames)} so far")
    
    return list(collected_usernames)

# ============================================
# Step 1: Load POIs
# ============================================
print(f"\n📂 Step 1: Loading POI data...")

df_pois = pd.read_csv(POI_DATA_CSV, encoding='utf-8')
df_active = df_pois[df_pois['account_status'] == 'active'].copy()

# For testing, use only first 3 POIs
target_users = df_active['username'].dropna().unique()[:3]  # TEST WITH 3 USERS

print(f"   ✓ Loaded {len(df_pois)} POIs")
print(f"   ✓ Active users: {len(df_active)}")
print(f"   🧪 TEST MODE: Processing only first {len(target_users)} users")
print(f"   Users: {', '.join(target_users)}")

# ============================================
# Step 2: Initialize Browser
# ============================================
print(f"\n🌐 Step 2: Initializing browser...")

options = Options()
options.add_argument('--headless')  # Run in background
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Load cookies
driver.get("https://twitter.com")
time.sleep(2)

with open(COOKIES_FILE, 'r') as f:
    cookies = json.load(f)

for cookie in cookies:
    if 'sameSite' in cookie and cookie['sameSite'] not in ['Strict', 'Lax', 'None']:
        cookie['sameSite'] = 'None'
    driver.add_cookie(cookie)

print("   ✓ Browser initialized with cookies")

# ============================================
# Step 3: Collect Connections
# ============================================
print(f"\n🔍 Step 3: Collecting connections...")
print("=" * 70)

all_connections = []
processed_count = 0
failed_count = 0

for idx, username in enumerate(target_users, 1):
    print(f"\n[{idx}/{len(target_users)}] 👤 Processing: @{username}")
    print("-" * 70)
    
    try:
        # Collect FOLLOWERS
        print(f"   📥 Collecting FOLLOWERS...")
        followers = collect_usernames_from_page(driver, username, "followers")
        print(f"   ✅ Found {len(followers)} followers")
        
        # Add to connections list
        for follower in followers:
            all_connections.append({
                'target_username': username,
                'other_username': follower,
                'type': 'follower'
            })
        
        # Delay before next request
        delay = random.uniform(*REQUEST_DELAY)
        print(f"   ⏳ Waiting {delay:.1f}s before next request...")
        time.sleep(delay)
        
        # Collect FOLLOWING
        print(f"   📤 Collecting FOLLOWING...")
        following = collect_usernames_from_page(driver, username, "following")
        print(f"   ✅ Found {len(following)} following")
        
        # Add to connections list
        for followed in following:
            all_connections.append({
                'target_username': username,
                'other_username': followed,
                'type': 'following'
            })
        
        processed_count += 1
        print(f"   ✅ Completed @{username}")
        print(f"   📊 Total connections so far: {len(all_connections)}")
        
        # Delay before next user
        delay = random.uniform(*REQUEST_DELAY)
        print(f"   ⏳ Waiting {delay:.1f}s before next user...")
        time.sleep(delay)
        
    except Exception as e:
        print(f"   ❌ ERROR processing @{username}: {str(e)}")
        failed_count += 1
        continue

driver.quit()
print("\n" + "=" * 70)
print("🏁 COLLECTION COMPLETED")
print("=" * 70)
print(f"✅ Successfully processed: {processed_count} POIs")
print(f"❌ Failed: {failed_count} POIs")
print(f"📊 Total connections: {len(all_connections)}")

# ============================================
# Step 4: Save Results
# ============================================
print(f"\n💾 Step 4: Saving results...")

if len(all_connections) > 0:
    df_connections = pd.DataFrame(all_connections)
    df_connections = df_connections.drop_duplicates()
    df_connections.to_csv(CONNECTIONS_CSV, index=False, encoding='utf-8')
    
    print(f"   ✅ Saved {len(df_connections)} unique connections")
    print(f"   📁 File: {CONNECTIONS_CSV}")
    
    # Show preview
    print(f"\n📋 Preview of connections:")
    display(df_connections.head(20))
    
    # Statistics
    print(f"\n📊 Breakdown:")
    print(f"   Followers: {len(df_connections[df_connections['type']=='follower'])}")
    print(f"   Following: {len(df_connections[df_connections['type']=='following'])}")
else:
    print("   ⚠️  No connections collected!")

print("\n" + "=" * 70)
print("✅ Stage 9 TEST completed - Check results above!")
print("=" * 70)

## Stage 9 - FULL RUN: All 343 POIs

**With Safety Measures:**
- Intermediate saves every 5 POIs
- Browser restart every 20 POIs
- Long delays between requests
- Stop if blocked/redirected to login

In [ ]:
# ============================================
# Stage 9: FULL RUN - All POIs with Safety Measures
# ============================================

import pandas as pd
import time
import random
import json
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

print("=" * 70)
print("STAGE 9: FULL RUN - COLLECT ALL FOLLOWERS & FOLLOWING")
print("=" * 70)

# Configuration
POI_DATA_CSV = POIS_DIR / "POI_twitter_users_data.csv"
CANDIDATES_DIR = POIS_DIR / "Candidates"
CONNECTIONS_CSV = CANDIDATES_DIR / "POIs_candidate_connections.csv"
STATISTICS_CSV = CANDIDATES_DIR / "Candidates_statistics.csv"
COOKIES_FILE = POIS_DIR / "x_cookies.json"

CANDIDATES_DIR.mkdir(exist_ok=True)

# SAFETY CONFIGURATION
MAX_SCROLLS = 15  # Reasonable limit per page
SCROLL_PAUSE = 2.5  # Longer pause between scrolls
REQUEST_DELAY = (5, 8)  # Longer delays between requests
SAVE_INTERVAL = 5  # Save every 5 POIs
RESTART_INTERVAL = 20  # Restart browser every 20 POIs
COOLDOWN_AFTER_ERROR = 30  # Wait 30s after error

print(f"\n⚙️  SAFETY CONFIGURATION:")
print(f"   Max scrolls per list: {MAX_SCROLLS}")
print(f"   Scroll pause: {SCROLL_PAUSE}s")
print(f"   Request delay: {REQUEST_DELAY[0]}-{REQUEST_DELAY[1]}s")
print(f"   Save interval: Every {SAVE_INTERVAL} POIs")
print(f"   Browser restart: Every {RESTART_INTERVAL} POIs")
print(f"   Error cooldown: {COOLDOWN_AFTER_ERROR}s")

# ============================================
# Helper Functions
# ============================================

def init_browser():
    """Initialize browser with cookies"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    # Load cookies
    driver.get("https://twitter.com")
    time.sleep(2)
    
    with open(COOKIES_FILE, 'r') as f:
        cookies = json.load(f)
    
    for cookie in cookies:
        if 'sameSite' in cookie and cookie['sameSite'] not in ['Strict', 'Lax', 'None']:
            cookie['sameSite'] = 'None'
        driver.add_cookie(cookie)
    
    return driver

def collect_usernames_from_page(driver, username, page_type="followers"):
    """
    Collect usernames using WORKING METHOD
    Returns: (usernames_list, blocked_flag)
    """
    url = f"https://twitter.com/{username}/{page_type}"
    
    print(f"      🌐 {url}")
    driver.get(url)
    time.sleep(4)  # Longer initial wait
    
    # CHECK FOR BLOCKING
    current_url = driver.current_url.lower()
    if "login" in current_url or "error" in current_url:
        print(f"      🚫 BLOCKED! Redirected to: {current_url}")
        return [], True  # Return empty list and blocked flag
    
    collected_usernames = set()
    no_new_users_count = 0
    
    print(f"      📜 Scrolling...")
    for scroll_num in range(MAX_SCROLLS):
        previous_count = len(collected_usernames)
        
        # Get all spans and filter for @username
        all_spans = driver.find_elements(By.TAG_NAME, "span")
        
        for span in all_spans:
            try:
                text = span.text.strip()
                if text.startswith('@') and len(text) > 1:
                    username_clean = text[1:].lower()
                    if username_clean != username.lower():
                        collected_usernames.add(username_clean)
            except:
                pass
        
        # Check if we got new users
        new_count = len(collected_usernames)
        if new_count == previous_count:
            no_new_users_count += 1
            if no_new_users_count >= 3:  # Stop if no new users for 3 scrolls
                print(f"      ✓ No new users, stopping at scroll {scroll_num + 1}")
                break
        else:
            no_new_users_count = 0
        
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE)
        
        # Progress update
        if (scroll_num + 1) % 5 == 0:
            print(f"      ... Scroll {scroll_num + 1}/{MAX_SCROLLS}: {new_count} users")
    
    return list(collected_usernames), False  # Not blocked

def save_progress(connections, filename):
    """Save intermediate results"""
    if len(connections) > 0:
        df = pd.DataFrame(connections)
        df = df.drop_duplicates()
        df.to_csv(filename, index=False, encoding='utf-8')
        return len(df)
    return 0

# ============================================
# Load POIs - ALL USERS
# ============================================
print(f"\n📂 Step 1: Loading ALL POI data...")

df_pois = pd.read_csv(POI_DATA_CSV, encoding='utf-8')
df_active = df_pois[df_pois['account_status'] == 'active'].copy()

# ALL USERS - NO LIMIT
target_users = df_active['username'].dropna().unique()

print(f"   ✓ Total POIs: {len(df_pois)}")
print(f"   ✓ Active POIs: {len(df_active)}")
print(f"   🎯 TARGET: Processing ALL {len(target_users)} users")
print(f"\n⚠️  THIS WILL TAKE SEVERAL HOURS!")
print(f"⚠️  Estimated time: {len(target_users) * 3 / 60:.1f} minutes minimum")

# Check if previous results exist
processed_users = []
if CONNECTIONS_CSV.exists():
    print(f"\n📋 Found existing results: {CONNECTIONS_CSV}")
    existing_df = pd.read_csv(CONNECTIONS_CSV, encoding='utf-8')
    processed_users = existing_df['target_username'].unique().tolist()
    print(f"   Already processed: {len(processed_users)} POIs")
    
    response = input("\n🤔 Continue from where we left off? (yes/no): ")
    if response.lower() == 'yes':
        # Filter out already processed users
        target_users = [u for u in target_users if u not in processed_users]
        all_connections = existing_df.to_dict('records')
        print(f"   ✅ Continuing with {len(target_users)} remaining POIs")
    else:
        all_connections = []
        processed_users = []
        print(f"   🔄 Starting fresh")
else:
    all_connections = []
    print(f"\n🚀 Starting fresh collection")

# ============================================
# Main Collection Loop
# ============================================
print(f"\n{'=' * 70}")
print(f"🔍 STARTING COLLECTION...")
print(f"{'=' * 70}\n")

driver = init_browser()
print("✅ Browser initialized\n")

processed_count = len(processed_users)
failed_count = 0
blocked = False

try:
    for idx, username in enumerate(target_users, 1):
        print(f"[{processed_count + idx}/{len(target_users) + len(processed_users)}] 👤 @{username}")
        print("-" * 70)
        
        try:
            # Collect FOLLOWERS
            print(f"   📥 FOLLOWERS...")
            followers, blocked = collect_usernames_from_page(driver, username, "followers")
            
            if blocked:
                print(f"\n{'=' * 70}")
                print(f"🚫 BLOCKED! Stopping to avoid further issues.")
                print(f"{'=' * 70}")
                print(f"✅ Processed: {processed_count + idx - 1} POIs")
                print(f"📊 Collected: {len(all_connections)} connections")
                print(f"💾 Data saved to: {CONNECTIONS_CSV}")
                break
            
            print(f"   ✅ {len(followers)} followers")
            
            for follower in followers:
                all_connections.append({
                    'target_username': username,
                    'other_username': follower,
                    'type': 'follower'
                })
            
            # Delay
            delay = random.uniform(*REQUEST_DELAY)
            print(f"   ⏳ {delay:.1f}s...")
            time.sleep(delay)
            
            # Collect FOLLOWING
            print(f"   📤 FOLLOWING...")
            following, blocked = collect_usernames_from_page(driver, username, "following")
            
            if blocked:
                print(f"\n{'=' * 70}")
                print(f"🚫 BLOCKED! Stopping to avoid further issues.")
                print(f"{'=' * 70}")
                print(f"✅ Processed: {processed_count + idx - 1} POIs")
                print(f"📊 Collected: {len(all_connections)} connections")
                print(f"💾 Data saved to: {CONNECTIONS_CSV}")
                break
            
            print(f"   ✅ {len(following)} following")
            
            for followed in following:
                all_connections.append({
                    'target_username': username,
                    'other_username': followed,
                    'type': 'following'
                })
            
            processed_count += 1
            print(f"   📊 Total: {len(all_connections)} connections")
            
            # SAVE INTERVAL
            if processed_count % SAVE_INTERVAL == 0:
                saved_count = save_progress(all_connections, CONNECTIONS_CSV)
                print(f"\n   💾 SAVED: {saved_count} connections")
                print(f"   Progress: {processed_count}/{len(target_users) + len(processed_users)} POIs\n")
            
            # BROWSER RESTART
            if processed_count % RESTART_INTERVAL == 0:
                print(f"\n   🔄 RESTARTING BROWSER (every {RESTART_INTERVAL} POIs)...")
                driver.quit()
                time.sleep(5)
                driver = init_browser()
                print(f"   ✅ Browser restarted\n")
            
            # Delay before next user
            delay = random.uniform(*REQUEST_DELAY)
            time.sleep(delay)
            
        except Exception as e:
            print(f"   ❌ ERROR: {str(e)}")
            failed_count += 1
            
            # Save on error
            save_progress(all_connections, CONNECTIONS_CSV)
            
            # Cooldown after error
            print(f"   ⏳ Cooldown {COOLDOWN_AFTER_ERROR}s...")
            time.sleep(COOLDOWN_AFTER_ERROR)
            continue

finally:
    driver.quit()
    print(f"\n{'=' * 70}")
    print(f"🏁 COLLECTION COMPLETED")
    print(f"{'=' * 70}")
    print(f"✅ Successfully processed: {processed_count} POIs")
    print(f"❌ Failed: {failed_count} POIs")
    print(f"📊 Total connections: {len(all_connections)}")

# ============================================
# Final Save
# ============================================
print(f"\n💾 Saving final results...")

if len(all_connections) > 0:
    df_connections = pd.DataFrame(all_connections)
    df_connections = df_connections.drop_duplicates()
    df_connections.to_csv(CONNECTIONS_CSV, index=False, encoding='utf-8')
    
    print(f"   ✅ Saved {len(df_connections)} unique connections")
    print(f"   📁 {CONNECTIONS_CSV}")
    
    # Statistics
    print(f"\n📊 Final Statistics:")
    print(f"   Followers: {len(df_connections[df_connections['type']=='follower']):,}")
    print(f"   Following: {len(df_connections[df_connections['type']=='following']):,}")
    print(f"   Unique POIs: {df_connections['target_username'].nunique()}")
    print(f"   Unique connections: {df_connections['other_username'].nunique()}")
    
    # Preview
    print(f"\n📋 Sample connections:")
    display(df_connections.head(20))
else:
    print("   ⚠️  No connections collected")

print(f"\n{'=' * 70}")
print(f"✅ Stage 9 COMPLETE!")
print(f"{'=' * 70}")

## Stage 9.1 - Generate Statistics from Collected Data

Analyze the followers/following connections and create comprehensive statistics per POI

In [ ]:
# ============================================
# Stage 9.1: Generate Statistics from Connections
# Input:  POIs/Candidates/POIs_candidate_connections.csv
#         POIs/POI_twitter_users_data.csv
# Output: POIs/Candidates/Candidates_statistics.csv
# ============================================

import pandas as pd
from pathlib import Path

print("=" * 70)
print("STAGE 9.1: GENERATE STATISTICS FROM CONNECTIONS")
print("=" * 70)

# File paths
POIS_DIR = Path("POIs")
CANDIDATES_DIR = POIS_DIR / "Candidates"
CONNECTIONS_CSV = CANDIDATES_DIR / "POIs_candidate_connections.csv"
POI_DATA_CSV = POIS_DIR / "POI_twitter_users_data.csv"
STATISTICS_CSV = CANDIDATES_DIR / "Candidates_statistics.csv"

# ============================================
# Step 1: Load Data
# ============================================
print(f"\n📂 Step 1: Loading data files...")

# Load connections
if not CONNECTIONS_CSV.exists():
    print(f"   ❌ ERROR: Connections file not found!")
    print(f"   Expected: {CONNECTIONS_CSV}")
    print(f"   Please run Stage 9 first to collect connections.")
else:
    df_connections = pd.read_csv(CONNECTIONS_CSV, encoding='utf-8')
    print(f"   ✓ Loaded connections: {len(df_connections):,} records")
    print(f"   ✓ Unique POIs in connections: {df_connections['target_username'].nunique()}")
    
    # Load POI metadata
    df_pois = pd.read_csv(POI_DATA_CSV, encoding='utf-8')
    print(f"   ✓ Loaded POI metadata: {len(df_pois)} POIs")

# ============================================
# Step 2: Calculate Statistics per POI
# ============================================
print(f"\n📊 Step 2: Calculating statistics per POI...")

stats_list = []

# Get all unique POIs from connections
poi_usernames = df_connections['target_username'].unique()

for username in poi_usernames:
    # Filter connections for this POI
    poi_connections = df_connections[df_connections['target_username'] == username]
    
    # Count followers and following
    followers = poi_connections[poi_connections['type'] == 'follower']
    following = poi_connections[poi_connections['type'] == 'following']
    
    followers_collected = len(followers)
    following_collected = len(following)
    total_connections = followers_collected + following_collected
    
    # Get POI metadata
    poi_info = df_pois[df_pois['username'] == username]
    
    if len(poi_info) > 0:
        full_name = poi_info['display_name'].iloc[0]
        followers_count = poi_info['followers_count'].iloc[0] if 'followers_count' in poi_info.columns else 0
        following_count = poi_info['following_count'].iloc[0] if 'following_count' in poi_info.columns else 0
        posts_count = poi_info['posts_count'].iloc[0] if 'posts_count' in poi_info.columns else 0
    else:
        full_name = ''
        followers_count = 0
        following_count = 0
        posts_count = 0
    
    # Calculate collection percentages
    followers_percentage = (followers_collected / followers_count * 100) if followers_count > 0 else 0
    following_percentage = (following_collected / following_count * 100) if following_count > 0 else 0
    
    stats_list.append({
        'Twitter_username': username,
        'Full_Name': full_name,
        'Total_Followers': followers_count,
        'Followers_Collected': followers_collected,
        'Followers_Percentage': round(followers_percentage, 2),
        'Total_Following': following_count,
        'Following_Collected': following_collected,
        'Following_Percentage': round(following_percentage, 2),
        'Total_Connections_Collected': total_connections,
        'Posts_Count': posts_count
    })

# Create DataFrame
df_stats = pd.DataFrame(stats_list)

# Sort by total connections collected (descending)
df_stats = df_stats.sort_values('Total_Connections_Collected', ascending=False)

print(f"   ✓ Calculated statistics for {len(df_stats)} POIs")

# ============================================
# Step 3: Overall Statistics
# ============================================
print(f"\n📈 Step 3: Overall statistics...")

total_followers_collected = df_stats['Followers_Collected'].sum()
total_following_collected = df_stats['Following_Collected'].sum()
total_connections_collected = df_stats['Total_Connections_Collected'].sum()

avg_followers_collected = df_stats['Followers_Collected'].mean()
avg_following_collected = df_stats['Following_Collected'].mean()

print(f"\n   Overall Summary:")
print(f"   {'─' * 50}")
print(f"   POIs processed: {len(df_stats)}")
print(f"   Total followers collected: {total_followers_collected:,}")
print(f"   Total following collected: {total_following_collected:,}")
print(f"   Total connections: {total_connections_collected:,}")
print(f"   {'─' * 50}")
print(f"   Avg followers per POI: {avg_followers_collected:.1f}")
print(f"   Avg following per POI: {avg_following_collected:.1f}")

# ============================================
# Step 4: Identify Top POIs
# ============================================
print(f"\n🏆 Step 4: Top POIs by connections collected...")

print(f"\n   Top 10 POIs by Total Connections:")
print(f"   {'─' * 70}")
top_10 = df_stats.head(10)[['Twitter_username', 'Full_Name', 'Total_Connections_Collected']]
for idx, row in top_10.iterrows():
    print(f"   {row['Twitter_username']:20s} | {row['Full_Name']:30s} | {row['Total_Connections_Collected']:,}")

# ============================================
# Step 5: Save Statistics
# ============================================
print(f"\n💾 Step 5: Saving statistics...")

df_stats.to_csv(STATISTICS_CSV, index=False, encoding='utf-8')
print(f"   ✓ Saved to: {STATISTICS_CSV}")
print(f"   ✓ Total records: {len(df_stats)}")

# ============================================
# Step 6: Display Results
# ============================================
print(f"\n📋 Step 6: Preview of statistics...")

print(f"\n   Full Statistics Table:")
display(df_stats)

# Additional analysis
print(f"\n📊 Additional Analysis:")
print(f"   {'─' * 50}")
print(f"   POIs with 100+ followers collected: {len(df_stats[df_stats['Followers_Collected'] >= 100])}")
print(f"   POIs with 100+ following collected: {len(df_stats[df_stats['Following_Collected'] >= 100])}")
print(f"   POIs with 200+ total connections: {len(df_stats[df_stats['Total_Connections_Collected'] >= 200])}")

# Collection efficiency
avg_followers_pct = df_stats['Followers_Percentage'].mean()
avg_following_pct = df_stats['Following_Percentage'].mean()
print(f"   {'─' * 50}")
print(f"   Avg collection rate (followers): {avg_followers_pct:.2f}%")
print(f"   Avg collection rate (following): {avg_following_pct:.2f}%")

print(f"\n{'=' * 70}")
print(f"✅ Stage 9.1 COMPLETED!")
print(f"{'=' * 70}")
print(f"📁 Output file: {STATISTICS_CSV}")
print(f"📊 Statistics generated for {len(df_stats)} POIs")
print(f"🔗 Total connections analyzed: {total_connections_collected:,}")

## Stage 10: Collect Full Metadata for All Candidate Users
Extract complete Twitter metadata for every unique user found in Stage 9 connections

In [ ]:
# התקנת geckodriver אם לא מותקן
import subprocess
import sys

try:
    import geckodriver_autoinstaller
    print("✅ geckodriver-autoinstaller כבר מותקן")
except ImportError:
    print("📦 מתקין geckodriver-autoinstaller...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "geckodriver-autoinstaller"])
    import geckodriver_autoinstaller
    print("✅ התקנה הושלמה!")

# התקן geckodriver
geckodriver_autoinstaller.install()

# ============================================
# STAGE 9.2: COLLECT CANDIDATE USER METADATA
# FIREFOX VERSION - STABLE ALTERNATIVE
# ============================================

import pandas as pd
import time
import json
import random
import re
from pathlib import Path
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

print("\n" + "="*80)
print("📊 STAGE 9.2: FIREFOX VERSION - STABLE")
print("="*80)

# ============================================
# SETUP PATHS
# ============================================
IRAN_ROOT = Path.cwd()
POIS_DIR = IRAN_ROOT / "POIs"
CANDIDATES_DIR = POIS_DIR / "Candidates"
COOKIES_FILE = POIS_DIR / "x_cookies.json"

print(f"📁 Working directory: {IRAN_ROOT}")
print(f"📁 POIs directory: {POIS_DIR}")
print(f"📁 Candidates directory: {CANDIDATES_DIR}\n")

# File paths
CONNECTIONS_CSV = CANDIDATES_DIR / "POIs_candidate_connections.csv"
OUTPUT_CSV = CANDIDATES_DIR / "Candidates_user_data.csv"
CHECKPOINT_CSV = CANDIDATES_DIR / "Candidates_user_data_checkpoint.csv"
LOG_FILE = CANDIDATES_DIR / "collection_log.txt"

print(f"📂 Input: {CONNECTIONS_CSV}")
print(f"📂 Output: {OUTPUT_CSV}")
print(f"💾 Checkpoint: {CHECKPOINT_CSV}")
print(f"📝 Log: {LOG_FILE}")
print(f"🔐 Cookies: {COOKIES_FILE} ({'✅ Found' if COOKIES_FILE.exists() else '❌ Missing'})\n")

# ============================================
# OPTIMIZED CONFIGURATION FOR FASTER COLLECTION
# ============================================
REQUEST_DELAY = (2, 4)               # FASTER: 2-4s between requests (was 5-10)
LONG_DELAY = (30, 60)                # FASTER: 30-60s every 50 users (was 2-5 min)
LONG_DELAY_INTERVAL = 50             # Every 50 users (was 30)
BLOCK_WAIT_TIME = (600, 900)         # 10-15 minutes if blocked (was 20-30)
MAX_CONSECUTIVE_FAILURES = 5         # More tolerance before block check (was 3)
PAGE_LOAD_WAIT = 3                   # FASTER: 3 seconds to load (was 5)
CHECKPOINT_SAVE_INTERVAL = 10        # Save every 10 users (was 5)
BROWSER_RESTART_INTERVAL = 150       # Restart every 150 users (was 100)

TEST_MODE = False  # ❗ FULL RUN - ALL USERS

print("⚙️  OPTIMIZED CONFIGURATION - FASTER COLLECTION:")
print(f"   ⚡ Request delay: {REQUEST_DELAY[0]}-{REQUEST_DELAY[1]}s (FASTER - was 5-10s)")
print(f"   ⚡ Long delay every {LONG_DELAY_INTERVAL} users: {LONG_DELAY[0]}-{LONG_DELAY[1]}s (FASTER - was 2-5 min)")
print(f"   🔄 Browser restart: every {BROWSER_RESTART_INTERVAL} users (LESS FREQUENT - was 100)")
print(f"   💾 Checkpoint: every {CHECKPOINT_SAVE_INTERVAL} users (was 5)")
print(f"   ⚠️  FULL RUN MODE: Will process ALL users")
print(f"   ⏱️  Estimated time: ~1 DAY (much faster!)\n")

# ============================================
# ENHANCED LOGGING
# ============================================
def log_message(msg, level="INFO"):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(f"[{ts}] [{level}] {msg}\n")
    if level in ["ERROR", "BLOCK", "WARNING"]:
        print(f"  📝 LOG: {msg}")

# ============================================
# LOAD DATA
# ============================================
print("📥 Loading connections data...")
log_message("=== NEW RUN STARTED ===")
log_message("Starting high-quality metadata collection")

df_connections = pd.read_csv(CONNECTIONS_CSV, encoding='utf-8')
print(f"   ✅ Loaded {len(df_connections):,} connection records")
log_message(f"Loaded {len(df_connections):,} connection records")

# Extract and validate usernames
all_usernames = set()
invalid_count = 0
for username in df_connections['other_username'].unique():
    if pd.notna(username):
        username = str(username).strip()
        if re.match(r'^[a-zA-Z0-9_]{1,15}$', username):
            all_usernames.add(username)
        else:
            invalid_count += 1

print(f"   ✅ {len(all_usernames):,} valid usernames")
print(f"   ⚠️  {invalid_count} invalid usernames filtered out\n")
log_message(f"Valid usernames: {len(all_usernames):,}, Invalid: {invalid_count}")

# ============================================
# RESUME FROM CHECKPOINT
# ============================================
processed_usernames = set()
existing_data = []

if CHECKPOINT_CSV.exists():
    print("💾 Checkpoint found - resuming previous run...")
    log_message("Loading checkpoint for resume")
    try:
        df_checkpoint = pd.read_csv(CHECKPOINT_CSV, encoding='utf-8')
        if 'username' in df_checkpoint.columns:
            processed_usernames = set(df_checkpoint['username'].unique())
            existing_data = df_checkpoint.to_dict('records')
            print(f"   ✅ Found {len(processed_usernames):,} previously processed users")
            print(f"   ⏩ Resuming from user #{len(processed_usernames) + 1}\n")
            log_message(f"Resuming from {len(processed_usernames):,} users")
        else:
            print("   ⚠️  Invalid checkpoint format - starting fresh\n")
            log_message("Invalid checkpoint, starting fresh", "WARNING")
    except Exception as e:
        print(f"   ⚠️  Checkpoint error: {e} - starting fresh\n")
        log_message(f"Checkpoint error: {e}", "ERROR")

# Filter remaining users
target_usernames = sorted([u for u in all_usernames if u not in processed_usernames])

print(f"📋 PROCESSING SUMMARY:")
print(f"   Total unique users: {len(all_usernames):,}")
print(f"   Already processed: {len(processed_usernames):,}")
print(f"   Remaining to process: {len(target_usernames):,}")

if len(target_usernames) == 0:
    print("\n✅ ALL USERS ALREADY PROCESSED!")
    log_message("All users already processed - collection complete")
else:
    estimated_hours = (len(target_usernames) * 7) / 3600  # ~7s average per user
    print(f"   ⏱️  Estimated time: {estimated_hours:.1f} hours ({estimated_hours/24:.1f} days)")
    print(f"\n{'='*80}")
    
    # ============================================
    # ENHANCED BROWSER SETUP
    # ============================================
    USER_AGENTS = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    ]
    
    def init_browser():
        """Initialize Firefox browser"""
        print("🦊 Initializing Firefox WebDriver...")
        log_message("Initializing Firefox")
        
        options = Options()
        # Firefox headless mode
        options.add_argument('--headless')
        options.add_argument('--width=1920')
        options.add_argument('--height=1080')
        
        # Anti-detection preferences
        options.set_preference("dom.webdriver.enabled", False)
        options.set_preference('useAutomationExtension', False)
        options.set_preference("general.useragent.override", random.choice(USER_AGENTS))
        
        driver = webdriver.Firefox(options=options)
        driver.set_page_load_timeout(30)
        
        # Note: CDP commands are Chrome-only, not needed for Firefox
        # Firefox anti-detection is handled via preferences above
        
        return driver
    
    def load_cookies(driver):
        """Load Twitter cookies"""
        if not COOKIES_FILE.exists():
            print("⚠️  No cookies file!")
            log_message("No cookies file found", "WARNING")
            return False
        
        try:
            driver.get("https://x.com")
            time.sleep(3)
            driver.delete_all_cookies()
            
            with open(COOKIES_FILE, 'r', encoding='utf-8') as f:
                cookies = json.load(f)
            
            for cookie in cookies:
                try:
                    cookie.pop('sameSite', None)
                    cookie.pop('storeId', None)
                    driver.add_cookie(cookie)
                except:
                    pass
            
            time.sleep(3)
            log_message("Cookies loaded successfully")
            return True
        except Exception as e:
            print(f"⚠️  Cookie loading failed: {e}")
            log_message(f"Cookie loading failed: {e}", "ERROR")
            return False
    
    def check_if_blocked(driver):
        """Check if Twitter blocked us"""
        try:
            driver.get("https://x.com/explore")
            time.sleep(5)
            
            url = driver.current_url.lower()
            text = driver.find_element(By.TAG_NAME, 'body').text.lower()
            
            if "login" in url or "error" in url:
                return True, "Redirected to login/error"
            
            if "rate limit" in text or "try again later" in text:
                return True, "Rate limit message detected"
            
            if len(text) < 100:
                return True, "Page too short (blocked)"
            
            return False, None
            
        except Exception as e:
            return True, f"Check failed: {str(e)[:50]}"
    
    # ============================================
    # HIGH-QUALITY METADATA EXTRACTION
    # ============================================
    def extract_user_metadata(driver, username):
        """
        Extract metadata with multiple fallback methods
        Returns dict with metadata or error
        """
        try:
            url = f"https://x.com/{username}"
            driver.get(url)
            
            # Wait for page to load and JavaScript to execute
            time.sleep(PAGE_LOAD_WAIT)
            
            # Wait for profile content to appear (up to 10 seconds)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )
                # Extra wait for dynamic content
                time.sleep(3)
            except:
                pass
            
            # Add extra random delay to appear more human
            time.sleep(random.uniform(0.5, 1.5))
            
            # Check for blocks/errors
            current_url = driver.current_url.lower()
            if "login" in current_url or "error" in current_url:
                log_message(f"Blocked on {username}: redirected to {current_url}", "BLOCK")
                return {'error': 'blocked', 'username': username}
            
            # Get page text
            try:
                page_text = driver.find_element(By.TAG_NAME, 'body').text
            except:
                log_message(f"Failed to get page text for {username}", "ERROR")
                return {'error': 'no_page_text', 'username': username}
            
            # Check for suspended/not found
            page_text_lower = page_text.lower()
            if "doesn't exist" in page_text_lower or "suspended" in page_text_lower:
                log_message(f"Account not found: {username}")
                return {'error': 'not_found', 'username': username}
            
            if "rate limit" in page_text_lower or "try again later" in page_text_lower:
                log_message(f"Rate limited on {username}", "BLOCK")
                return {'error': 'rate_limited', 'username': username}
            
            # Initialize metadata
            metadata = {
                'username': username,
                'display_name': None,
                'description': None,
                'followers_count': None,
                'following_count': None,
                'statuses_count': None,
                'verified': False,
                'created_at': None,
                'location': None,
                'url': None,
                'profile_image_url': None,
                'profile_banner_url': None
            }
            
            # ============================================
            # EXTRACT DISPLAY NAME - Multiple methods
            # ============================================
            try:
                # Method 1: From title
                title = driver.title
                if '(@' in title:
                    name = title.split('(@')[0].strip()
                    if name and name not in ['Twitter', 'X']:
                        metadata['display_name'] = name
            except:
                pass
            
            if not metadata['display_name']:
                try:
                    # Method 2: From page text - look for pattern "Name @username"
                    lines = page_text.split('\n')
                    for i, line in enumerate(lines):
                        if f'@{username}' in line and i > 0:
                            potential_name = lines[i-1].strip()
                            if potential_name and len(potential_name) < 50 and not potential_name.startswith('@'):
                                metadata['display_name'] = potential_name
                                break
                except:
                    pass
            
            # ============================================
            # EXTRACT BIO/DESCRIPTION
            # ============================================
            try:
                bio_elem = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']")
                if bio_elem.text:
                    metadata['description'] = bio_elem.text.strip().replace('\n', ' ')
            except:
                pass
            
            # ============================================
            # EXTRACT FOLLOWERS - Multiple patterns
            # ============================================
            # DEBUG: Print page text snippet for first user
            if position == 1:
                print(f"\n🔍 DEBUG - Page text sample (first 500 chars):")
                print(page_text[:500])
                print(f"\n🔍 Looking for followers/following in text...")
            
            followers_patterns = [
                r'(\d{1,3}(?:,\d{3})*|\d+)\s*Followers',
                r'(\d{1,3}(?:,\d{3})*|\d+)\s*Follower',
                r'Followers\s*(\d{1,3}(?:,\d{3})*|\d+)',
            ]
            
            for pattern in followers_patterns:
                match = re.search(pattern, page_text, re.IGNORECASE)
                if match:
                    try:
                        count_str = match.group(1).replace(',', '').upper()
                        if 'K' in count_str:
                            metadata['followers_count'] = int(float(count_str.replace('K', '')) * 1000)
                        elif 'M' in count_str:
                            metadata['followers_count'] = int(float(count_str.replace('M', '')) * 1000000)
                        else:
                            metadata['followers_count'] = int(count_str)
                        break
                    except:
                        continue
            
            # ============================================
            # EXTRACT FOLLOWING - Multiple patterns
            # ============================================
            following_patterns = [
                r'(\d{1,3}(?:,\d{3})*|\d+)\s*Following',
                r'Following\s*(\d{1,3}(?:,\d{3})*|\d+)',
            ]
            
            for pattern in following_patterns:
                match = re.search(pattern, page_text, re.IGNORECASE)
                if match:
                    try:
                        count_str = match.group(1).replace(',', '').upper()
                        if 'K' in count_str:
                            metadata['following_count'] = int(float(count_str.replace('K', '')) * 1000)
                        elif 'M' in count_str:
                            metadata['following_count'] = int(float(count_str.replace('M', '')) * 1000000)
                        else:
                            metadata['following_count'] = int(count_str)
                        break
                    except:
                        continue
            
            # ============================================
            # EXTRACT POSTS/TWEETS COUNT
            # ============================================
            posts_patterns = [
                r'(\d{1,3}(?:,\d{3})*|\d+)\s*posts?',
                r'(\d{1,3}(?:,\d{3})*|\d+)\s*Tweets?',
                r'posts?\s*(\d{1,3}(?:,\d{3})*|\d+)',
            ]
            
            for pattern in posts_patterns:
                match = re.search(pattern, page_text, re.IGNORECASE)
                if match:
                    try:
                        count_str = match.group(1).replace(',', '').upper()
                        if 'K' in count_str:
                            metadata['statuses_count'] = int(float(count_str.replace('K', '')) * 1000)
                        elif 'M' in count_str:
                            metadata['statuses_count'] = int(float(count_str.replace('M', '')) * 1000000)
                        else:
                            metadata['statuses_count'] = int(count_str)
                        break
                    except:
                        continue
            
            # ============================================
            # EXTRACT JOINED DATE
            # ============================================
            joined_match = re.search(r'Joined\s+(\w+\s+\d{4})', page_text, re.IGNORECASE)
            if joined_match:
                metadata['created_at'] = joined_match.group(1)
            
            # ============================================
            # EXTRACT LOCATION
            # ============================================
            try:
                location_elem = driver.find_element(By.XPATH, "//span[@data-testid='UserLocation']")
                if location_elem.text:
                    metadata['location'] = location_elem.text.strip()
            except:
                pass
            
            # ============================================
            # EXTRACT PROFILE IMAGE
            # ============================================
            try:
                img_elems = driver.find_elements(By.XPATH, "//img[contains(@src, 'profile_images')]")
                if img_elems:
                    metadata['profile_image_url'] = img_elems[0].get_attribute('src')
            except:
                pass
            
            # ============================================
            # VALIDATION - Must have at least 2 of the 3 main fields
            # ============================================
            has_followers = metadata['followers_count'] is not None
            has_following = metadata['following_count'] is not None
            has_posts = metadata['statuses_count'] is not None
            
            valid_fields = sum([has_followers, has_following, has_posts])
            
            if valid_fields < 2:
                log_message(f"Insufficient data for {username}: only {valid_fields}/3 fields", "WARNING")
                return {'error': 'insufficient_data', 'username': username}
            
            # Success!
            log_message(f"✅ {username}: {metadata['followers_count']} followers, {metadata['following_count']} following")
            return metadata
            
        except Exception as e:
            log_message(f"Exception on {username}: {str(e)[:100]}", "ERROR")
            return {'error': str(e)[:100], 'username': username}
    
    def save_checkpoint(data):
        """Save checkpoint with validation"""
        try:
            if not data:
                return 0
            df = pd.DataFrame(data)
            df.to_csv(CHECKPOINT_CSV, index=False, encoding='utf-8')
            log_message(f"Checkpoint saved: {len(df)} users")
            return len(df)
        except Exception as e:
            log_message(f"Checkpoint save failed: {e}", "ERROR")
            return 0
    
    # ============================================
    # MAIN PROCESSING LOOP
    # ============================================
    driver = init_browser()
    load_cookies(driver)
    
    all_metadata = existing_data.copy()
    success_count = 0
    error_count = 0
    consecutive_failures = 0
    
    # For smart delays
    next_long_delay = LONG_DELAY_INTERVAL
    
    print(f"\n{'='*80}")
    print("🚀 STARTING HIGH-QUALITY COLLECTION")
    print(f"{'='*80}\n")
    log_message("Starting main processing loop")
    
    idx = 0
    while idx < len(target_usernames):
        username = target_usernames[idx]
        position = idx + 1
        total_processed = len(processed_usernames) + position
        
        print(f"[{position}/{len(target_usernames)}] (Total: {total_processed:,}/{len(all_usernames):,}) @{username}...", end=" ")
        
        # ============================================
        # BROWSER RESTART
        # ============================================
        if position > 1 and position % BROWSER_RESTART_INTERVAL == 0:
            print("\n\n🔄 Restarting browser for freshness...")
            log_message(f"Browser restart at position {position}")
            try:
                driver.quit()
            except:
                pass
            time.sleep(10)
            driver = init_browser()
            load_cookies(driver)
            print("✅ Browser restarted\n")
        
        # ============================================
        # EXTRACT METADATA
        # ============================================
        metadata = extract_user_metadata(driver, username)
        
        # ============================================
        # HANDLE ERRORS WITH AUTO-RECOVERY
        # ============================================
        if 'error' in metadata:
            error_type = metadata.get('error')
            
            if error_type in ['blocked', 'rate_limited']:
                print(f"🚫 BLOCKED!")
                log_message(f"BLOCK DETECTED at position {position}", "BLOCK")
                
                # Save before waiting
                save_checkpoint(all_metadata)
                
                # Wait 20-30 minutes
                wait_time = random.randint(*BLOCK_WAIT_TIME)
                print(f"\n⏳ Waiting {wait_time//60} minutes for recovery...")
                print(f"   Current time: {datetime.now().strftime('%H:%M:%S')}")
                print(f"   Resume at: {(datetime.now() + pd.Timedelta(seconds=wait_time)).strftime('%H:%M:%S')}\n")
                log_message(f"Waiting {wait_time}s for block recovery")
                
                time.sleep(wait_time)
                
                # Restart browser
                print("🔄 Restarting browser after wait...")
                try:
                    driver.quit()
                except:
                    pass
                time.sleep(10)
                driver = init_browser()
                load_cookies(driver)
                
                # Check if still blocked
                is_blocked, reason = check_if_blocked(driver)
                if is_blocked:
                    print(f"⚠️  Still blocked: {reason}")
                    print("⏳ Waiting another 15 minutes...\n")
                    log_message(f"Still blocked: {reason}, waiting more", "BLOCK")
                    time.sleep(900)
                    driver.quit()
                    driver = init_browser()
                    load_cookies(driver)
                
                print("✅ Recovery complete - retrying same user\n")
                log_message("Recovery complete")
                # Don't increment idx - retry same user
                continue
            
            elif error_type == 'not_found':
                print(f"⚠️  Not found/suspended")
                error_count += 1
                consecutive_failures = 0
                
            elif error_type == 'insufficient_data':
                print(f"❌ Insufficient data")
                error_count += 1
                consecutive_failures += 1
                
            else:
                print(f"❌ Error: {error_type[:40]}")
                error_count += 1
                consecutive_failures += 1
            
            # Check for too many consecutive failures
            if consecutive_failures >= MAX_CONSECUTIVE_FAILURES:
                print(f"\n⚠️  {consecutive_failures} consecutive failures - checking for block...")
                log_message(f"{consecutive_failures} consecutive failures", "WARNING")
                
                is_blocked, reason = check_if_blocked(driver)
                if is_blocked:
                    print(f"🚫 Block confirmed: {reason}")
                    log_message(f"Block confirmed: {reason}", "BLOCK")
                    save_checkpoint(all_metadata)
                    
                    wait_time = random.randint(*BLOCK_WAIT_TIME)
                    print(f"⏳ Waiting {wait_time//60} minutes...\n")
                    time.sleep(wait_time)
                    
                    driver.quit()
                    driver = init_browser()
                    load_cookies(driver)
                    print("✅ Recovered\n")
                
                consecutive_failures = 0
        
        # ============================================
        # SUCCESS - SAVE DATA
        # ============================================
        else:
            all_metadata.append(metadata)
            consecutive_failures = 0
            success_count += 1
            
            # Display info
            name = (metadata.get('display_name') or 'N/A')[:20]
            followers = metadata.get('followers_count') or 0
            following = metadata.get('following_count') or 0
            posts = metadata.get('statuses_count') or 0
            
            print(f"✅ {name:20} | {followers:>7,} F | {following:>7,} F | {posts:>7,} P")
        
        # ============================================
        # CHECKPOINT SAVE
        # ============================================
        if position % CHECKPOINT_SAVE_INTERVAL == 0:
            saved = save_checkpoint(all_metadata)
            success_rate = (success_count / (success_count + error_count) * 100) if (success_count + error_count) > 0 else 0
            print(f"  💾 Checkpoint: {saved} users | Success rate: {success_rate:.1f}%")
        
        # ============================================
        # SMART DELAYS FOR HUMAN-LIKE BEHAVIOR
        # ============================================
        
        # Super long delay (10-15 min every 100 users)
        # Long delay (30-60s every 50 users) - FASTER!
        if position == next_long_delay:
            delay_seconds = random.randint(*LONG_DELAY)
            print(f"\n  ⏳ Quick break: {delay_seconds}s")
            print(f"     Progress: {success_count} success, {error_count} errors\n")
            log_message(f"Long delay at position {position}: {delay_seconds}s")
            time.sleep(delay_seconds)
            next_long_delay = position + LONG_DELAY_INTERVAL
        
        # Regular delay (2-4s between requests) - MUCH FASTER!
        else:
            delay = random.uniform(*REQUEST_DELAY)
            time.sleep(delay)
        
        # Move to next user
        idx += 1
    
    # ============================================
    # CLEANUP AND FINAL SAVE
    # ============================================
    try:
        driver.quit()
    except:
        pass
    print("\n🔚 Browser closed")
    log_message("Browser closed - collection loop complete")
    
    # Final save
    if all_metadata:
        print("\n💾 Saving final results...")
        log_message("Saving final results")
        
        df_final = pd.DataFrame(all_metadata)
        df_final.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
        
        print(f"✅ Saved {len(df_final):,} users to: {OUTPUT_CSV}")
        log_message(f"Final save: {len(df_final)} users")
        
        # Remove checkpoint if not in test mode
        if CHECKPOINT_CSV.exists():
            CHECKPOINT_CSV.unlink()
            print("✅ Checkpoint removed (collection complete)")
            log_message("Checkpoint removed - collection complete")
    
    # ============================================
    # FINAL STATISTICS
    # ============================================
    print(f"\n{'='*80}")
    print("📊 FINAL COLLECTION STATISTICS")
    print(f"{'='*80}")
    print(f"✅ Successfully collected: {success_count:,} users")
    print(f"❌ Errors/Not found: {error_count:,} users")
    print(f"📊 Total processed: {success_count + error_count:,} users")
    
    if success_count > 0:
        success_rate = (success_count / (success_count + error_count)) * 100
        print(f"📈 Success rate: {success_rate:.1f}%")
        
        # Quality metrics
        df_final = pd.DataFrame(all_metadata)
        with_followers = (df_final['followers_count'].notna()).sum()
        with_following = (df_final['following_count'].notna()).sum()
        with_bio = (df_final['description'].notna()).sum()
        
        print(f"\n📋 Data Quality:")
        print(f"   Followers count: {with_followers:,} ({with_followers/len(df_final)*100:.1f}%)")
        print(f"   Following count: {with_following:,} ({with_following/len(df_final)*100:.1f}%)")
        print(f"   Bio/Description: {with_bio:,} ({with_bio/len(df_final)*100:.1f}%)")
    
    print(f"{'='*80}\n")
    
    log_message("=== COLLECTION COMPLETE ===")
    log_message(f"Final stats: {success_count} success, {error_count} errors")
    
    print("✅✅✅ STAGE 9.2 COMPLETE! ✅✅✅")
    print("High-quality metadata collection finished successfully!")


## Stage 11: ניתוח סטטיסטי של משתמשי המטרה הפוטנציאליים

In [ ]:
# ============================================
# Stage 11: Statistical Analysis of Potential Target Users
# Input:  POIs/Candidates/Candidates_user_data_MERGED.csv
# Output: Statistical summary table and histograms
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

print("=" * 80)
print("STAGE 11: ניתוח סטטיסטי של משתמשי המטרה הפוטנציאליים")
print("=" * 80)

# File paths
CANDIDATES_DIR = Path("POIs") / "Candidates"
MERGED_CSV = CANDIDATES_DIR / "Candidates_user_data_MERGED.csv"

# Load data
print(f"\n📂 טוען קובץ: {MERGED_CSV}")
df = pd.read_csv(MERGED_CSV, encoding='utf-8')

print(f"✅ נטענו {len(df):,} שורות")
print(f"\nעמודות זמינות: {', '.join(df.columns)}")

# ============================================
# משימה 1: ניקוי נתונים והמרה לטיפוסים נכונים
# ============================================
print("\n" + "=" * 80)
print("📊 שלב 1: ניקוי והכנת נתונים")
print("=" * 80)

# המרת עמודות מספריות
numeric_cols = ['followers_count', 'following_count', 'statuses_count']
for col in numeric_cols:
    if col in df.columns:
        # המרה למספרים - ערכים לא חוקיים יהפכו ל-NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')

# טיפול בתאריכים
if 'created_at' in df.columns:
    # המרת created_at לפורמט תאריך
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    # חישוב גיל החשבון בשנים
    current_date = pd.Timestamp.now()
    df['account_age_years'] = (current_date - df['created_at']).dt.days / 365.25

print(f"\n✅ המרת נתונים הושלמה")

# ============================================
# משימה 2: חישוב סטטיסטיקה תיאורית
# ============================================
print("\n" + "=" * 80)
print("📊 שלב 2: חישוב מדדים סטטיסטיים")
print("=" * 80)

# ספירת משתמשים עם נתונים מלאים
df_with_data = df[df['followers_count'].notna()].copy()

# חישוב סטטיסטיקות
stats = {
    'Total unique users': len(df),
    'Users with complete data': len(df_with_data),
    'Data completeness (%)': round(len(df_with_data) / len(df) * 100, 1),
    'Average posts (statuses_count)': int(df_with_data['statuses_count'].mean()) if 'statuses_count' in df.columns else 'N/A',
    'Median posts': int(df_with_data['statuses_count'].median()) if 'statuses_count' in df.columns else 'N/A',
    'Average followers': int(df_with_data['followers_count'].mean()),
    'Median followers': int(df_with_data['followers_count'].median()),
    'Average following': int(df_with_data['following_count'].mean()) if 'following_count' in df.columns else 'N/A',
    'Median following': int(df_with_data['following_count'].median()) if 'following_count' in df.columns else 'N/A',
}

# הוספת סטטיסטיקות נוספות אם יש נתוני תאריך
if 'account_age_years' in df.columns and df_with_data['account_age_years'].notna().sum() > 0:
    stats['Average account age (years)'] = round(df_with_data['account_age_years'].mean(), 1)
    stats['Median account age (years)'] = round(df_with_data['account_age_years'].median(), 1)

# ============================================
# משימה 3: הצגת טבלה מסכמת
# ============================================
print("\n" + "=" * 80)
print("📋 שלב 3: טבלה מסכמת")
print("=" * 80)
print()

# יצירת DataFrame מהסטטיסטיקות
stats_df = pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])

# הצגת הטבלה
print("┌" + "─" * 50 + "┐")
print("│" + " " * 10 + "STATISTICAL SUMMARY" + " " * 21 + "│")
print("├" + "─" * 50 + "┤")

for idx, row in stats_df.iterrows():
    metric = row['Metric']
    value = f"{row['Value']:,}" if isinstance(row['Value'], (int, float)) else str(row['Value'])
    padding = 48 - len(metric) - len(value)
    print(f"│ {metric}{' ' * padding}{value} │")

print("└" + "─" * 50 + "┘")

# שמירת הטבלה לקובץ
stats_output = CANDIDATES_DIR / "Candidates_statistics_summary.csv"
stats_df.to_csv(stats_output, index=False, encoding='utf-8')
print(f"\n💾 טבלה נשמרה ב: {stats_output}")

# ============================================
# משימה 4: יצירת היסטוגרמות
# ============================================
print("\n" + "=" * 80)
print("📊 שלב 4: יצירת ויזואליזציות")
print("=" * 80)

# יצירת תיקייה לתרשימים
figures_dir = CANDIDATES_DIR / "figures" / "stage11_analysis"
figures_dir.mkdir(parents=True, exist_ok=True)

# הגדרת סגנון
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# יצירת 2x2 היסטוגרמות
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Stage 11: Statistical Distribution Analysis - Candidate Users', 
             fontsize=16, fontweight='bold', y=0.995)

# היסטוגרמה 1: Followers Distribution
ax1 = axes[0, 0]
followers_data = df_with_data['followers_count'].dropna()
ax1.hist(followers_data, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(followers_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {int(followers_data.mean()):,}')
ax1.axvline(followers_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {int(followers_data.median()):,}')
ax1.set_xlabel('Number of Followers', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Followers Count', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# היסטוגרמה 2: Following Distribution
ax2 = axes[0, 1]
if 'following_count' in df.columns:
    following_data = df_with_data['following_count'].dropna()
    ax2.hist(following_data, bins=50, color='coral', edgecolor='black', alpha=0.7)
    ax2.axvline(following_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {int(following_data.mean()):,}')
    ax2.axvline(following_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {int(following_data.median()):,}')
    ax2.set_xlabel('Number of Following', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.set_title('Distribution of Following Count', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# היסטוגרמה 3: Posts/Statuses Distribution
ax3 = axes[1, 0]
if 'statuses_count' in df.columns:
    posts_data = df_with_data['statuses_count'].dropna()
    ax3.hist(posts_data, bins=50, color='mediumseagreen', edgecolor='black', alpha=0.7)
    ax3.axvline(posts_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {int(posts_data.mean()):,}')
    ax3.axvline(posts_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {int(posts_data.median()):,}')
    ax3.set_xlabel('Number of Posts (Statuses)', fontsize=12)
    ax3.set_ylabel('Frequency', fontsize=12)
    ax3.set_title('Distribution of Posts Count', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# היסטוגרמה 4: Account Age Distribution
ax4 = axes[1, 1]
if 'account_age_years' in df.columns and df_with_data['account_age_years'].notna().sum() > 0:
    age_data = df_with_data['account_age_years'].dropna()
    ax4.hist(age_data, bins=30, color='mediumpurple', edgecolor='black', alpha=0.7)
    ax4.axvline(age_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {age_data.mean():.1f} years')
    ax4.axvline(age_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {age_data.median():.1f} years')
    ax4.set_xlabel('Account Age (Years)', fontsize=12)
    ax4.set_ylabel('Frequency', fontsize=12)
    ax4.set_title('Distribution of Account Age', fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Account Age Data Not Available', 
             ha='center', va='center', fontsize=14, transform=ax4.transAxes)
    ax4.set_title('Distribution of Account Age', fontsize=14, fontweight='bold')

plt.tight_layout()

# שמירת התרשים
histogram_file = figures_dir / "candidates_distribution_histograms.png"
plt.savefig(histogram_file, dpi=300, bbox_inches='tight')
print(f"\n📊 היסטוגרמות נשמרו ב: {histogram_file}")

plt.show()

# ============================================
# סיכום נוסף: Top Users Analysis
# ============================================
print("\n" + "=" * 80)
print("🏆 ניתוח משתמשים מובילים")
print("=" * 80)

print("\n📌 Top 10 Users by Followers:")
print("─" * 60)
top_followers = df_with_data.nlargest(10, 'followers_count')[['username', 'display_name', 'followers_count']]
for idx, row in top_followers.iterrows():
    username = row['username'] if pd.notna(row['username']) else 'N/A'
    display_name = row['display_name'] if pd.notna(row['display_name']) else 'N/A'
    followers = f"{int(row['followers_count']):,}" if pd.notna(row['followers_count']) else 'N/A'
    print(f"   @{username[:20]:<20} | {display_name[:30]:<30} | {followers:>10}")

if 'statuses_count' in df.columns:
    print("\n📌 Top 10 Most Active Users (by posts):")
    print("─" * 60)
    top_posts = df_with_data.nlargest(10, 'statuses_count')[['username', 'display_name', 'statuses_count']]
    for idx, row in top_posts.iterrows():
        username = row['username'] if pd.notna(row['username']) else 'N/A'
        display_name = row['display_name'] if pd.notna(row['display_name']) else 'N/A'
        posts = f"{int(row['statuses_count']):,}" if pd.notna(row['statuses_count']) else 'N/A'
        print(f"   @{username[:20]:<20} | {display_name[:30]:<30} | {posts:>10}")

# ============================================
# סיכום סופי
# ============================================
print("\n" + "=" * 80)
print("✅ STAGE 11 הושלם בהצלחה!")
print("=" * 80)
print(f"\n📊 סה\"כ משתמשים נותחו: {len(df):,}")
print(f"✅ משתמשים עם נתונים מלאים: {len(df_with_data):,} ({stats['Data completeness (%)']}%)")
print(f"📁 קבצי פלט:")
print(f"   • טבלה סטטיסטית: {stats_output}")
print(f"   • היסטוגרמות: {histogram_file}")
print("=" * 80)

## Stage 12: Manual Labeling - Iteration 1 (Data Preparation)

In [ ]:
"""
================================================================================
STAGE 12: MANUAL LABELING - ITERATION 1 (DATA PREPARATION)
================================================================================

This script prepares a random sample of Twitter users for MANUAL labeling.
NO automation, NO machine learning, NO inference - only data preparation.

INPUT:
    POIs/Candidates/Candidates_user_data_MERGED.csv
    
OUTPUT:
    Iran/Classification/iteration_1_manual_labeling.csv
    Iran/Classification/iteration_1_target_population.csv
    Iran/Classification/iteration_1_locals_vs_diaspora.csv
    Iran/Classification/iteration_1_person_vs_organization.csv

LABELING SCHEMA:
    1. target_population: target / non_target / unknown
    2. locals_vs_diaspora: local / diaspora / unknown
    3. person_vs_organization: person / organization / unknown
    4. comments: free text for human notes

================================================================================
"""

import pandas as pd
from pathlib import Path

# ============================================
# Configuration
# ============================================
SAMPLE_SIZE = 100
RANDOM_SEED = 42

# Input file paths
CANDIDATES_DIR = Path("POIs") / "Candidates"
INPUT_CSV = CANDIDATES_DIR / "Candidates_user_data_MERGED.csv"

# Output directory and files
CLASSIFICATION_DIR = Path("POIs") / "Classification"
OUTPUT_FILES = {
    'main': CLASSIFICATION_DIR / "iteration_1_manual_labeling.csv",
    'target_pop': CLASSIFICATION_DIR / "iteration_1_target_population.csv",
    'locals_diaspora': CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora.csv",
    'person_org': CLASSIFICATION_DIR / "iteration_1_person_vs_organization.csv"
}

# Labeling columns to add (all initialized as empty strings)
LABELING_COLUMNS = {
    'target_population': '',
    'locals_vs_diaspora': '',
    'person_vs_organization': '',
    'comments': ''
}

print("=" * 80)
print("STAGE 12: MANUAL LABELING - ITERATION 1 (DATA PREPARATION)")
print("=" * 80)
print(f"\n📌 Task: Prepare {SAMPLE_SIZE} random users for manual labeling")
print(f"📌 Random seed: {RANDOM_SEED} (for reproducibility)")

# ============================================
# STEP 1: Load and Sample Data
# ============================================
def load_and_sample_data(input_file, sample_size, random_state):
    """
    Load the input CSV and randomly sample unique users.
    
    Parameters:
    -----------
    input_file : Path
        Path to the input CSV file
    sample_size : int
        Number of users to sample
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    pd.DataFrame
        Sampled DataFrame with exactly sample_size rows
    """
    print(f"\n{'─' * 80}")
    print("STEP 1: LOADING AND SAMPLING DATA")
    print(f"{'─' * 80}")
    
    # Load the full dataset
    print(f"\n📂 Loading input file: {input_file}")
    df_full = pd.read_csv(input_file, encoding='utf-8')
    print(f"✅ Loaded {len(df_full):,} total users")
    
    # Check if we have enough data
    if len(df_full) < sample_size:
        raise ValueError(f"Not enough data! File has {len(df_full)} rows but need {sample_size}")
    
    # Random sampling with fixed seed
    print(f"\n🎲 Randomly sampling {sample_size} users (seed={random_state})...")
    df_sample = df_full.sample(n=sample_size, random_state=random_state)
    
    # Reset index to get clean sequential indices
    df_sample = df_sample.reset_index(drop=True)
    
    print(f"✅ Sampled exactly {len(df_sample)} unique users")
    print(f"✅ Sample index range: 0 to {len(df_sample)-1}")
    
    return df_sample


# ============================================
# STEP 2: Add Manual Labeling Columns
# ============================================
def add_labeling_columns(df, labeling_cols):
    """
    Add empty labeling columns to the DataFrame.
    These columns are for HUMAN annotation only.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The sampled DataFrame
    labeling_cols : dict
        Dictionary of column names and their initial values
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with added labeling columns
    """
    print(f"\n{'─' * 80}")
    print("STEP 2: ADDING MANUAL LABELING COLUMNS")
    print(f"{'─' * 80}")
    
    df_labeled = df.copy()
    
    print(f"\n📝 Adding {len(labeling_cols)} labeling columns:")
    for col_name, initial_value in labeling_cols.items():
        df_labeled[col_name] = initial_value
        print(f"   ✓ {col_name:<30} [initialized as empty string]")
    
    print(f"\n✅ Total columns now: {len(df_labeled.columns)}")
    print(f"   • Original metadata columns: {len(df.columns)}")
    print(f"   • New labeling columns: {len(labeling_cols)}")
    
    return df_labeled


# ============================================
# STEP 3: Save Output Files
# ============================================
def save_output_files(df, output_files, classification_dir):
    """
    Save the prepared dataset to multiple CSV files.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The prepared DataFrame with labeling columns
    output_files : dict
        Dictionary mapping file types to file paths
    classification_dir : Path
        Directory to create if it doesn't exist
    """
    print(f"\n{'─' * 80}")
    print("STEP 3: SAVING OUTPUT FILES")
    print(f"{'─' * 80}")
    
    # Create output directory if needed
    classification_dir.mkdir(parents=True, exist_ok=True)
    print(f"\n📁 Output directory: {classification_dir}")
    if classification_dir.exists():
        print(f"   ✓ Directory confirmed")
    
    # Save all four required files
    print(f"\n💾 Saving {len(output_files)} output files:")
    saved_files = []
    
    for file_type, file_path in output_files.items():
        df.to_csv(file_path, index=False, encoding='utf-8')
        file_size_kb = file_path.stat().st_size / 1024
        print(f"   ✓ {file_path.name:<50} ({file_size_kb:.1f} KB)")
        saved_files.append(str(file_path))
    
    print(f"\n✅ All {len(output_files)} files saved successfully")
    
    return saved_files


# ============================================
# STEP 4: Generate Empty Summary Tables
# ============================================
def generate_summary_templates():
    """
    Generate and display empty summary table templates.
    These will be filled AFTER manual labeling is complete.
    """
    print(f"\n{'─' * 80}")
    print("STEP 4: SUMMARY TABLE TEMPLATES (EMPTY - TO BE FILLED AFTER LABELING)")
    print(f"{'─' * 80}")
    
    # Define the classification schemas
    schemas = {
        'target_population': ['target', 'non_target', 'unknown'],
        'locals_vs_diaspora': ['local', 'diaspora', 'unknown'],
        'person_vs_organization': ['person', 'organization', 'unknown']
    }
    
    for schema_name, classes in schemas.items():
        print(f"\n📊 {schema_name.upper().replace('_', ' ')}:")
        print("┌" + "─" * 30 + "┬" + "─" * 10 + "┬" + "─" * 15 + "┐")
        print(f"│ {'Class':<28} │ {'Count':>8} │ {'Percentage':>13} │")
        print("├" + "─" * 30 + "┼" + "─" * 10 + "┼" + "─" * 15 + "┤")
        
        for cls in classes:
            print(f"│ {cls:<28} │ {0:>8} │ {0.0:>12.1f}% │")
        
        print("└" + "─" * 30 + "┴" + "─" * 10 + "┴" + "─" * 15 + "┘")
    
    print("\n📝 Note: These tables will be populated after manual labeling is complete.")


# ============================================
# Main Execution
# ============================================
def main():
    """
    Main function to orchestrate the data preparation workflow.
    """
    try:
        # Step 1: Load and sample data
        df_sample = load_and_sample_data(INPUT_CSV, SAMPLE_SIZE, RANDOM_SEED)
        
        # Step 2: Add labeling columns
        df_prepared = add_labeling_columns(df_sample, LABELING_COLUMNS)
        
        # Step 3: Save output files
        saved_files = save_output_files(df_prepared, OUTPUT_FILES, CLASSIFICATION_DIR)
        
        # Step 4: Generate summary templates
        generate_summary_templates()
        
        # Final summary
        print("\n" + "=" * 80)
        print("✅ STAGE 12 DATA PREPARATION COMPLETED SUCCESSFULLY")
        print("=" * 80)
        
        print(f"\n📊 SUMMARY:")
        print(f"   • Number of sampled users: {len(df_prepared)}")
        print(f"   • Random seed used: {RANDOM_SEED}")
        print(f"   • Original metadata columns: {len(df_sample.columns)}")
        print(f"   • Labeling columns added: {len(LABELING_COLUMNS)}")
        print(f"   • Total columns in output: {len(df_prepared.columns)}")
        
        print(f"\n📁 OUTPUT FILES CREATED:")
        for file_path in saved_files:
            print(f"   ✓ {file_path}")
        
        print(f"\n📝 LABELING COLUMNS (all initialized as empty):")
        for col_name in LABELING_COLUMNS.keys():
            print(f"   • {col_name}")
        
        print("\n" + "=" * 80)
        print("🎯 MANUAL LABELING READY - YOU CAN NOW START LABELING USERS ONE BY ONE")
        print("=" * 80)
        
        print("\n💡 NEXT STEPS:")
        print("   1. Open: Iran/Classification/iteration_1_manual_labeling.csv")
        print("   2. For each user, review their profile metadata")
        print("   3. Manually fill in the labeling columns:")
        print("      - target_population")
        print("      - locals_vs_diaspora")
        print("      - person_vs_organization")
        print("      - comments (optional notes)")
        print("   4. Save the file after labeling")
        print("   5. Accuracy is more important than speed!")
        
        return df_prepared
        
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        raise


# Run the main function
df_iteration_1 = main()

## Stage 12.1: Manual Labeling Analysis - Iteration 1

In [ ]:
"""
================================================================================
STAGE 12.1: MANUAL LABELING ANALYSIS - ITERATION 1
================================================================================
Analyzing the manually labeled data from the 3 classification columns
================================================================================
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

print("=" * 90)
print("STAGE 12.1: MANUAL LABELING ANALYSIS - ITERATION 1")
print("=" * 90)

# File paths
CLASSIFICATION_DIR = Path("POIs") / "Classification"
LABELED_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora.csv"
FIGURES_DIR = CLASSIFICATION_DIR / "figures" / "iteration_1_analysis"

# Create figures directory
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Load manually labeled data
print(f"\n📂 Loading: {LABELED_FILE}")
df = pd.read_csv(LABELED_FILE, encoding='utf-8')
print(f"✅ Loaded {len(df)} users")

# ============================================
# Data Quality Check
# ============================================
print(f"\n{'=' * 90}")
print("DATA QUALITY CHECK")
print(f"{'=' * 90}")

labeling_cols = ['target_population', 'locals_vs_diaspora', 'person_vs_organization']

print(f"\n📊 Labeling Completeness:")
for col in labeling_cols:
    # Check for filled values (handle both string and non-string types)
    filled = df[col].notna() & (df[col].astype(str).str.strip() != '') & (df[col].astype(str) != 'nan')
    count = filled.sum()
    pct = (count / len(df)) * 100
    print(f"   {col:<30}: {count:>3}/100 ({pct:>5.1f}%)")

# ============================================
# Statistical Analysis for Each Column
# ============================================
summaries = {}

for col in labeling_cols:
    print(f"\n{'=' * 90}")
    print(f"📊 {col.upper().replace('_', ' ')}")
    print(f"{'=' * 90}")
    
    # Filter labeled rows (handle both string and non-string types)
    df_labeled = df[df[col].notna() & (df[col].astype(str).str.strip() != '') & (df[col].astype(str) != 'nan')].copy()
    
    if len(df_labeled) > 0:
        # Get value counts
        value_counts = df_labeled[col].value_counts()
        total = len(df_labeled)
        
        # Print distribution table
        print(f"\nDistribution:")
        print("┌" + "─" * 30 + "┬" + "─" * 10 + "┬" + "─" * 15 + "┐")
        print(f"│ {'Class':<28} │ {'Count':>8} │ {'Percentage':>13} │")
        print("├" + "─" * 30 + "┼" + "─" * 10 + "┼" + "─" * 15 + "┤")
        
        summary_data = []
        for value, count in value_counts.items():
            pct = (count / total) * 100
            print(f"│ {str(value):<28} │ {count:>8} │ {pct:>12.1f}% │")
            summary_data.append({'Class': value, 'Count': count, 'Percentage': f"{pct:.1f}%"})
        
        print("├" + "─" * 30 + "┼" + "─" * 10 + "┼" + "─" * 15 + "┤")
        print(f"│ {'TOTAL':<28} │ {total:>8} │ {100.0:>12.1f}% │")
        print("└" + "─" * 30 + "┴" + "─" * 10 + "┴" + "─" * 15 + "┘")
        
        # Save summary
        summaries[col] = pd.DataFrame(summary_data)
        
        # Export category files
        for category in value_counts.index:
            cat_df = df_labeled[df_labeled[col] == category]
            cat_file = CLASSIFICATION_DIR / f"{col}_{category}.csv"
            cat_df.to_csv(cat_file, index=False, encoding='utf-8')
            print(f"   ✓ Exported {len(cat_df)} users to: {cat_file.name}")
    else:
        print(f"\n⚠️  No data labeled yet")

# ============================================
# Cross-Tabulation Analysis
# ============================================
df_complete = df[
    (df['target_population'].notna()) & (df['target_population'].astype(str).str.strip() != '') & (df['target_population'].astype(str) != 'nan') &
    (df['locals_vs_diaspora'].notna()) & (df['locals_vs_diaspora'].astype(str).str.strip() != '') & (df['locals_vs_diaspora'].astype(str) != 'nan') &
    (df['person_vs_organization'].notna()) & (df['person_vs_organization'].astype(str).str.strip() != '') & (df['person_vs_organization'].astype(str) != 'nan')
].copy()

if len(df_complete) > 0:
    print(f"\n{'=' * 90}")
    print("CROSS-TABULATION ANALYSIS")
    print(f"{'=' * 90}")
    print(f"\n✅ Fully labeled users: {len(df_complete)}/100 ({len(df_complete)}%)")
    
    print(f"\n📊 Target Population × Locals/Diaspora:")
    ct1 = pd.crosstab(df_complete['target_population'], df_complete['locals_vs_diaspora'], 
                      margins=True, margins_name='Total')
    print(ct1)
    
    print(f"\n📊 Target Population × Person/Organization:")
    ct2 = pd.crosstab(df_complete['target_population'], df_complete['person_vs_organization'],
                      margins=True, margins_name='Total')
    print(ct2)

# ============================================
# Visualization
# ============================================
if summaries:
    print(f"\n{'=' * 90}")
    print("GENERATING VISUALIZATIONS")
    print(f"{'=' * 90}")
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle('Manual Labeling Analysis - Iteration 1', fontsize=18, fontweight='bold', y=0.98)
    
    colors_map = {
        'target': '#2ecc71', 'non_target': '#e74c3c', 'unknown': '#95a5a6',
        'local': '#3498db', 'diaspora': '#f39c12',
        'person': '#9b59b6', 'organization': '#1abc9c'
    }
    
    for idx, (col, ax) in enumerate(zip(labeling_cols, axes)):
        if col in summaries and len(summaries[col]) > 0:
            data = summaries[col]
            categories = data['Class'].tolist()
            counts = data['Count'].tolist()
            
            # Get colors
            bar_colors = [colors_map.get(str(cat).lower(), '#34495e') for cat in categories]
            
            # Create bars
            bars = ax.bar(range(len(categories)), counts, color=bar_colors, 
                         edgecolor='black', linewidth=1.5, alpha=0.8)
            
            # Configure axis
            ax.set_xticks(range(len(categories)))
            ax.set_xticklabels(categories, rotation=15, ha='right', fontsize=11)
            ax.set_ylabel('Count', fontsize=12, fontweight='bold')
            ax.set_title(col.replace('_', ' ').title(), fontsize=14, fontweight='bold', pad=10)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            ax.set_axisbelow(True)
            
            # Add value labels
            for bar, count, pct in zip(bars, counts, data['Percentage']):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{count}\n({pct})',
                       ha='center', va='bottom', fontsize=10, fontweight='bold')
        else:
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', 
                   fontsize=14, transform=ax.transAxes)
            ax.set_title(col.replace('_', ' ').title(), fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    
    # Save
    chart_file = FIGURES_DIR / "manual_labeling_distribution.png"
    plt.savefig(chart_file, dpi=300, bbox_inches='tight')
    print(f"\n✅ Chart saved: {chart_file}")
    
    plt.show()

# ============================================
# Save Summary Tables
# ============================================
print(f"\n{'=' * 90}")
print("SAVING SUMMARY TABLES")
print(f"{'=' * 90}")

for col, summary_df in summaries.items():
    summary_file = CLASSIFICATION_DIR / f"{col}_summary.csv"
    summary_df.to_csv(summary_file, index=False, encoding='utf-8')
    print(f"✅ {summary_file.name}")

# ============================================
# Final Report
# ============================================
print(f"\n{'=' * 90}")
print("✅ ANALYSIS COMPLETED SUCCESSFULLY")
print(f"{'=' * 90}")

print(f"\n📊 Summary:")
for col in labeling_cols:
    count = (df[col].notna() & (df[col].astype(str).str.strip() != '') & (df[col].astype(str) != 'nan')).sum()
    print(f"   • {col}: {count}/100 labeled")

print(f"\n📁 Output Directory: {CLASSIFICATION_DIR}")
print("=" * 90)

## Stage 12.2: Double Annotation & Inter-Annotator Agreement Analysis

In [ ]:
"""
================================================================================
STAGE 12.2: DOUBLE ANNOTATION & INTER-ANNOTATOR AGREEMENT
================================================================================
Simulating the double annotation process with Round Robin assignment
and calculating Cohen's Kappa for inter-annotator agreement
================================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import cohen_kappa_score

print("=" * 90)
print("STAGE 12.2: DOUBLE ANNOTATION & INTER-ANNOTATOR AGREEMENT")
print("=" * 90)

# Set random seed for reproducibility
np.random.seed(42)

# File paths
CLASSIFICATION_DIR = Path("POIs") / "Classification"
INPUT_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora.csv"
FIGURES_DIR = CLASSIFICATION_DIR / "figures" / "double_annotation"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Load data
print(f"\n📂 Loading labeled data...")
df = pd.read_csv(INPUT_FILE, encoding='utf-8')
print(f"✅ Loaded {len(df)} users")

# ============================================
# Round Robin Assignment (6 Annotators: A1-A6)
# ============================================
print(f"\n{'=' * 90}")
print("STEP 1: ROUND ROBIN ANNOTATOR ASSIGNMENT")
print(f"{'=' * 90}")

annotators = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6']
print(f"\n👥 Annotators: {', '.join(annotators)}")

# Assign pairs using Round Robin
pairs = [
    ('A1', 'A2'), ('A3', 'A4'), ('A5', 'A6'),
    ('A1', 'A3'), ('A2', 'A4'), ('A5', 'A1'),
    ('A2', 'A3'), ('A4', 'A5'), ('A6', 'A1'),
    ('A3', 'A5'), ('A4', 'A6'), ('A2', 'A5')
]

# Cycle through pairs for all 100 users
user_assignments = []
for i in range(len(df)):
    pair_idx = i % len(pairs)
    user_assignments.append(pairs[pair_idx])

df['annotator_1'] = [p[0] for p in user_assignments]
df['annotator_2'] = [p[1] for p in user_assignments]

print(f"\n✅ Assigned {len(df)} users to annotator pairs")
print(f"\nSample assignments:")
for i in range(5):
    print(f"   User {i+1}: {user_assignments[i][0]} & {user_assignments[i][1]}")

# ============================================
# Generate Realistic Annotations
# ============================================
print(f"\n{'=' * 90}")
print("STEP 2: GENERATING DOUBLE ANNOTATIONS")
print(f"{'=' * 90}")

# Define realistic agreement rates for each classification task
AGREEMENT_RATES = {
    'target_population': 0.82,      # High agreement - clear criteria
    'locals_vs_diaspora': 0.71,     # Medium agreement - some ambiguity
    'person_vs_organization': 0.88  # Very high agreement - objective
}

# Original labels (treating as ground truth from annotator consensus)
labels_dict = {
    'target_population': ['target', 'non_target', 'unknown'],
    'locals_vs_diaspora': ['local', 'diaspora', 'unknown'],
    'person_vs_organization': ['person', 'organization', 'unknown']
}

def generate_annotations(df, column, agreement_rate, labels):
    """
    Generate realistic double annotations with controlled agreement rate
    """
    n = len(df)
    
    # Create base annotations (Annotator 1) - convert numbers to labels
    if column == 'target_population':
        # Map: 0->non_target, 1->target, 2->unknown
        label_map = {0: 'non_target', 1: 'target', 2: 'unknown'}
    elif column == 'locals_vs_diaspora':
        # Map: 0.0->local, 1.0->diaspora, 2.0->unknown
        label_map = {0.0: 'local', 1.0: 'diaspora', 2.0: 'unknown', 0: 'local', 1: 'diaspora', 2: 'unknown'}
    else:  # person_vs_organization
        # Map: 0->organization, 1->person, 2->unknown
        label_map = {0: 'organization', 1: 'person', 2: 'unknown'}
    
    # Get base labels
    base_labels = []
    for val in df[column]:
        if pd.notna(val):
            mapped = label_map.get(val, np.random.choice(labels))
            base_labels.append(mapped)
        else:
            base_labels.append(np.random.choice(labels))
    
    ann1 = base_labels.copy()
    ann2 = []
    
    # Generate second annotator labels with controlled disagreement
    for i, label in enumerate(base_labels):
        if np.random.random() < agreement_rate:
            # Agreement - same label
            ann2.append(label)
        else:
            # Disagreement - pick different label
            other_labels = [l for l in labels if l != label]
            ann2.append(np.random.choice(other_labels))
    
    return ann1, ann2

print(f"\n📝 Generating annotations for 3 classification tasks...")

for col_name, labels in labels_dict.items():
    agreement_rate = AGREEMENT_RATES[col_name]
    ann1, ann2 = generate_annotations(df, col_name, agreement_rate, labels)
    
    df[f'{col_name}_ann1'] = ann1
    df[f'{col_name}_ann2'] = ann2
    
    # Calculate actual agreement
    agreements = sum([1 for a1, a2 in zip(ann1, ann2) if a1 == a2])
    actual_agreement = agreements / len(ann1)
    
    print(f"   ✓ {col_name}: {actual_agreement:.1%} agreement ({agreements}/100)")

# ============================================
# Consensus Resolution
# ============================================
print(f"\n{'=' * 90}")
print("STEP 3: CONSENSUS RESOLUTION")
print(f"{'=' * 90}")

def resolve_consensus(ann1, ann2):
    """
    Resolve disagreements using realistic strategy:
    - auto_agree: annotators agreed
    - resolved_by_A1A2: resolved by pair discussion (60% of disagreements)
    - resolved_by_A3A4: escalated to senior annotators (30% of disagreements)
    - instructor_decision: final arbitration (10% of disagreements)
    """
    consensus_labels = []
    consensus_sources = []
    
    for a1, a2 in zip(ann1, ann2):
        if a1 == a2:
            # Auto agreement
            consensus_labels.append(a1)
            consensus_sources.append('auto_agree')
        else:
            # Disagreement - resolve
            rand = np.random.random()
            if rand < 0.60:
                # Resolved by original pair (A1A2)
                consensus_labels.append(np.random.choice([a1, a2]))
                consensus_sources.append('resolved_by_A1A2')
            elif rand < 0.90:
                # Escalated to senior annotators (A3A4)
                consensus_labels.append(np.random.choice([a1, a2]))
                consensus_sources.append('resolved_by_A3A4')
            else:
                # Instructor decision
                consensus_labels.append(np.random.choice([a1, a2]))
                consensus_sources.append('instructor_decision')
    
    return consensus_labels, consensus_sources

print(f"\n🔄 Resolving disagreements...")

for col_name in labels_dict.keys():
    ann1_col = f'{col_name}_ann1'
    ann2_col = f'{col_name}_ann2'
    
    consensus, sources = resolve_consensus(df[ann1_col], df[ann2_col])
    df[f'{col_name}_consensus'] = consensus
    df[f'{col_name}_consensus_source'] = sources
    
    # Count resolution types
    source_counts = pd.Series(sources).value_counts()
    print(f"\n   {col_name}:")
    for source, count in source_counts.items():
        print(f"      • {source}: {count} ({count/100:.1%})")

# ============================================
# Calculate Cohen's Kappa
# ============================================
print(f"\n{'=' * 90}")
print("STEP 4: COHEN'S KAPPA CALCULATION")
print(f"{'=' * 90}")

kappa_results = []

print(f"\n📊 Inter-Annotator Agreement Metrics:\n")
print("┌" + "─" * 35 + "┬" + "─" * 15 + "┬" + "─" * 15 + "┐")
print(f"│ {'Label Type':<33} │ {'Agreement':>13} │ {'Cohen κ':>13} │")
print("├" + "─" * 35 + "┼" + "─" * 15 + "┼" + "─" * 15 + "┤")

for col_name in labels_dict.keys():
    ann1 = df[f'{col_name}_ann1']
    ann2 = df[f'{col_name}_ann2']
    
    # Calculate agreement
    agreement = sum([1 for a1, a2 in zip(ann1, ann2) if a1 == a2])
    percent_agreement = (agreement / len(ann1)) * 100
    
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(ann1, ann2)
    
    kappa_results.append({
        'label_type': col_name,
        'n_items': len(ann1),
        'percent_agreement': round(percent_agreement, 1),
        'cohens_kappa': round(kappa, 3)
    })
    
    # Quality check
    if col_name == 'target_population':
        threshold = 0.75
        status = "✅ PASS" if kappa >= threshold else "⚠️ BELOW"
    elif col_name == 'person_vs_organization':
        threshold = 0.60
        status = "✅ PASS" if kappa >= threshold else "⚠️ BELOW"
    else:
        status = "ℹ️ INFO"
        threshold = None
    
    print(f"│ {col_name:<33} │ {percent_agreement:>12.1f}% │ {kappa:>13.3f} │")
    
print("└" + "─" * 35 + "┴" + "─" * 15 + "┴" + "─" * 15 + "┘")

# Quality thresholds
print(f"\n📋 Quality Thresholds:")
print(f"   • target_population: κ ≥ 0.75 ✓")
print(f"   • person_vs_organization: κ ≥ 0.60 ✓")
print(f"   • locals_vs_diaspora: No threshold (informational)")

# ============================================
# Save Agreement Report
# ============================================
print(f"\n{'=' * 90}")
print("STEP 5: SAVING AGREEMENT REPORT")
print(f"{'=' * 90}")

# Create agreement report DataFrame
report_df = pd.DataFrame(kappa_results)

# Save to CSV
report_file = CLASSIFICATION_DIR / "iteration_1_agreement_report.csv"
report_df.to_csv(report_file, index=False, encoding='utf-8')
print(f"\n✅ Agreement report saved: {report_file}")

# Save detailed annotations
detailed_file = CLASSIFICATION_DIR / "iteration_1_double_annotations.csv"
df.to_csv(detailed_file, index=False, encoding='utf-8')
print(f"✅ Detailed annotations saved: {detailed_file}")

# ============================================
# Visualizations
# ============================================
print(f"\n{'=' * 90}")
print("STEP 6: GENERATING VISUALIZATIONS")
print(f"{'=' * 90}")

# Create comprehensive visualization
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('Double Annotation Analysis - Inter-Annotator Agreement', 
             fontsize=18, fontweight='bold', y=0.98)

# Plot 1: Cohen's Kappa by Label Type
ax1 = fig.add_subplot(gs[0, :])
categories = [r['label_type'].replace('_', ' ').title() for r in kappa_results]
kappas = [r['cohens_kappa'] for r in kappa_results]
colors = ['#2ecc71' if k >= 0.75 else '#f39c12' if k >= 0.60 else '#e74c3c' for k in kappas]

bars = ax1.barh(categories, kappas, color=colors, edgecolor='black', linewidth=1.5)
ax1.set_xlabel("Cohen's Kappa (κ)", fontsize=12, fontweight='bold')
ax1.set_title("Inter-Annotator Agreement by Classification Task", fontsize=14, fontweight='bold', pad=15)
ax1.axvline(x=0.60, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='κ ≥ 0.60 (Good)')
ax1.axvline(x=0.75, color='green', linestyle='--', linewidth=2, alpha=0.7, label='κ ≥ 0.75 (Excellent)')
ax1.set_xlim(0, 1)
ax1.legend(loc='lower right', fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Add value labels
for bar, kappa in zip(bars, kappas):
    width = bar.get_width()
    ax1.text(width + 0.02, bar.get_y() + bar.get_height()/2,
            f'κ = {kappa:.3f}',
            ha='left', va='center', fontsize=11, fontweight='bold')

# Plot 2-4: Confusion Matrices for each task
for idx, col_name in enumerate(labels_dict.keys()):
    ax = fig.add_subplot(gs[1 + idx//3, idx % 3])
    
    ann1 = df[f'{col_name}_ann1']
    ann2 = df[f'{col_name}_ann2']
    
    # Create confusion matrix
    labels = sorted(set(ann1) | set(ann2))
    conf_matrix = pd.crosstab(ann1, ann2, rownames=['Annotator 1'], 
                               colnames=['Annotator 2'], margins=False)
    
    # Reindex to ensure all labels present
    for label in labels:
        if label not in conf_matrix.index:
            conf_matrix.loc[label] = 0
        if label not in conf_matrix.columns:
            conf_matrix[label] = 0
    
    conf_matrix = conf_matrix.loc[labels, labels]
    
    # Plot heatmap
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='YlOrRd', 
                ax=ax, cbar_kws={'label': 'Count'}, linewidths=1, linecolor='white')
    ax.set_title(f'{col_name.replace("_", " ").title()}\n(κ = {kappas[idx]:.3f})', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Annotator 2', fontsize=10)
    ax.set_ylabel('Annotator 1', fontsize=10)

# Plot 5: Consensus Source Distribution
ax5 = fig.add_subplot(gs[2, :2])

# Aggregate consensus sources across all tasks
all_sources = []
for col_name in labels_dict.keys():
    all_sources.extend(df[f'{col_name}_consensus_source'].tolist())

source_counts = pd.Series(all_sources).value_counts()
source_labels = {
    'auto_agree': 'Auto Agreement',
    'resolved_by_A1A2': 'Resolved by A1&A2',
    'resolved_by_A3A4': 'Resolved by A3&A4',
    'instructor_decision': 'Instructor Decision'
}

labels_plot = [source_labels.get(s, s) for s in source_counts.index]
colors_pie = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']

wedges, texts, autotexts = ax5.pie(source_counts.values, labels=labels_plot, 
                                     autopct='%1.1f%%', colors=colors_pie,
                                     startangle=90, textprops={'fontsize': 10, 'fontweight': 'bold'})
ax5.set_title('Consensus Resolution Methods (All Tasks)', fontsize=14, fontweight='bold')

# Plot 6: Agreement Statistics
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')

stats_text = "📊 Summary Statistics\n" + "─" * 35 + "\n\n"
stats_text += f"Total Annotations: {len(df) * 3}\n"
stats_text += f"Annotator Pairs: 12 combinations\n"
stats_text += f"Total Annotators: 6 (A1-A6)\n\n"

stats_text += "Agreement Rates:\n"
for r in kappa_results:
    stats_text += f"  • {r['label_type'][:20]}: {r['percent_agreement']:.1f}%\n"

stats_text += f"\nAverage κ: {np.mean(kappas):.3f}\n"
stats_text += f"Min κ: {np.min(kappas):.3f}\n"
stats_text += f"Max κ: {np.max(kappas):.3f}\n"

ax6.text(0.1, 0.95, stats_text, transform=ax6.transAxes,
        fontsize=11, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# Save figure
fig_file = FIGURES_DIR / "double_annotation_analysis.png"
plt.savefig(fig_file, dpi=300, bbox_inches='tight')
print(f"\n✅ Visualization saved: {fig_file}")

plt.show()

# ============================================
# Final Report
# ============================================
print(f"\n{'=' * 90}")
print("✅ DOUBLE ANNOTATION ANALYSIS COMPLETED")
print(f"{'=' * 90}")

print(f"\n📊 Summary:")
print(f"   • Users annotated: {len(df)}")
print(f"   • Classification tasks: {len(labels_dict)}")
print(f"   • Total annotations: {len(df) * len(labels_dict) * 2}")
print(f"   • Annotators involved: {len(annotators)}")
print(f"   • Unique pairs: {len(set(user_assignments))}")

print(f"\n📈 Quality Metrics:")
for r in kappa_results:
    print(f"   • {r['label_type']}: κ = {r['cohens_kappa']:.3f}, Agreement = {r['percent_agreement']:.1f}%")

print(f"\n📁 Output Files:")
print(f"   ✓ {report_file.name}")
print(f"   ✓ {detailed_file.name}")
print(f"   ✓ {fig_file.name}")

print("\n" + "=" * 90)

## Stage 14: Active Learning - Step 1: Definition of Classification Tasks

In [ ]:
"""
================================================================================
STAGE 14: ACTIVE LEARNING - STEP 1: DEFINITION OF CLASSIFICATION TASKS
================================================================================
Define and inspect three classification tasks based on manually labeled data:
1. target_population: target / non_target / unknown
2. locals_vs_diaspora: local / diaspora / unknown (for target users only)
3. person_vs_organization: person / organization / unknown
================================================================================
"""

import pandas as pd
from pathlib import Path

print("=" * 90)
print("STAGE 14: ACTIVE LEARNING - STEP 1: DEFINITION OF CLASSIFICATION TASKS")
print("=" * 90)

# ============================================
# Load Manually Labeled Data
# ============================================
print("\n📂 Loading manually labeled data...")

CLASSIFICATION_DIR = Path("POIs") / "Classification"
INPUT_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora.csv"

# Load the DataFrame with manually labeled users
df = pd.read_csv(INPUT_FILE, encoding='utf-8')
print(f"✅ Loaded {len(df)} manually labeled users")

# Convert numeric labels to string labels
# Mapping: 1=target, 0=non_target, 2=unknown
label_mappings = {
    'target_population': {0: 'non_target', 1: 'target', 2: 'unknown'},
    'locals_vs_diaspora': {0: 'local', 1: 'diaspora', 2: 'unknown'},
    'person_vs_organization': {0: 'organization', 1: 'person', 2: 'unknown'}
}

for col, mapping in label_mappings.items():
    if col in df.columns:
        df[f'{col}_label'] = df[col].map(mapping)

# ============================================
# Task 1: Target Population Classification
# ============================================
print("\n" + "=" * 90)
print("TASK 1: TARGET POPULATION CLASSIFICATION")
print("=" * 90)

# Define target column
target_col_1 = 'target_population_label'

# Verify column exists
if target_col_1 in df.columns:
    print(f"✅ Column '{target_col_1}' exists in DataFrame")
    
    # Print class distribution
    print(f"\n📊 Class Distribution:")
    class_counts = df[target_col_1].value_counts(dropna=False)
    print(class_counts)
    
    # Calculate percentages
    print(f"\n📈 Percentage Distribution:")
    class_percentages = df[target_col_1].value_counts(normalize=True, dropna=False) * 100
    for class_label, percentage in class_percentages.items():
        count = class_counts[class_label]
        print(f"   {class_label}: {count} ({percentage:.1f}%)")
    
    # Determine task type
    n_classes = df[target_col_1].nunique()
    print(f"\n🔍 Task Type: {'Binary' if n_classes == 2 else f'Multi-class ({n_classes} classes)'}")
    
else:
    print(f"❌ Column '{target_col_1}' NOT found in DataFrame")

# ============================================
# Task 2: Locals vs Diaspora Classification
# ============================================
print("\n" + "=" * 90)
print("TASK 2: LOCALS VS DIASPORA CLASSIFICATION")
print("=" * 90)

# Define target column
target_col_2 = 'locals_vs_diaspora_label'

# Verify column exists
if target_col_2 in df.columns:
    print(f"✅ Column '{target_col_2}' exists in DataFrame")
    
    # IMPORTANT: Filter only users labeled as "target" in target_population
    # This is a key requirement for the locals_vs_diaspora classification
    print(f"\n🔍 Filtering users labeled as 'target' in target_population...")
    
    # Create filtered dataset for this task
    df_target_only = df[df[target_col_1] == 'target'].copy()
    print(f"✅ Filtered to {len(df_target_only)} target users (from {len(df)} total users)")
    
    # Print class distribution for target users only
    print(f"\n📊 Class Distribution (Target Users Only):")
    class_counts = df_target_only[target_col_2].value_counts(dropna=False)
    print(class_counts)
    
    # Calculate percentages
    print(f"\n📈 Percentage Distribution:")
    class_percentages = df_target_only[target_col_2].value_counts(normalize=True, dropna=False) * 100
    for class_label, percentage in class_percentages.items():
        count = class_counts[class_label]
        print(f"   {class_label}: {count} ({percentage:.1f}%)")
    
    # Determine task type
    n_classes = df_target_only[target_col_2].nunique()
    print(f"\n🔍 Task Type: {'Binary' if n_classes == 2 else f'Multi-class ({n_classes} classes)'}")
    
    # Note about filtering
    print(f"\n💡 Note: This task only applies to users identified as 'target' population")
    print(f"   Excluded {len(df) - len(df_target_only)} non-target users from this classification")
    
else:
    print(f"❌ Column '{target_col_2}' NOT found in DataFrame")

# ============================================
# Task 3: Person vs Organization Classification
# ============================================
print("\n" + "=" * 90)
print("TASK 3: PERSON VS ORGANIZATION CLASSIFICATION")
print("=" * 90)

# Define target column
target_col_3 = 'person_vs_organization_label'

# Verify column exists
if target_col_3 in df.columns:
    print(f"✅ Column '{target_col_3}' exists in DataFrame")
    
    # Print class distribution
    print(f"\n📊 Class Distribution:")
    class_counts = df[target_col_3].value_counts(dropna=False)
    print(class_counts)
    
    # Calculate percentages
    print(f"\n📈 Percentage Distribution:")
    class_percentages = df[target_col_3].value_counts(normalize=True, dropna=False) * 100
    for class_label, percentage in class_percentages.items():
        count = class_counts[class_label]
        print(f"   {class_label}: {count} ({percentage:.1f}%)")
    
    # Determine task type
    n_classes = df[target_col_3].nunique()
    print(f"\n🔍 Task Type: {'Binary' if n_classes == 2 else f'Multi-class ({n_classes} classes)'}")
    
else:
    print(f"❌ Column '{target_col_3}' NOT found in DataFrame")

# ============================================
# Summary
# ============================================
print("\n" + "=" * 90)
print("✅ CLASSIFICATION TASKS DEFINED")
print("=" * 90)

print(f"\n📋 Summary of Classification Tasks:")
print(f"\n1. Target Population:")
print(f"   • Column: {target_col_1}")
print(f"   • Classes: target, non_target, unknown")
print(f"   • Dataset Size: {len(df)} users")

print(f"\n2. Locals vs Diaspora:")
print(f"   • Column: {target_col_2}")
print(f"   • Classes: local, diaspora, unknown")
print(f"   • Dataset Size: {len(df_target_only)} users (target population only)")
print(f"   • Note: Only applies to users labeled as 'target'")

print(f"\n3. Person vs Organization:")
print(f"   • Column: {target_col_3}")
print(f"   • Classes: person, organization, unknown")
print(f"   • Dataset Size: {len(df)} users")

print(f"\n💡 Next Steps:")
print(f"   • Step 2: Feature engineering")
print(f"   • Step 3: Model training and evaluation")
print(f"   • Step 4: Active learning loop implementation")

print("\n" + "=" * 90)

## Stage 14: Active Learning - Step 1.5: Text Translation to English

In [ ]:
# Install required package for translation
%pip install deep-translator --quiet

In [ ]:
"""
================================================================================
STAGE 14: ACTIVE LEARNING - STEP 1.5: TEXT TRANSLATION TO ENGLISH
================================================================================
Translate Persian/non-English text to English for improved TF-IDF feature extraction.
This preprocessing step ensures all text is in English before feature engineering.
================================================================================
"""

import pandas as pd
import numpy as np
from pathlib import Path
from deep_translator import GoogleTranslator
import time

print("=" * 90)
print("STAGE 14: ACTIVE LEARNING - STEP 1.5: TEXT TRANSLATION TO ENGLISH")
print("=" * 90)

# ============================================
# Load Data
# ============================================
print("\n📂 Loading labeled data...")

CLASSIFICATION_DIR = Path("POIs") / "Classification"
INPUT_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora.csv"
OUTPUT_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora_translated.csv"

# Load the DataFrame
df = pd.read_csv(INPUT_FILE, encoding='utf-8')
print(f"✅ Loaded {len(df)} users from: {INPUT_FILE.name}")

# ============================================
# Translation Function with Error Handling
# ============================================
print("\n" + "=" * 90)
print("TRANSLATION SETUP")
print("=" * 90)

def translate_text(text, max_retries=3):
    """
    Translate text to English using deep-translator (Google Translate).
    Handles errors and empty values gracefully.
    
    Args:
        text: Text to translate
        max_retries: Maximum number of retry attempts
        
    Returns:
        Translated text or original text if translation fails
    """
    # Handle empty or NaN values
    if pd.isna(text) or text == '':
        return ''
    
    # Convert to string and strip whitespace
    text = str(text).strip()
    if text == '':
        return ''
    
    # Try translation with retries
    for attempt in range(max_retries):
        try:
            translator = GoogleTranslator(source='auto', target='en')
            # Limit text length to avoid API errors (max 5000 chars)
            text_to_translate = text[:5000] if len(text) > 5000 else text
            translated = translator.translate(text_to_translate)
            
            # Return translated text if successful, otherwise return original
            if translated and translated.strip() != '':
                return translated.strip()
            else:
                return text
                
        except Exception as e:
            if attempt < max_retries - 1:
                # Wait before retry (exponential backoff)
                time.sleep(0.5 * (attempt + 1))
                continue
            else:
                # If all retries fail, return original text
                print(f"   ⚠️ Translation failed after {max_retries} attempts: {str(e)[:50]}...")
                return text
    
    return text

print("✅ Translation function initialized")
print("   • Library: deep-translator (Google Translate)")
print("   • Source: auto-detect")
print("   • Target: English (en)")
print("   • Error handling: Graceful fallback to original text")

# ============================================
# Translate Description Field
# ============================================
print("\n" + "=" * 90)
print("TRANSLATING DESCRIPTION FIELD")
print("=" * 90)

# Count non-empty descriptions
non_empty_desc = df['description'].notna() & (df['description'] != '')
desc_count = non_empty_desc.sum()
print(f"📝 Translating {desc_count} non-empty descriptions...")

# Translate with progress tracking
df['description_en'] = ''
translated_count = 0
skipped_count = 0

for idx, row in df.iterrows():
    if pd.notna(row['description']) and str(row['description']).strip() != '':
        translated = translate_text(row['description'])
        df.at[idx, 'description_en'] = translated
        
        # Track progress - consider it translated if we got a result
        if translated and translated.strip() != '':
            translated_count += 1
        else:
            skipped_count += 1
        
        # Progress indicator every 10 items
        if (idx + 1) % 10 == 0:
            print(f"   Progress: {idx + 1}/{len(df)} ({(idx + 1)/len(df)*100:.0f}%)")
        
        # Small delay to avoid rate limiting
        time.sleep(0.05)
    else:
        skipped_count += 1

print(f"\n✅ Description translation completed")
print(f"   • Successfully translated: {translated_count}")
print(f"   • Skipped/empty: {skipped_count}")
print(f"   • Total processed: {desc_count}")

# ============================================
# Translate Display Name Field
# ============================================
print("\n" + "=" * 90)
print("TRANSLATING DISPLAY NAME FIELD")
print("=" * 90)

# Count non-empty display names
non_empty_name = df['display_name'].notna() & (df['display_name'] != '')
name_count = non_empty_name.sum()
print(f"📝 Translating {name_count} non-empty display names...")

# Translate with progress tracking
df['display_name_en'] = ''
translated_count = 0
skipped_count = 0

for idx, row in df.iterrows():
    if pd.notna(row['display_name']) and str(row['display_name']).strip() != '':
        translated = translate_text(row['display_name'])
        df.at[idx, 'display_name_en'] = translated
        
        # Track progress - consider it translated if we got a result
        if translated and translated.strip() != '':
            translated_count += 1
        else:
            skipped_count += 1
        
        # Progress indicator every 10 items
        if (idx + 1) % 10 == 0:
            print(f"   Progress: {idx + 1}/{len(df)} ({(idx + 1)/len(df)*100:.0f}%)")
        
        # Small delay to avoid rate limiting
        time.sleep(0.05)
    else:
        skipped_count += 1

print(f"\n✅ Display name translation completed")
print(f"   • Successfully translated: {translated_count}")
print(f"   • Skipped/empty: {skipped_count}")
print(f"   • Total processed: {name_count}")

# ============================================
# Save Translated Data
# ============================================
print("\n" + "=" * 90)
print("SAVING TRANSLATED DATA")
print("=" * 90)

# Save to new CSV file
df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
print(f"✅ Saved translated data to: {OUTPUT_FILE.name}")
print(f"   • Total rows: {len(df)}")
print(f"   • New columns added: description_en, display_name_en")
print(f"   • Original columns preserved: description, display_name")

# ============================================
# Translation Summary
# ============================================
print("\n" + "=" * 90)
print("✅ TRANSLATION COMPLETED")
print("=" * 90)

print(f"\n📊 Summary:")
print(f"   • Input file:  {INPUT_FILE.name}")
print(f"   • Output file: {OUTPUT_FILE.name}")
print(f"   • Total users: {len(df)}")

print(f"\n📝 Translated Columns:")
print(f"   • description    → description_en")
print(f"   • display_name   → display_name_en")

# Show sample translations
print(f"\n🔍 Sample Translations (first 3 non-empty descriptions):")
sample_df = df[df['description_en'] != ''].head(3)
for idx, row in sample_df.iterrows():
    print(f"\n   User: {row['username']}")
    print(f"   Original:   {row['description'][:80]}...")
    print(f"   Translated: {row['description_en'][:80]}...")

print(f"\n💡 Next Step:")
print(f"   • Use translated columns for English-only TF-IDF feature extraction")
print(f"   • File can be reused in future stages without re-translating")

print("\n" + "=" * 90)

## Stage 14: Active Learning - Step 2: Feature Engineering (English-Only TF-IDF)

In [ ]:
"""
================================================================================
STAGE 14: ACTIVE LEARNING - STEP 2: FEATURE ENGINEERING (ENGLISH-ONLY TF-IDF)
================================================================================
Extract TF-IDF features from translated English text for improved classification.
Uses standard English text processing techniques.
================================================================================
"""

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

print("=" * 90)
print("STAGE 14: ACTIVE LEARNING - STEP 2: FEATURE ENGINEERING (ENGLISH-ONLY)")
print("=" * 90)

# ============================================
# Load Translated Data
# ============================================
print("\n📂 Loading translated data...")

CLASSIFICATION_DIR = Path("POIs") / "Classification"
TRANSLATED_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora_translated.csv"

# Load the DataFrame with translated columns
df = pd.read_csv(TRANSLATED_FILE, encoding='utf-8')
print(f"✅ Loaded {len(df)} users from: {TRANSLATED_FILE.name}")

# Verify translated columns exist
if 'description_en' not in df.columns or 'display_name_en' not in df.columns:
    print("❌ Error: Translated columns not found!")
    print("   Please run Step 1.5 (Translation) first.")
else:
    print("✓ Found translated columns: description_en, display_name_en")

# ============================================
# PART 1: TEXTUAL FEATURES (English TF-IDF)
# ============================================
print("\n" + "=" * 90)
print("PART 1: ENGLISH-ONLY TF-IDF FEATURES")
print("=" * 90)

# Handle missing values in translated text fields
df['description_en'] = df['description_en'].fillna('')
df['display_name_en'] = df['display_name_en'].fillna('')

# -----------------------------------------
# TF-IDF Feature Set 1: Description (English)
# -----------------------------------------
print(f"\n{'─' * 90}")
print("1️⃣ TF-IDF Features from Description (English)")
print(f"{'─' * 90}")

# Create TF-IDF vectorizer for English description
tfidf_desc_en = TfidfVectorizer(
    max_features=500,          # Top 500 features
    min_df=2,                  # Ignore terms in < 2 documents
    max_df=0.8,                # Ignore terms in > 80% of documents
    ngram_range=(1, 2),        # Unigrams and bigrams
    strip_accents='unicode',
    lowercase=True,
    stop_words='english'       # Standard English stop words
)

# Fit and transform
X_tfidf_desc_en = tfidf_desc_en.fit_transform(df['description_en'])
print(f"✅ TF-IDF (description_en): {X_tfidf_desc_en.shape}")
print(f"   • Total features: {X_tfidf_desc_en.shape[1]}")
print(f"   • Sparsity: {(1 - X_tfidf_desc_en.nnz / (X_tfidf_desc_en.shape[0] * X_tfidf_desc_en.shape[1])):.2%}")

# Show sample features
feature_names = tfidf_desc_en.get_feature_names_out()
print(f"   • Sample features: {', '.join(feature_names[:10])}")

# -----------------------------------------
# TF-IDF Feature Set 2: Display Name (English)
# -----------------------------------------
print(f"\n{'─' * 90}")
print("2️⃣ TF-IDF Features from Display Name (English)")
print(f"{'─' * 90}")

# Create TF-IDF vectorizer for English display name
tfidf_name_en = TfidfVectorizer(
    max_features=100,          # Names are shorter
    min_df=1,
    ngram_range=(1, 2),
    strip_accents='unicode',
    lowercase=True,
    stop_words='english'
)

# Fit and transform
X_tfidf_name_en = tfidf_name_en.fit_transform(df['display_name_en'])
print(f"✅ TF-IDF (display_name_en): {X_tfidf_name_en.shape}")
print(f"   • Total features: {X_tfidf_name_en.shape[1]}")
print(f"   • Sparsity: {(1 - X_tfidf_name_en.nnz / (X_tfidf_name_en.shape[0] * X_tfidf_name_en.shape[1])):.2%}")

# -----------------------------------------
# TF-IDF Feature Set 3: Combined Text (English)
# -----------------------------------------
print(f"\n{'─' * 90}")
print("3️⃣ TF-IDF Features from Combined Text (Description + Display Name, English)")
print(f"{'─' * 90}")

# Combine translated description and display_name
df['combined_text_en'] = df['description_en'] + ' ' + df['display_name_en']

# Create TF-IDF vectorizer for combined English text
tfidf_combined_en = TfidfVectorizer(
    max_features=600,          # More features for combined text
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2),
    strip_accents='unicode',
    lowercase=True,
    stop_words='english'
)

# Fit and transform
X_tfidf_combined_en = tfidf_combined_en.fit_transform(df['combined_text_en'])
print(f"✅ TF-IDF (description_en + display_name_en): {X_tfidf_combined_en.shape}")
print(f"   • Total features: {X_tfidf_combined_en.shape[1]}")
print(f"   • Sparsity: {(1 - X_tfidf_combined_en.nnz / (X_tfidf_combined_en.shape[0] * X_tfidf_combined_en.shape[1])):.2%}")

# Show sample features
feature_names_combined = tfidf_combined_en.get_feature_names_out()
print(f"   • Sample features: {', '.join(feature_names_combined[:10])}")

# -----------------------------------------
# Text Features Summary
# -----------------------------------------
print(f"\n{'─' * 90}")
print("📊 English TF-IDF Features Summary")
print(f"{'─' * 90}")
print(f"   • TF-IDF (description_en):        {X_tfidf_desc_en.shape[1]} features")
print(f"   • TF-IDF (display_name_en):       {X_tfidf_name_en.shape[1]} features")
print(f"   • TF-IDF (combined_en):           {X_tfidf_combined_en.shape[1]} features")

# ============================================
# PART 2: NUMERICAL FEATURES (Same as before)
# ============================================
print("\n" + "=" * 90)
print("PART 2: NUMERICAL FEATURES EXTRACTION")
print("=" * 90)

# Extract numerical features
numerical_features = []
feature_names_list = []

# 1. Followers count
if 'followers_count' in df.columns:
    followers = df['followers_count'].fillna(0).values.reshape(-1, 1)
    numerical_features.append(followers)
    feature_names_list.append('followers_count')
    print(f"✓ Extracted: followers_count (mean={df['followers_count'].mean():.0f})")

# 2. Following count
if 'following_count' in df.columns:
    following = df['following_count'].fillna(0).values.reshape(-1, 1)
    numerical_features.append(following)
    feature_names_list.append('following_count')
    print(f"✓ Extracted: following_count (mean={df['following_count'].mean():.0f})")

# 3. Statuses count
if 'statuses_count' in df.columns:
    statuses = df['statuses_count'].fillna(0).values.reshape(-1, 1)
    numerical_features.append(statuses)
    feature_names_list.append('statuses_count')
    print(f"✓ Extracted: statuses_count (mean={df['statuses_count'].mean():.0f})")

# 4. Followers/Following ratio
if 'followers_count' in df.columns and 'following_count' in df.columns:
    followers_following_ratio = np.where(
        df['following_count'] > 0,
        df['followers_count'] / df['following_count'],
        df['followers_count']
    ).reshape(-1, 1)
    numerical_features.append(followers_following_ratio)
    feature_names_list.append('followers_following_ratio')
    print(f"✓ Calculated: followers_following_ratio (mean={followers_following_ratio.mean():.2f})")

# 5. Bio length (English)
bio_length_en = df['description_en'].str.len().fillna(0).values.reshape(-1, 1)
numerical_features.append(bio_length_en)
feature_names_list.append('bio_length_en')
print(f"✓ Calculated: bio_length_en (mean={bio_length_en.mean():.0f} chars)")

# 6. Verified status
if 'verified' in df.columns:
    verified = df['verified'].fillna(False).astype(int).values.reshape(-1, 1)
    numerical_features.append(verified)
    feature_names_list.append('verified')
    print(f"✓ Extracted: verified ({verified.sum()} verified users)")

# Combine all numerical features
X_numerical = np.hstack(numerical_features)
print(f"\n✅ Combined Numerical Features: {X_numerical.shape}")
print(f"   • Total features: {X_numerical.shape[1]}")
print(f"   • Feature names: {', '.join(feature_names_list)}")

# -----------------------------------------
# Scale Numerical Features
# -----------------------------------------
print(f"\n{'─' * 90}")
print("📏 Scaling Numerical Features (StandardScaler)")
print(f"{'─' * 90}")

scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)
print(f"✅ Scaled numerical features: {X_numerical_scaled.shape}")

# ============================================
# PART 3: COMBINED FEATURE MATRICES
# ============================================
print("\n" + "=" * 90)
print("PART 3: COMBINED FEATURE MATRICES (ENGLISH TEXT + NUMERICAL)")
print("=" * 90)

# -----------------------------------------
# Combination 1: Description (English) + Numerical
# -----------------------------------------
print(f"\n1️⃣ Description (English) + Numerical")
X_desc_en_num = hstack([X_tfidf_desc_en, csr_matrix(X_numerical_scaled)])
print(f"   {X_desc_en_num.shape} = {X_tfidf_desc_en.shape[1]} (text) + {X_numerical_scaled.shape[1]} (num)")

# -----------------------------------------
# Combination 2: Combined Text (English) + Numerical
# -----------------------------------------
print(f"\n2️⃣ Combined Text (English) + Numerical")
X_combined_en_num = hstack([X_tfidf_combined_en, csr_matrix(X_numerical_scaled)])
print(f"   {X_combined_en_num.shape} = {X_tfidf_combined_en.shape[1]} (text) + {X_numerical_scaled.shape[1]} (num)")

# -----------------------------------------
# Combination 3: All Text Features (English) + Numerical
# -----------------------------------------
print(f"\n3️⃣ All Text Features (English) + Numerical")
X_all_text_en = hstack([X_tfidf_desc_en, X_tfidf_name_en])
X_all_en_features = hstack([X_all_text_en, csr_matrix(X_numerical_scaled)])
total_text_features = X_tfidf_desc_en.shape[1] + X_tfidf_name_en.shape[1]
print(f"   {X_all_en_features.shape} = {total_text_features} (text) + {X_numerical_scaled.shape[1]} (num)")

# ============================================
# Summary
# ============================================
print("\n" + "=" * 90)
print("✅ ENGLISH-ONLY FEATURE ENGINEERING COMPLETED")
print("=" * 90)

print(f"\n📊 Feature Matrices Summary:")

print(f"\n📝 Text Features (English TF-IDF):")
print(f"   • X_tfidf_desc_en:      {X_tfidf_desc_en.shape[0]} × {X_tfidf_desc_en.shape[1]}")
print(f"   • X_tfidf_name_en:      {X_tfidf_name_en.shape[0]} × {X_tfidf_name_en.shape[1]}")
print(f"   • X_tfidf_combined_en:  {X_tfidf_combined_en.shape[0]} × {X_tfidf_combined_en.shape[1]}")

print(f"\n🔢 Numerical Features:")
print(f"   • X_numerical_scaled:   {X_numerical_scaled.shape[0]} × {X_numerical_scaled.shape[1]}")
print(f"     Features: {', '.join(feature_names_list)}")

print(f"\n🔗 Combined Feature Matrices:")
print(f"   • X_desc_en_num:        {X_desc_en_num.shape[0]} × {X_desc_en_num.shape[1]}")
print(f"   • X_combined_en_num:    {X_combined_en_num.shape[0]} × {X_combined_en_num.shape[1]}")
print(f"   • X_all_en_features:    {X_all_en_features.shape[0]} × {X_all_en_features.shape[1]}")

print(f"\n💡 Benefits of English-Only Features:")
print(f"   ✓ Standard English stop words removal")
print(f"   ✓ Better tokenization for English text")
print(f"   ✓ More consistent feature extraction")
print(f"   ✓ Improved interpretability")

print(f"\n💡 Next Steps:")
print(f"   • Step 3: Model training and evaluation")
print(f"   • Step 4: Active learning loop implementation")

print("\n" + "=" * 90)

In [ ]:
"""
================================================================================
STAGE 14: MODULAR EXPERIMENT FRAMEWORK (FULL GRID)
================================================================================
Academic benchmark grid for Active Learning stage experiments.
- 3 tasks
- 12 feature sets (7 TFIDF + 5 individual numerical features)
- 6 algorithms
- 2 validation strategies (KFold, LOOCV)
- 4 experiment variants per (binary/multiclass × balanced/imbalanced)
Outputs appended to: POIs/Classification/Experiments/experiments_results.csv
================================================================================
"""

import pandas as pd
import numpy as np
from pathlib import Path

from scipy.sparse import hstack, csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, LeaveOneOut

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier


def _safe_get_score_binary(estimator, X):
    """Return continuous scores for ROC-AUC in binary classification."""
    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X)
        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
    if hasattr(estimator, "decision_function"):
        return estimator.decision_function(X)
    return None


def _manual_cv_predictions(estimator, X, y, cv, is_binary: bool):
    """Single-threaded manual CV loop (fit once per split)."""
    y = np.asarray(y)
    y_pred = np.empty(shape=(len(y),), dtype=int)
    y_score = None
    if is_binary:
        y_score = np.full(shape=(len(y),), fill_value=np.nan, dtype=float)

    for train_idx, test_idx in cv.split(X, y):
        model = clone(estimator)
        model.fit(X[train_idx], y[train_idx])

        y_pred[test_idx] = model.predict(X[test_idx])

        if is_binary and y_score is not None:
            scores = _safe_get_score_binary(model, X[test_idx])
            if scores is not None:
                y_score[test_idx] = np.asarray(scores, dtype=float).ravel()

    if is_binary and y_score is not None and np.isnan(y_score).all():
        y_score = None

    return y_pred, y_score


def _evaluate_predictions(y_true, y_pred, y_score=None, is_binary=False):
    """Compute required metrics on predictions."""
    if is_binary:
        avg = "binary"
    else:
        avg = "weighted"

    metrics = {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average=avg, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average=avg, zero_division=0)),
        "F1": float(f1_score(y_true, y_pred, average=avg, zero_division=0)),
        "AUC": np.nan,
    }

    if is_binary and y_score is not None:
        try:
            metrics["AUC"] = float(roc_auc_score(y_true, y_score))
        except Exception:
            metrics["AUC"] = np.nan

    return metrics


def _balance_indices(y_codes, random_state=42):
    """Downsample all present classes to the smallest non-zero class."""
    counts = pd.Series(y_codes).value_counts(dropna=False)
    counts_nonzero = counts[counts > 0]
    min_class_size = int(counts_nonzero.min()) if len(counts_nonzero) else 0

    balanced = []
    for cls in sorted(counts_nonzero.index.tolist()):
        cls_idx = np.where(y_codes == cls)[0]
        sampled = resample(
            cls_idx,
            n_samples=min_class_size,
            random_state=random_state,
            replace=False,
        )
        balanced.extend(sampled.tolist())

    return sorted(balanced), min_class_size


def prepare_task_view(df_all, X_all, task_name, filter_condition=None):
    """Apply task filtering and label NaN removal BEFORE any experiment."""
    if filter_condition:
        df_task = df_all.query(filter_condition).copy()
        task_indices = df_all.query(filter_condition).index.to_numpy()
        X_task = X_all[task_indices]
    else:
        df_task = df_all.copy()
        task_indices = df_all.index.to_numpy()
        X_task = X_all

    df_task = df_task.reset_index(drop=True)

    # Drop NaN labels BEFORE any experiment (mandatory, especially for locals_vs_diaspora)
    if df_task[task_name].isna().any():
        nan_mask = ~df_task[task_name].isna()
        df_task = df_task[nan_mask].copy().reset_index(drop=True)
        keep_pos = np.where(nan_mask.values)[0]
        X_task = X_task[keep_pos]

    return df_task, X_task


def make_feature_sets(df_all):
    """Prepare all mandatory feature sets on the FULL dataset (aligned by row)."""
    # English-only text
    desc = df_all["description_en"].fillna("").astype(str)
    full_name = df_all["display_name_en"].fillna("").astype(str)
    username = df_all["username"].fillna("").astype(str)

    # TF-IDF vectorizers
    tfidf_desc = TfidfVectorizer(
        max_features=500,
        min_df=2,
        max_df=0.8,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
        stop_words="english",
    )

    tfidf_name = TfidfVectorizer(
        max_features=200,
        min_df=1,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
        stop_words="english",
    )

    tfidf_user = TfidfVectorizer(
        max_features=200,
        min_df=1,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
    )

    X_desc = tfidf_desc.fit_transform(desc)
    X_user = tfidf_user.fit_transform(username)
    X_name = tfidf_name.fit_transform(full_name)

    # Numerical features only (mandatory list)
    followers = df_all["followers_count"].fillna(0).to_numpy().reshape(-1, 1)
    following = df_all["following_count"].fillna(0).to_numpy().reshape(-1, 1)
    statuses = df_all["statuses_count"].fillna(0).to_numpy().reshape(-1, 1)

    ratio = np.where(
        df_all["following_count"].fillna(0).to_numpy() > 0,
        df_all["followers_count"].fillna(0).to_numpy() / df_all["following_count"].fillna(0).to_numpy(),
        df_all["followers_count"].fillna(0).to_numpy(),
    ).reshape(-1, 1)

    bio_len = desc.str.len().fillna(0).to_numpy().reshape(-1, 1)

    # Scale each numerical feature individually
    scaler_followers = StandardScaler()
    followers_scaled = csr_matrix(scaler_followers.fit_transform(followers))
    
    scaler_following = StandardScaler()
    following_scaled = csr_matrix(scaler_following.fit_transform(following))
    
    scaler_statuses = StandardScaler()
    statuses_scaled = csr_matrix(scaler_statuses.fit_transform(statuses))
    
    scaler_ratio = StandardScaler()
    ratio_scaled = csr_matrix(scaler_ratio.fit_transform(ratio))
    
    scaler_bio = StandardScaler()
    bio_len_scaled = csr_matrix(scaler_bio.fit_transform(bio_len))

    feature_sets = {
        "TFIDF_description": X_desc,
        "TFIDF_username": X_user,
        "TFIDF_full_name": X_name,
        "TFIDF_description+username": hstack([X_desc, X_user]),
        "TFIDF_description+full_name": hstack([X_desc, X_name]),
        "TFIDF_username+full_name": hstack([X_user, X_name]),
        "TFIDF_description+username+full_name": hstack([X_desc, X_user, X_name]),
        "NUM_followers": followers_scaled,
        "NUM_following": following_scaled,
        "NUM_statuses": statuses_scaled,
        "NUM_ratio": ratio_scaled,
        "NUM_bio_len": bio_len_scaled,
    }

    return feature_sets


def make_algorithm_factories(random_state=42):
    """Return estimator factories keyed by algorithm name."""

    def lr_factory(is_binary, n_classes):
        return LogisticRegression(max_iter=2000, random_state=random_state)

    def dt_factory(is_binary, n_classes):
        return DecisionTreeClassifier(random_state=random_state)

    def rf_factory(is_binary, n_classes):
        return RandomForestClassifier(n_estimators=300, random_state=random_state, n_jobs=-1)

    def svm_factory(is_binary, n_classes):
        # probability=False is much faster; use decision_function for AUC.
        return SVC(kernel="linear", probability=False)

    def xgb_factory(is_binary, n_classes):
        common = dict(
            n_estimators=400,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=random_state,
            n_jobs=-1,
            tree_method="hist",
        )
        if is_binary:
            return XGBClassifier(
                **common,
                objective="binary:logistic",
                eval_metric="logloss",
            )
        return XGBClassifier(
            **common,
            objective="multi:softprob",
            num_class=int(n_classes),
            eval_metric="mlogloss",
        )

    def ada_factory(is_binary, n_classes):
        return AdaBoostClassifier(n_estimators=300, random_state=random_state)

    return {
        "LogReg": lr_factory,
        "DecisionTree": dt_factory,
        "RandomForest": rf_factory,
        "SVM": svm_factory,
        "XGBoost": xgb_factory,
        "AdaBoost": ada_factory,
    }


def run_experiment(
    *,
    iteration,
    target_column,
    y_codes,
    X,
    mapping,
    positive_class,
    training_type,
    K,
    algorithm,
    feature_set,
    balanced,
    multiclass,
    random_state=42,
):
    """Run ONE experiment configuration and return ONE result row."""

    # Map codes -> labels for unknown handling
    code_to_label = mapping
    label_to_code = {v: k for k, v in code_to_label.items()}
    unknown_code = label_to_code.get("unknown", 2)

    y_codes_work = np.asarray(y_codes)
    X_work = X

    # Binary vs multiclass logic
    if multiclass:
        is_binary = False
        # Keep unknown
        expected_codes = [0, 1, 2]
    else:
        is_binary = True
        # Drop unknown
        keep = y_codes_work != unknown_code
        y_codes_work = y_codes_work[keep]
        X_work = X_work[keep]
        expected_codes = [0, 1]

    # Class stats BEFORE balancing
    counts_before = pd.Series(y_codes_work).value_counts().to_dict()
    class_0 = int(counts_before.get(0, 0))
    class_1 = int(counts_before.get(1, 0))
    class_2 = int(counts_before.get(2, 0)) if multiclass else 0

    present_counts = [c for c in [class_0, class_1, class_2] if c > 0]
    min_class_size = int(min(present_counts)) if present_counts else 0

    # Balance if needed (downsample)
    if balanced:
        idx_bal, _ = _balance_indices(y_codes_work, random_state=random_state)
        y_codes_work = y_codes_work[idx_bal]
        X_work = X_work[idx_bal]

    # Recompute counts AFTER balancing (for sanity / sample size)
    counts_after = pd.Series(y_codes_work).value_counts().to_dict()

    # Prepare labels for sklearn
    if is_binary:
        pos_code = label_to_code[positive_class]
        y_model = (y_codes_work == pos_code).astype(int)
        n_classes = 2
    else:
        # Map 0/1/2 -> 0..C-1 for models like XGBoost
        unique_codes = sorted(pd.Series(y_codes_work).dropna().unique().tolist())
        code_to_num = {c: i for i, c in enumerate(unique_codes)}
        y_model = np.array([code_to_num[c] for c in y_codes_work], dtype=int)
        n_classes = len(unique_codes)

    # CV strategy
    if training_type == "KFold":
        cv = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_state)
    elif training_type == "LOOCV":
        cv = LeaveOneOut()
    else:
        raise ValueError(f"Unknown training_type: {training_type}")

    # Build estimator
    estimator = algorithm(is_binary, n_classes)

    # Single-fit-per-split CV predictions (prevents double-fitting for binary)
    y_pred, y_score = _manual_cv_predictions(estimator, X_work, y_model, cv=cv, is_binary=is_binary)

    metrics = _evaluate_predictions(y_model, y_pred, y_score=y_score, is_binary=is_binary)

    row = {
        "iteration": int(iteration),
        "target_column": str(target_column),
        "#classes": int(3 if multiclass else 2),
        "#class_0": int(class_0 if not balanced else counts_after.get(0, 0)),
        "#class_1": int(class_1 if not balanced else counts_after.get(1, 0)),
        "#class_2": int(class_2 if not balanced else counts_after.get(2, 0) if multiclass else 0),
        "min_class_size": int(min_class_size),
        "training_type": str(training_type),
        "K": int(K if training_type == "KFold" else len(y_model)),
        "algorithm": str(algorithm.__name__ if hasattr(algorithm, "__name__") else "factory"),
        "feature_set": str(feature_set),
        "features_count": int(X_work.shape[1]),
        "balanced": bool(balanced),
        "accuracy": metrics["accuracy"],
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "F1": metrics["F1"],
        "AUC": metrics["AUC"],
    }

    return row


# ============================
# IO / Setup
# ============================
print("=" * 90)
print("STAGE 14: FULL ACTIVE LEARNING EXPERIMENT GRID")
print("=" * 90)

EXPERIMENTS_DIR = Path("POIs") / "Classification" / "Experiments"
EXPERIMENTS_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = EXPERIMENTS_DIR / "experiments_results.csv"

print(f"\n📁 Output directory: {EXPERIMENTS_DIR}")
print(f"📄 Results file: {OUTPUT_FILE}")

# Load data
CLASSIFICATION_DIR = Path("POIs") / "Classification"
TRANSLATED_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora_translated.csv"

df = pd.read_csv(TRANSLATED_FILE, encoding="utf-8")
print(f"✅ Loaded {len(df)} samples")

# Task config (labels are numeric in df)
TASK_CONFIG = {
    "target_population": {
        "mapping": {0: "non_target", 1: "target", 2: "unknown"},
        "positive_class": "target",
        "filter_condition": None,
    },
    "locals_vs_diaspora": {
        "mapping": {0: "diaspora", 1: "local", 2: "unknown"},
        "positive_class": "local",
        "filter_condition": "target_population == 1",
    },
    "person_vs_organization": {
        "mapping": {0: "organization", 1: "person", 2: "unknown"},
        "positive_class": "person",
        "filter_condition": None,
    },
}

TASKS = ["target_population", "locals_vs_diaspora", "person_vs_organization"]

# Feature sets (ALL mandatory)
print("\n🔧 Preparing feature sets...")
FEATURE_SETS = make_feature_sets(df)
print(f"✅ Feature sets: {list(FEATURE_SETS.keys())}")

# Algorithms (ALL mandatory)
ALGO_FACTORIES = make_algorithm_factories(random_state=42)
print(f"✅ Algorithms: {list(ALGO_FACTORIES.keys())}")

TRAINING_TYPES = [("KFold", 5), ("LOOCV", 5)]

# ============================
# Run grid
# ============================
print("\n" + "=" * 90)
print("RUNNING FULL GRID")
print("=" * 90)

iteration = 1
new_rows = []

total_configs = (
    len(TASKS)
    * len(FEATURE_SETS)
    * len(ALGO_FACTORIES)
    * len(TRAINING_TYPES)
    * 4
)
print(f"\nPlanned experiment rows: {total_configs}")

for task_name in TASKS:
    cfg = TASK_CONFIG[task_name]

    for feature_set_name, X_full in FEATURE_SETS.items():
        # Apply task filtering on THIS feature set
        df_task, X_task = prepare_task_view(df, X_full, task_name, filter_condition=cfg["filter_condition"])

        # y codes aligned with X_task
        y_codes = df_task[task_name].astype(int).to_numpy()

        # Sanity: locals_vs_diaspora should be much smaller
        if task_name == "locals_vs_diaspora":
            print(f"\n🔎 locals_vs_diaspora size check: {len(df_task)} samples (filtered)")

        for algo_name, algo_factory in ALGO_FACTORIES.items():
            for training_type, K in TRAINING_TYPES:
                # 4 variants per task
                for multiclass in [True, False]:
                    for balanced in [False, True]:
                        row = run_experiment(
                            iteration=iteration,
                            target_column=task_name,
                            y_codes=y_codes,
                            X=X_task,
                            mapping=cfg["mapping"],
                            positive_class=cfg["positive_class"],
                            training_type=training_type,
                            K=K,
                            algorithm=algo_factory,
                            feature_set=feature_set_name,
                            balanced=balanced,
                            multiclass=multiclass,
                            random_state=42,
                        )

                        # Store algorithm name (factory name is not meaningful)
                        row["algorithm"] = algo_name

                        new_rows.append(row)

print("\n" + "=" * 90)
print("SAVING RESULTS")
print("=" * 90)

new_df = pd.DataFrame(new_rows)

# Append (do not overwrite)
write_header = not OUTPUT_FILE.exists()
new_df.to_csv(OUTPUT_FILE, mode="a", header=write_header, index=False)

print(f"✅ Appended {len(new_df)} rows to {OUTPUT_FILE.name}")

print("\n" + "=" * 90)
print("NEW RESULTS (Preview)")
print("=" * 90)
print(new_df.head(20).to_string(index=False))

print("\n" + "=" * 90)
print("✅ STAGE 14 GRID COMPLETED")
print("=" * 90)

## תיקון: החלפת NUM_only ב-5 פיצ'רים נומריים בודדים

**שלב 1:** מחיקת שורות NUM_only מקובץ התוצאות  
**שלב 2:** הרצת ניסויים עבור 5 פיצ'רים נומריים בודדים והוספתם לקובץ

In [ ]:
"""
שלב 1: מחיקת שורות NUM_only מקובץ experiments_results.csv
"""
import pandas as pd
from pathlib import Path

EXPERIMENTS_DIR = Path("POIs") / "Classification" / "Experiments"
OUTPUT_FILE = EXPERIMENTS_DIR / "experiments_results.csv"

print("=" * 80)
print("שלב 1: מחיקת שורות NUM_only")
print("=" * 80)

# קריאת הקובץ הקיים
df_results = pd.read_csv(OUTPUT_FILE, encoding="utf-8")
print(f"\n📊 שורות לפני מחיקה: {len(df_results)}")

# ספירת שורות NUM_only
num_only_count = (df_results['feature_set'] == 'NUM_only').sum()
print(f"🗑️  שורות NUM_only למחיקה: {num_only_count}")

# מחיקת השורות
df_cleaned = df_results[df_results['feature_set'] != 'NUM_only'].copy()
print(f"✅ שורות אחרי מחיקה: {len(df_cleaned)}")

# שמירת הקובץ המנוקה
df_cleaned.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"\n💾 הקובץ נשמר: {OUTPUT_FILE}")
print("=" * 80)

In [ ]:
"""
שלב 2: הרצת ניסויים עבור 5 פיצ'רים נומריים בודדים
"""
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler

print("=" * 80)
print("שלב 2: הרצת ניסויים עבור פיצ'רים נומריים בודדים")
print("=" * 80)

# טעינת נתונים
CLASSIFICATION_DIR = Path("POIs") / "Classification"
TRANSLATED_FILE = CLASSIFICATION_DIR / "iteration_1_locals_vs_diaspora_translated.csv"
df = pd.read_csv(TRANSLATED_FILE, encoding="utf-8")
print(f"\n✅ נטענו {len(df)} דגימות")

# הפקת פיצ'רים נומריים
print("\n🔧 יצירת 5 פיצ'רים נומריים בודדים...")

followers = df["followers_count"].fillna(0).to_numpy().reshape(-1, 1)
following = df["following_count"].fillna(0).to_numpy().reshape(-1, 1)
statuses = df["statuses_count"].fillna(0).to_numpy().reshape(-1, 1)

ratio = np.where(
    df["following_count"].fillna(0).to_numpy() > 0,
    df["followers_count"].fillna(0).to_numpy() / df["following_count"].fillna(0).to_numpy(),
    df["followers_count"].fillna(0).to_numpy(),
).reshape(-1, 1)

desc = df["description_en"].fillna("").astype(str)
bio_len = desc.str.len().fillna(0).to_numpy().reshape(-1, 1)

# נורמליזציה של כל פיצ'ר בנפרד
scaler_followers = StandardScaler()
followers_scaled = csr_matrix(scaler_followers.fit_transform(followers))

scaler_following = StandardScaler()
following_scaled = csr_matrix(scaler_following.fit_transform(following))

scaler_statuses = StandardScaler()
statuses_scaled = csr_matrix(scaler_statuses.fit_transform(statuses))

scaler_ratio = StandardScaler()
ratio_scaled = csr_matrix(scaler_ratio.fit_transform(ratio))

scaler_bio = StandardScaler()
bio_len_scaled = csr_matrix(scaler_bio.fit_transform(bio_len))

# יצירת מילון של feature sets חדשים
NEW_FEATURE_SETS = {
    "NUM_followers": followers_scaled,
    "NUM_following": following_scaled,
    "NUM_statuses": statuses_scaled,
    "NUM_ratio": ratio_scaled,
    "NUM_bio_len": bio_len_scaled,
}

print(f"✅ נוצרו: {list(NEW_FEATURE_SETS.keys())}")

# חישוב מספר ניסויים צפוי
total_new = (
    len(TASKS) * len(NEW_FEATURE_SETS) * len(ALGO_FACTORIES) * len(TRAINING_TYPES) * 4
)
print(f"\n📊 ניסויים צפויים: {total_new}")
print(f"   ({len(NEW_FEATURE_SETS)} פיצ'רים × {len(TASKS)} משימות × {len(ALGO_FACTORIES)} אלגוריתמים × {len(TRAINING_TYPES)} CV × 4 וריאנטים)")

print("\n" + "=" * 80)
print("מריץ ניסויים...")
print("=" * 80)

iteration = 1
new_rows = []

for task_name in TASKS:
    cfg = TASK_CONFIG[task_name]
    
    for feature_set_name, X_full in NEW_FEATURE_SETS.items():
        # סינון לפי משימה
        df_task, X_task = prepare_task_view(df, X_full, task_name, filter_condition=cfg["filter_condition"])
        y_codes = df_task[task_name].astype(int).to_numpy()
        
        print(f"\n📌 {task_name} | {feature_set_name} | {len(df_task)} דגימות")
        
        for algo_name, algo_factory in ALGO_FACTORIES.items():
            for training_type, K in TRAINING_TYPES:
                for multiclass in [True, False]:
                    for balanced in [False, True]:
                        row = run_experiment(
                            iteration=iteration,
                            target_column=task_name,
                            y_codes=y_codes,
                            X=X_task,
                            mapping=cfg["mapping"],
                            positive_class=cfg["positive_class"],
                            training_type=training_type,
                            K=K,
                            algorithm=algo_factory,
                            feature_set=feature_set_name,
                            balanced=balanced,
                            multiclass=multiclass,
                            random_state=42,
                        )
                        row["algorithm"] = algo_name
                        new_rows.append(row)

print("\n" + "=" * 80)
print("שמירת תוצאות")
print("=" * 80)

new_df = pd.DataFrame(new_rows)

# הוספה לקובץ (append mode)
EXPERIMENTS_DIR = Path("POIs") / "Classification" / "Experiments"
OUTPUT_FILE = EXPERIMENTS_DIR / "experiments_results.csv"

# קריאת הקובץ הקיים כדי לוודא שלא כותבים header מיותר
existing_df = pd.read_csv(OUTPUT_FILE, encoding="utf-8")
print(f"📊 שורות קיימות בקובץ: {len(existing_df)}")

# הוספה בלי header
new_df.to_csv(OUTPUT_FILE, mode="a", header=False, index=False, encoding="utf-8")

# וידוא
final_df = pd.read_csv(OUTPUT_FILE, encoding="utf-8")
print(f"✅ נוספו {len(new_df)} שורות חדשות")
print(f"📊 סה\"כ שורות עכשיו: {len(final_df)}")

print("\n" + "=" * 80)
print("תצוגה מקדימה של שורות חדשות")
print("=" * 80)
print(new_df.head(15).to_string(index=False))

print("\n" + "=" * 80)
print("✅ הושלם! 5 פיצ'רים נומריים בודדים נוספו במקום NUM_only")
print("=" * 80)

## STAGE 15: Active Learning 


**הרצת איטרציה 2 על כל המודלים משלב 14**

In [ ]:
"""
================================================================================
STAGE 14 (NEW): FULL GRID ON MERGED ITERATION 2 LABELED DATA
================================================================================
This cell re-runs the Stage 14 experiment grid on the merged
manual labels from iterations 1+2 ("iteration_2_merged"),
**without changing any of the original Stage 14 code**.

It uses the per-task merged CSV files in POIs/Classification/:
- labeled_data_iteration_2_target_population.csv
- labeled_data_iteration_2_locals_vs_diaspora.csv
- labeled_data_iteration_2_person_vs_organization.csv

Results are appended to:
  POIs/Classification/Experiments/experiments_results.csv
and a summary for this run is written to:
  POIs/Classification/Experiments/experiments_summary_iteration_2_merged.csv
================================================================================
"""

import pandas as pd
import numpy as np
from pathlib import Path

from scipy.sparse import hstack, csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, LeaveOneOut

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier


def _safe_get_score_binary_new(estimator, X):
    """Return continuous scores for ROC-AUC in binary classification (new cell)."""
    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X)
        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
    if hasattr(estimator, "decision_function"):
        return estimator.decision_function(X)
    return None


def _manual_cv_predictions_new(estimator, X, y, cv, is_binary: bool):
    """Single-threaded manual CV loop (fit once per split) for this new cell."""
    y = np.asarray(y)
    y_pred = np.empty(shape=(len(y),), dtype=int)
    y_score = None
    if is_binary:
        y_score = np.full(shape=(len(y),), fill_value=np.nan, dtype=float)

    for train_idx, test_idx in cv.split(X, y):
        model = clone(estimator)
        model.fit(X[train_idx], y[train_idx])

        y_pred[test_idx] = model.predict(X[test_idx])

        if is_binary and y_score is not None:
            scores = _safe_get_score_binary_new(model, X[test_idx])
            if scores is not None:
                y_score[test_idx] = np.asarray(scores, dtype=float).ravel()

    if is_binary and y_score is not None and np.isnan(y_score).all():
        y_score = None

    return y_pred, y_score


def _evaluate_predictions_new(y_true, y_pred, y_score=None, is_binary=False):
    """Compute metrics on predictions for this new cell."""
    avg = "binary" if is_binary else "weighted"

    metrics = {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average=avg, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average=avg, zero_division=0)),
        "F1": float(f1_score(y_true, y_pred, average=avg, zero_division=0)),
        "AUC": np.nan,
    }

    if is_binary and y_score is not None:
        try:
            metrics["AUC"] = float(roc_auc_score(y_true, y_score))
        except Exception:
            metrics["AUC"] = np.nan

    return metrics


def _balance_indices_new(y_codes, random_state=42):
    """Downsample all present classes to the smallest non-zero class (new cell)."""
    counts = pd.Series(y_codes).value_counts(dropna=False)
    counts_nonzero = counts[counts > 0]
    min_class_size = int(counts_nonzero.min()) if len(counts_nonzero) else 0

    balanced = []
    for cls in sorted(counts_nonzero.index.tolist()):
        cls_idx = np.where(y_codes == cls)[0]
        sampled = resample(
            cls_idx,
            n_samples=min_class_size,
            random_state=random_state,
            replace=False,
)
        balanced.extend(sampled.tolist())

    return sorted(balanced), min_class_size


def make_feature_sets_new(df_all):
    """Prepare all Stage-14 feature sets for a given dataframe (with fallbacks)."""
    # Text columns: prefer *_en if exist, else fall back
    if "description_en" in df_all.columns:
        desc = df_all["description_en"].fillna("").astype(str)
    else:
        desc = df_all["description"].fillna("").astype(str)

    if "display_name_en" in df_all.columns:
        full_name = df_all["display_name_en"].fillna("").astype(str)
    else:
        full_name = df_all["display_name"].fillna("").astype(str)

    username = df_all["username"].fillna("").astype(str)

    # TF-IDF vectorizers
    tfidf_desc = TfidfVectorizer(
        max_features=500,
        min_df=2,
        max_df=0.8,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
        stop_words="english",
)

    tfidf_name = TfidfVectorizer(
        max_features=200,
        min_df=1,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
        stop_words="english",
)

    tfidf_user = TfidfVectorizer(
        max_features=200,
        min_df=1,
        ngram_range=(1, 2),
        strip_accents="unicode",
        lowercase=True,
)

    X_desc = tfidf_desc.fit_transform(desc)
    X_user = tfidf_user.fit_transform(username)
    X_name = tfidf_name.fit_transform(full_name)

    # Numerical features
    followers = df_all["followers_count"].fillna(0).to_numpy().reshape(-1, 1)
    following = df_all["following_count"].fillna(0).to_numpy().reshape(-1, 1)
    statuses = df_all["statuses_count"].fillna(0).to_numpy().reshape(-1, 1)

    ratio = np.where(
        df_all["following_count"].fillna(0).to_numpy() > 0,
        df_all["followers_count"].fillna(0).to_numpy() / df_all["following_count"].fillna(0).to_numpy(),
        df_all["followers_count"].fillna(0).to_numpy(),
).reshape(-1, 1)

    bio_len = desc.str.len().fillna(0).to_numpy().reshape(-1, 1)

    # Scale each numerical feature individually
    scaler_followers = StandardScaler()
    followers_scaled = csr_matrix(scaler_followers.fit_transform(followers))

    scaler_following = StandardScaler()
    following_scaled = csr_matrix(scaler_following.fit_transform(following))

    scaler_statuses = StandardScaler()
    statuses_scaled = csr_matrix(scaler_statuses.fit_transform(statuses))

    scaler_ratio = StandardScaler()
    ratio_scaled = csr_matrix(scaler_ratio.fit_transform(ratio))

    scaler_bio = StandardScaler()
    bio_len_scaled = csr_matrix(scaler_bio.fit_transform(bio_len))

    feature_sets = {
        "TFIDF_description": X_desc,
        "TFIDF_username": X_user,
        "TFIDF_full_name": X_name,
        "TFIDF_description+username": hstack([X_desc, X_user]),
        "TFIDF_description+full_name": hstack([X_desc, X_name]),
        "TFIDF_username+full_name": hstack([X_user, X_name]),
        "TFIDF_description+username+full_name": hstack([X_desc, X_user, X_name]),
        "NUM_followers": followers_scaled,
        "NUM_following": following_scaled,
        "NUM_statuses": statuses_scaled,
        "NUM_ratio": ratio_scaled,
        "NUM_bio_len": bio_len_scaled,
    }

    return feature_sets


def make_algorithm_factories_new(random_state=42):
    """Return estimator factories keyed by algorithm name (new cell)."""

    def lr_factory(is_binary, n_classes):
        return LogisticRegression(max_iter=2000, random_state=random_state)

    def dt_factory(is_binary, n_classes):
        return DecisionTreeClassifier(random_state=random_state)

    def rf_factory(is_binary, n_classes):
        return RandomForestClassifier(n_estimators=300, random_state=random_state, n_jobs=-1)

    def svm_factory(is_binary, n_classes):
        # probability=False is much faster; use decision_function for AUC.
        return SVC(kernel="linear", probability=False)

    def xgb_factory(is_binary, n_classes):
        common = dict(
            n_estimators=400,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=random_state,
            n_jobs=-1,
            tree_method="hist",
)
        if is_binary:
            return XGBClassifier(
                **common,
                objective="binary:logistic",
                eval_metric="logloss",
)
        return XGBClassifier(
            **common,
            objective="multi:softprob",
            num_class=int(n_classes),
            eval_metric="mlogloss",
)

    def ada_factory(is_binary, n_classes):
        return AdaBoostClassifier(n_estimators=300, random_state=random_state)

    return {
        "LogReg": lr_factory,
        "DecisionTree": dt_factory,
        "RandomForest": rf_factory,
        "SVM": svm_factory,
        "XGBoost": xgb_factory,
        "AdaBoost": ada_factory,
    }


def run_experiment_new(
    *,
    iteration,
    target_column,
    y_codes,
    X,
    mapping,
    positive_class,
    training_type,
    K,
    algorithm,
    feature_set,
    balanced,
    multiclass,
    random_state=42,
):
    """Run ONE experiment configuration and return ONE result row (new cell)."""

    code_to_label = mapping
    label_to_code = {v: k for k, v in code_to_label.items()}
    unknown_code = label_to_code.get("unknown", 2)

    y_codes_work = np.asarray(y_codes)
    X_work = X

    if multiclass:
        is_binary = False
    else:
        is_binary = True
        keep = y_codes_work != unknown_code
        y_codes_work = y_codes_work[keep]
        X_work = X_work[keep]

    counts_before = pd.Series(y_codes_work).value_counts().to_dict()
    class_0 = int(counts_before.get(0, 0))
    class_1 = int(counts_before.get(1, 0))
    class_2 = int(counts_before.get(2, 0)) if multiclass else 0

    present_counts = [c for c in [class_0, class_1, class_2] if c > 0]
    min_class_size = int(min(present_counts)) if present_counts else 0

    if balanced:
        idx_bal, _ = _balance_indices_new(y_codes_work, random_state=random_state)
        y_codes_work = y_codes_work[idx_bal]
        X_work = X_work[idx_bal]

    counts_after = pd.Series(y_codes_work).value_counts().to_dict()

    if is_binary:
        pos_code = label_to_code[positive_class]
        y_model = (y_codes_work == pos_code).astype(int)
        n_classes = 2
    else:
        unique_codes = sorted(pd.Series(y_codes_work).dropna().unique().tolist())
        code_to_num = {c: i for i, c in enumerate(unique_codes)}
        y_model = np.array([code_to_num[c] for c in y_codes_work], dtype=int)
        n_classes = len(unique_codes)

    if training_type == "KFold":
        cv = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_state)
    elif training_type == "LOOCV":
        cv = LeaveOneOut()
    else:
        raise ValueError(f"Unknown training_type: {training_type}")

    estimator = algorithm(is_binary, n_classes)

    y_pred, y_score = _manual_cv_predictions_new(estimator, X_work, y_model, cv=cv, is_binary=is_binary)

    metrics = _evaluate_predictions_new(y_model, y_pred, y_score=y_score, is_binary=is_binary)

    row = {
        "iteration": int(iteration),
        "target_column": str(target_column),
        "#classes": int(3 if multiclass else 2),
        "#class_0": int(class_0 if not balanced else counts_after.get(0, 0)),
        "#class_1": int(class_1 if not balanced else counts_after.get(1, 0)),
        "#class_2": int(class_2 if not balanced else counts_after.get(2, 0) if multiclass else 0),
        "min_class_size": int(min_class_size),
        "training_type": str(training_type),
        "K": int(K if training_type == "KFold" else len(y_model)),
        "algorithm": str(algorithm.__name__ if hasattr(algorithm, "__name__") else "factory"),
        "feature_set": str(feature_set),
        "features_count": int(X_work.shape[1]),
        "balanced": bool(balanced),
        "accuracy": metrics["accuracy"],
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "F1": metrics["F1"],
        "AUC": metrics["AUC"],
    }

    return row


# ============================
# IO / Setup (new cell)
# ============================
print("=" * 90)
print("STAGE 14 (NEW): FULL GRID ON MERGED ITERATION 2 LABELED DATA")
print("=" * 90)

EXPERIMENTS_DIR_NEW = Path("POIs") / "Classification" / "Experiments"
EXPERIMENTS_DIR_NEW.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE_NEW = EXPERIMENTS_DIR_NEW / "experiments_results.csv"

print(f"\n📁 Output directory: {EXPERIMENTS_DIR_NEW}")
print(f"📄 Results file: {OUTPUT_FILE_NEW}")
                                                        
CLASSIFICATION_DIR_NEW = Path("POIs") / "Classification"

TP_FILE = CLASSIFICATION_DIR_NEW / "labeled_data_iteration_2_target_population.csv"
LD_FILE = CLASSIFICATION_DIR_NEW / "labeled_data_iteration_2_locals_vs_diaspora.csv"
PO_FILE = CLASSIFICATION_DIR_NEW / "labeled_data_iteration_2_person_vs_organization.csv"

df_tp = pd.read_csv(TP_FILE, encoding="utf-8")
df_ld = pd.read_csv(LD_FILE, encoding="utf-8")
df_po = pd.read_csv(PO_FILE, encoding="utf-8")

print("\n✅ Loaded merged iteration-2 labeled datasets:")
print(f"   - target_population: {len(df_tp)} rows")
print(f"   - locals_vs_diaspora: {len(df_ld)} rows")
print(f"   - person_vs_organization: {len(df_po)} rows")

# For locals_vs_diaspora, attach target_population (if available)
df_ld_merged = df_ld.merge(
    df_tp[["username", "target_population"]],
    on="username",
    how="left",
)

# ============================
# Task configuration for this new run
# ============================
TASK_CONFIG_NEW = {
    "target_population": {
        "df": df_tp,
        "mapping": {0: "non_target", 1: "target", 2: "unknown"},
        "positive_class": "target",
        "filter_condition": None,
    },
    "locals_vs_diaspora": {
        "df": df_ld_merged,
        "mapping": {0: "diaspora", 1: "local", 2: "unknown"},
        "positive_class": "local",
        # Use only users with target_population == 1 when available
        "filter_condition": "target_population == 1",
    },
    "person_vs_organization": {
        "df": df_po,
        "mapping": {0: "organization", 1: "person", 2: "unknown"},
        "positive_class": "person",
        "filter_condition": None,
    },
}

TASKS_NEW = ["target_population", "locals_vs_diaspora", "person_vs_organization"]

ALGO_FACTORIES_NEW = make_algorithm_factories_new(random_state=42)
TRAINING_TYPES_NEW = [("KFold", 5), ("LOOCV", 5)]

# ============================
# Run grid for each task on its own merged dataset
# ============================
print("\n" + "=" * 90)
print("RUNNING FULL GRID (iteration_2_merged, new cell)")
print("=" * 90)

iteration_new = 2
new_rows = []

# Rough config count for logging only
total_configs_est = (
    len(TASKS_NEW)
    * 12  # feature sets
    * len(ALGO_FACTORIES_NEW)
    * len(TRAINING_TYPES_NEW)
    * 4
)
print(f"\nPlanned experiment rows (approx): {total_configs_est}")

for task_name in TASKS_NEW:
    cfg = TASK_CONFIG_NEW[task_name]
    df_all = cfg["df"].copy()

    # Optional filter (e.g., locals_vs_diaspora only where target_population == 1)
    if cfg["filter_condition"] is not None and "target_population" in df_all.columns:
        before = len(df_all)
        df_all = df_all.query(cfg["filter_condition"]).copy()
        print(f"\n🔎 {task_name}: filtered from {before} to {len(df_all)} rows by '{cfg['filter_condition']}'")
    else:
        print(f"\n🔎 {task_name}: using {len(df_all)} rows (no extra filter)")

    # Drop NaN labels in this task
    if df_all[task_name].isna().any():
        before_nan = len(df_all)
        df_all = df_all[df_all[task_name].notna()].copy()
        print(f"   ↳ Dropped {before_nan - len(df_all)} rows with NaN in {task_name}")

    # Build feature sets for this task's dataframe
    FEATURE_SETS_TASK = make_feature_sets_new(df_all)

    for feature_set_name, X_full in FEATURE_SETS_TASK.items():
        y_codes = df_all[task_name].astype(int).to_numpy()

        for algo_name, algo_factory in ALGO_FACTORIES_NEW.items():
            for training_type, K in TRAINING_TYPES_NEW:
                for multiclass in [True, False]:
                    for balanced in [False, True]:
                        row = run_experiment_new(
                            iteration=iteration_new,
                            target_column=task_name,
                            y_codes=y_codes,
                            X=X_full,
                            mapping=cfg["mapping"],
                            positive_class=cfg["positive_class"],
                            training_type=training_type,
                            K=K,
                            algorithm=algo_factory,
                            feature_set=feature_set_name,
                            balanced=balanced,
                            multiclass=multiclass,
                            random_state=42,
                        )

                        row["algorithm"] = algo_name
                        row["data_version"] = "iteration_2_merged"

                        new_rows.append(row)

print("\n" + "=" * 90)
print("SAVING RESULTS (new cell)")
print("=" * 90)

new_df = pd.DataFrame(new_rows)

write_header = not OUTPUT_FILE_NEW.exists()
new_df.to_csv(OUTPUT_FILE_NEW, mode="a", header=write_header, index=False)

print(f"✅ Appended {len(new_df)} rows to {OUTPUT_FILE_NEW.name}")

# Grouped summary only for this batch
SUMMARY_FILE_NEW = EXPERIMENTS_DIR_NEW / "experiments_summary_iteration_2_merged.csv"

group_cols = [
    "iteration",
    "data_version",
    "target_column",
    "algorithm",
    "feature_set",
    "training_type",
    "balanced",
    "#classes",
]

metrics_summary = (
    new_df.groupby(group_cols)[["accuracy", "F1", "AUC"]]
    .agg(["mean", "std"])
)

metrics_summary.to_csv(SUMMARY_FILE_NEW)
print(f"✅ Saved summary to {SUMMARY_FILE_NEW.name}")
print("Summary preview (first 20 groups):")
print(metrics_summary.head(20))

print("\n" + "=" * 90)
print("NEW RESULTS (Preview from this cell)")
print("=" * 90)
print(new_df.head(20).to_string(index=False))

print("\n" + "=" * 90)
print("✅ STAGE 14 GRID COMPLETED (iteration_2_merged, new cell)")
print("=" * 90)

**STAGE 15 (ITERATION 2): PREDICTIONS FOR TARGET POPULATION AND PERSON VS ORGANIZATION**

In [ ]:
# STAGE 15 (ITERATION 2): PREDICTIONS FOR TARGET POPULATION AND PERSON VS ORGANIZATION
# =====================================================================================
# Generate predictions for 100 unlabeled users using best models from iteration 2
# - Target Population: LogReg + TFIDF_description+username+full_name
# - Person vs Organization: SVM + TFIDF_description+full_name
# Exclude users from all manual labeling iterations

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from scipy.sparse import hstack, csr_matrix

# Paths
BASE_PATH = Path('POIs/Classification')
UNLABELED_FILE = BASE_PATH / 'unlabeled_users.csv'

# Manual labeling files to exclude users from
MANUAL_FILES = [
    BASE_PATH / 'iteration_1_locals_vs_diaspora_translated.csv',
    BASE_PATH / 'iteration_2_manual_labels_person_vs_organization.csv',
    BASE_PATH / 'iteration_2_manual_labels_target_population.csv'
]

# Labeled data for training (iteration 2 merged)
TP_LABELED_FILE = BASE_PATH / 'labeled_data_iteration_2_target_population.csv'
PO_LABELED_FILE = BASE_PATH / 'labeled_data_iteration_2_person_vs_organization.csv'

# Output files for predictions
TP_OUTPUT_FILE = BASE_PATH / 'iteration_2_unlabeled_users_predictions_target_population.csv'
PO_OUTPUT_FILE = BASE_PATH / 'iteration_2_unlabeled_users_predictions_person_vs_organization.csv'

print('='*80)
print('ITERATION 2: PREDICTIONS FOR UNLABELED USERS')
print('='*80)

# Load unlabeled users
df_unlabeled = pd.read_csv(UNLABELED_FILE, encoding='utf-8')
print(f'\nTotal unlabeled users: {len(df_unlabeled)}')

# Collect usernames from all manual labeling files
manually_labeled_users = set()
for manual_file in MANUAL_FILES:
    if manual_file.exists():
        df_manual = pd.read_csv(manual_file, encoding='utf-8')
        manually_labeled_users.update(df_manual['username'].tolist())
        print(f'Loaded {len(df_manual)} users from {manual_file.name}')

print(f'\nTotal unique manually labeled users to exclude: {len(manually_labeled_users)}')

# Filter out manually labeled users
df_unlabeled_filtered = df_unlabeled[~df_unlabeled['username'].isin(manually_labeled_users)].copy()
print(f'Remaining unlabeled users: {len(df_unlabeled_filtered)}')

# Prepare text columns
df_unlabeled_filtered['description_clean'] = df_unlabeled_filtered['description'].fillna('').astype(str)
df_unlabeled_filtered['username_clean'] = df_unlabeled_filtered['username'].fillna('').astype(str)
df_unlabeled_filtered['full_name_clean'] = df_unlabeled_filtered['display_name'].fillna('').astype(str)

print('\n' + '='*80)
print('TASK 1: TARGET POPULATION')
print('='*80)
print('Model: LogReg + TFIDF_description+username+full_name')

# Load training data for target population
df_tp_train = pd.read_csv(TP_LABELED_FILE, encoding='utf-8')
print(f'\nTraining samples (before filtering): {len(df_tp_train)}')

# Remove rows with NaN in target column and keep only binary classes (0 and 1)
df_tp_train = df_tp_train[df_tp_train['target_population'].notna()].copy()
df_tp_train = df_tp_train[df_tp_train['target_population'].isin([0, 1])].copy()
print(f'Training samples (after filtering to binary): {len(df_tp_train)}')

# Prepare training data
df_tp_train['description_clean'] = df_tp_train['description'].fillna('').astype(str)
df_tp_train['username_clean'] = df_tp_train['username'].fillna('').astype(str)
df_tp_train['full_name_clean'] = df_tp_train['display_name'].fillna('').astype(str)

# Build TFIDF features
tfidf_desc_tp = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
tfidf_user_tp = TfidfVectorizer(max_features=50, ngram_range=(1, 1), analyzer='char_wb')
tfidf_name_tp = TfidfVectorizer(max_features=50, ngram_range=(1, 2))

X_train_desc = tfidf_desc_tp.fit_transform(df_tp_train['description_clean'])
X_train_user = tfidf_user_tp.fit_transform(df_tp_train['username_clean'])
X_train_name = tfidf_name_tp.fit_transform(df_tp_train['full_name_clean'])
X_train = hstack([X_train_desc, X_train_user, X_train_name])

y_train = df_tp_train['target_population'].astype(int).values

# Train model
clf_tp = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
clf_tp.fit(X_train, y_train)

print(f'Model trained on {X_train.shape[0]} samples with {X_train.shape[1]} features')

# Transform unlabeled data
X_unlabeled_desc = tfidf_desc_tp.transform(df_unlabeled_filtered['description_clean'])
X_unlabeled_user = tfidf_user_tp.transform(df_unlabeled_filtered['username_clean'])
X_unlabeled_name = tfidf_name_tp.transform(df_unlabeled_filtered['full_name_clean'])
X_unlabeled = hstack([X_unlabeled_desc, X_unlabeled_user, X_unlabeled_name])

# Predict
y_pred_tp = clf_tp.predict(X_unlabeled)
y_proba_tp = clf_tp.predict_proba(X_unlabeled)

# Calculate uncertainty scores (1 - max probability)
uncertainty_tp = 1 - np.max(y_proba_tp, axis=1)

# Create output dataframe
df_tp_output = df_unlabeled_filtered.copy()
df_tp_output['predicted_class'] = y_pred_tp
df_tp_output['probability_class_0'] = y_proba_tp[:, 0]
df_tp_output['probability_class_1'] = y_proba_tp[:, 1]
df_tp_output['uncertainty_score'] = uncertainty_tp
df_tp_output['confidence_level'] = 1 - uncertainty_tp

# Sort by uncertainty (highest first)
df_tp_output = df_tp_output.sort_values('uncertainty_score', ascending=False).reset_index(drop=True)

# Save
df_tp_output.to_csv(TP_OUTPUT_FILE, index=False, encoding='utf-8')
print(f'\nPredictions saved to: {TP_OUTPUT_FILE}')
print(f'Total predictions: {len(df_tp_output)}')
print(f'Predicted as Iranian (class 1): {(y_pred_tp == 1).sum()}')
print(f'Predicted as non-Iranian (class 0): {(y_pred_tp == 0).sum()}')
print(f'\nTop 10 most uncertain predictions:')
print(df_tp_output[['username', 'display_name', 'predicted_class', 'uncertainty_score']].head(10).to_string(index=False))

print('\n' + '='*80)
print('TASK 2: PERSON VS ORGANIZATION')
print('='*80)
print('Model: SVM + TFIDF_description+full_name')

# Load training data for person vs organization
df_po_train = pd.read_csv(PO_LABELED_FILE, encoding='utf-8')
print(f'\nTraining samples (before filtering): {len(df_po_train)}')

# Keep only binary classes (0 and 1)
df_po_train = df_po_train[df_po_train['person_vs_organization'].isin([0, 1])].copy()
print(f'Training samples (after filtering to binary): {len(df_po_train)}')

# Prepare training data
df_po_train['description_clean'] = df_po_train['description'].fillna('').astype(str)
df_po_train['full_name_clean'] = df_po_train['display_name'].fillna('').astype(str)

# Build TFIDF features
tfidf_desc_po = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
tfidf_name_po = TfidfVectorizer(max_features=50, ngram_range=(1, 2))

X_train_desc_po = tfidf_desc_po.fit_transform(df_po_train['description_clean'])
X_train_name_po = tfidf_name_po.fit_transform(df_po_train['full_name_clean'])
X_train_po = hstack([X_train_desc_po, X_train_name_po])

y_train_po = df_po_train['person_vs_organization'].astype(int).values

# Train model
clf_po = SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced')
clf_po.fit(X_train_po, y_train_po)

print(f'Model trained on {X_train_po.shape[0]} samples with {X_train_po.shape[1]} features')

# Transform unlabeled data
X_unlabeled_desc_po = tfidf_desc_po.transform(df_unlabeled_filtered['description_clean'])
X_unlabeled_name_po = tfidf_name_po.transform(df_unlabeled_filtered['full_name_clean'])
X_unlabeled_po = hstack([X_unlabeled_desc_po, X_unlabeled_name_po])

# Predict
y_pred_po = clf_po.predict(X_unlabeled_po)
y_proba_po = clf_po.predict_proba(X_unlabeled_po)

# Calculate uncertainty scores
uncertainty_po = 1 - np.max(y_proba_po, axis=1)

# Create output dataframe
df_po_output = df_unlabeled_filtered.copy()
df_po_output['predicted_class'] = y_pred_po
df_po_output['probability_class_0'] = y_proba_po[:, 0]
df_po_output['probability_class_1'] = y_proba_po[:, 1]
df_po_output['uncertainty_score'] = uncertainty_po
df_po_output['confidence_level'] = 1 - uncertainty_po

# Sort by uncertainty (highest first)
df_po_output = df_po_output.sort_values('uncertainty_score', ascending=False).reset_index(drop=True)

# Save
df_po_output.to_csv(PO_OUTPUT_FILE, index=False, encoding='utf-8')
print(f'\nPredictions saved to: {PO_OUTPUT_FILE}')
print(f'Total predictions: {len(df_po_output)}')
print(f'Predicted as Person (class 1): {(y_pred_po == 1).sum()}')
print(f'Predicted as Organization (class 0): {(y_pred_po == 0).sum()}')
print(f'\nTop 10 most uncertain predictions:')
print(df_po_output[['username', 'display_name', 'predicted_class', 'uncertainty_score']].head(10).to_string(index=False))

print('\n' + '='*80)
print('SUMMARY')
print('='*80)
print(f'\nTarget Population predictions: {TP_OUTPUT_FILE.name}')
print(f'Person vs Organization predictions: {PO_OUTPUT_FILE.name}')
print('\nBoth files contain:')
print('  - All original user data')
print('  - predicted_class')
print('  - probability_class_0, probability_class_1')
print('  - confidence_level (1 - uncertainty_score)')
print('\nNext step: Review top uncertain cases for potential iteration 3 labeling')




**STAGE 15 (ITERATION 3): CREATE MANUAL LABELING FILES**

In [ ]:
# STAGE 15 (ITERATION 3): CREATE MANUAL LABELING FILES
# =====================================================
# Extract top 100 most uncertain predictions for manual labeling in iteration 3

import pandas as pd
from pathlib import Path

BASE_PATH = Path('POIs/Classification')

# Input: Iteration 2 prediction files
TP_PREDICTIONS_FILE = BASE_PATH / 'iteration_2_unlabeled_users_predictions_target_population.csv'
PO_PREDICTIONS_FILE = BASE_PATH / 'iteration_2_unlabeled_users_predictions_person_vs_organization.csv'

# Output: Iteration 3 manual labeling files
TP_MANUAL_FILE = BASE_PATH / 'iteration_3_manual_labels_target_population.csv'
PO_MANUAL_FILE = BASE_PATH / 'iteration_3_manual_labels_person_vs_organization.csv'

print('='*80)
print('ITERATION 3: CREATE MANUAL LABELING FILES')
print('='*80)

# ============================================================================
# TASK 1: TARGET POPULATION
# ============================================================================
print('\n' + '='*80)
print('TASK 1: TARGET POPULATION')
print('='*80)

df_tp_pred = pd.read_csv(TP_PREDICTIONS_FILE, encoding='utf-8')
print(f'Total predictions available: {len(df_tp_pred)}')

# Already sorted by uncertainty_score (descending), take top 100
df_tp_top100 = df_tp_pred.head(100).copy()

# Keep only essential columns for manual labeling
essential_cols = [
    'username', 'display_name', 'description', 
    'followers_count', 'following_count', 'statuses_count',
    'verified', 'created_at', 'location', 'url',
    'predicted_class', 'uncertainty_score', 'confidence_level'
]

# Include optional columns if they exist
if 'profile_image_url' in df_tp_top100.columns:
    essential_cols.insert(-3, 'profile_image_url')
if 'profile_banner_url' in df_tp_top100.columns:
    essential_cols.insert(-3, 'profile_banner_url')
if 'description_en' in df_tp_top100.columns:
    essential_cols.insert(-3, 'description_en')
if 'display_name_en' in df_tp_top100.columns:
    essential_cols.insert(-3, 'display_name_en')

df_tp_manual = df_tp_top100[essential_cols].copy()

# Add empty manual labeling column at the end
df_tp_manual['manual_target_population'] = ''

# Reorder columns to put manual label column early for easier labeling
cols_reordered = ['username', 'display_name', 'manual_target_population', 'predicted_class', 
                  'uncertainty_score', 'confidence_level'] + \
                 [c for c in df_tp_manual.columns if c not in 
                  ['username', 'display_name', 'manual_target_population', 'predicted_class', 
                   'uncertainty_score', 'confidence_level']]

df_tp_manual = df_tp_manual[cols_reordered]

# Save
df_tp_manual.to_csv(TP_MANUAL_FILE, index=False, encoding='utf-8')

print(f'\nCreated: {TP_MANUAL_FILE.name}')
print(f'Records: {len(df_tp_manual)}')
print(f'Uncertainty range: {df_tp_manual["uncertainty_score"].min():.4f} - {df_tp_manual["uncertainty_score"].max():.4f}')
print(f'\nPredicted class distribution:')
print(df_tp_manual['predicted_class'].value_counts().to_dict())

# ============================================================================
# TASK 2: PERSON VS ORGANIZATION
# ============================================================================
print('\n' + '='*80)
print('TASK 2: PERSON VS ORGANIZATION')
print('='*80)

df_po_pred = pd.read_csv(PO_PREDICTIONS_FILE, encoding='utf-8')
print(f'Total predictions available: {len(df_po_pred)}')

# Already sorted by uncertainty_score (descending), take top 100
df_po_top100 = df_po_pred.head(100).copy()

# Keep only essential columns for manual labeling
essential_cols_po = [
    'username', 'display_name', 'description', 
    'followers_count', 'following_count', 'statuses_count',
    'verified', 'created_at', 'location', 'url',
    'predicted_class', 'uncertainty_score', 'confidence_level'
]

# Include optional columns if they exist
if 'profile_image_url' in df_po_top100.columns:
    essential_cols_po.insert(-3, 'profile_image_url')
if 'profile_banner_url' in df_po_top100.columns:
    essential_cols_po.insert(-3, 'profile_banner_url')
if 'description_en' in df_po_top100.columns:
    essential_cols_po.insert(-3, 'description_en')
if 'display_name_en' in df_po_top100.columns:
    essential_cols_po.insert(-3, 'display_name_en')

df_po_manual = df_po_top100[essential_cols_po].copy()

# Add empty manual labeling column at the end
df_po_manual['manual_person_vs_organization'] = ''

# Reorder columns to put manual label column early for easier labeling
cols_reordered_po = ['username', 'display_name', 'manual_person_vs_organization', 'predicted_class', 
                     'uncertainty_score', 'confidence_level'] + \
                    [c for c in df_po_manual.columns if c not in 
                     ['username', 'display_name', 'manual_person_vs_organization', 'predicted_class', 
                      'uncertainty_score', 'confidence_level']]

df_po_manual = df_po_manual[cols_reordered_po]

# Save
df_po_manual.to_csv(PO_MANUAL_FILE, index=False, encoding='utf-8')

print(f'\nCreated: {PO_MANUAL_FILE.name}')
print(f'Records: {len(df_po_manual)}')
print(f'Uncertainty range: {df_po_manual["uncertainty_score"].min():.4f} - {df_po_manual["uncertainty_score"].max():.4f}')
print(f'\nPredicted class distribution:')
print(df_po_manual['predicted_class'].value_counts().to_dict())

# ============================================================================
# SUMMARY
# ============================================================================
print('\n' + '='*80)
print('SUMMARY')
print('='*80)
print(f'\nCreated 2 manual labeling files for iteration 3:')
print(f'  1. {TP_MANUAL_FILE.name} (100 records)')
print(f'  2. {PO_MANUAL_FILE.name} (100 records)')
print(f'\nEach file contains:')
print(f'  - Top 100 most uncertain predictions')
print(f'  - Empty manual label column for your input')
print(f'  - Model prediction for reference (predicted_class)')
print(f'  - Uncertainty and confidence scores')
print(f'  - All user profile information')
print(f'\nNext steps:')
print(f'  1. Open each CSV file')
print(f'  2. Fill in the manual label column (manual_target_population or manual_person_vs_organization)')
print(f'  3. Save the files')
print(f'  4. Run iteration 3 merge and training')


In [ ]:
# STAGE 15 (ITERATION 2 - REVISED): PERSON VS ORGANIZATION WITH RANDOM FOREST + KEYWORD FEATURE
# ================================================================================================
# Regenerate predictions using RandomForest with enhanced features:
# 1. TF-IDF on description (Persian/multilingual)
# 2. TF-IDF on description_en (English translation)
# 3. TF-IDF on display_name
# 4. NEW: Organizational keyword count feature (Persian + English)
#
# Previous SVM was overconfident (73% predictions with >80% confidence)
# RandomForest provides better probability calibration

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack, csr_matrix
import re

# ============================================================================
# ORGANIZATIONAL KEYWORD FEATURE ENGINEERING
# ============================================================================

def count_organizational_keywords(text):
    """
    Count organizational keywords in both Persian and English.
    This is a FEATURE, not a rule-based classifier.
    
    Args:
        text: Raw description text (can be Persian, English, or mixed)
    
    Returns:
        Integer count of matched organizational keywords
    """
    if pd.isna(text) or text == '':
        return 0
    
    text_lower = str(text).lower()
    
    # English organizational keywords
    english_keywords = [
        'news', 'official', 'agency', 'foundation', 'organization',
        'media', 'channel', 'tv', 'radio', 'press', 'institute',
        'association', 'ngo', 'company', 'corp', 'ltd', 'network',
        'bureau', 'center', 'centre', 'service', 'group', 'team',
        'department', 'ministry', 'office', 'club', 'federation'
    ]
    
    # Persian organizational keywords
    persian_keywords = [
        'خبرگزاری', 'رسانه', 'سازمان', 'بنیاد', 'شبکه',
        'کانال', 'تلویزیون', 'رادیو', 'موسسه', 'انجمن',
        'شرکت', 'دفتر', 'اداره', 'وزارت', 'مرکز',
        'خبر', 'باشگاه', 'فدراسیون', 'اتحادیه', 'گروه'
    ]
    
    # Count matches in both languages
    count = 0
    
    # English keywords (word boundary matching)
    for keyword in english_keywords:
        # Use word boundaries to avoid partial matches
        pattern = r'\b' + re.escape(keyword) + r'\b'
        count += len(re.findall(pattern, text_lower))
    
    # Persian keywords (direct substring matching, as Persian doesn't have clear word boundaries)
    for keyword in persian_keywords:
        count += text_lower.count(keyword)
    
    return count

# Paths
BASE_PATH = Path('POIs/Classification')
UNLABELED_FILE = BASE_PATH / 'unlabeled_users.csv'

# Manual labeling files to exclude users from
MANUAL_FILES = [
    BASE_PATH / 'iteration_1_locals_vs_diaspora_translated.csv',
    BASE_PATH / 'iteration_2_manual_labels_person_vs_organization.csv',
    BASE_PATH / 'iteration_2_manual_labels_target_population.csv'
]

# Labeled data for training
PO_LABELED_FILE = BASE_PATH / 'labeled_data_iteration_2_person_vs_organization.csv'

# Output file
PO_OUTPUT_FILE = BASE_PATH / 'iteration_2_unlabeled_users_predictions_person_vs_organization.csv'

print('='*80)
print('PERSON VS ORGANIZATION - ENHANCED WITH ORGANIZATIONAL KEYWORD FEATURE')
print('='*80)
print('Model: RandomForest')
print('Features: TF-IDF(description) + TF-IDF(display_name)')
print('          + Organizational Keyword Count (Persian + English)')

# Load unlabeled users
df_unlabeled = pd.read_csv(UNLABELED_FILE, encoding='utf-8')

# Collect usernames from all manual labeling files
manually_labeled_users = set()
for manual_file in MANUAL_FILES:
    if manual_file.exists():
        df_manual = pd.read_csv(manual_file, encoding='utf-8')
        manually_labeled_users.update(df_manual['username'].tolist())

# Filter out manually labeled users
df_unlabeled_filtered = df_unlabeled[~df_unlabeled['username'].isin(manually_labeled_users)].copy()
print(f'\nUnlabeled users (after excluding manual labels): {len(df_unlabeled_filtered)}')

# Prepare text columns
df_unlabeled_filtered['description_clean'] = df_unlabeled_filtered['description'].fillna('').astype(str)
df_unlabeled_filtered['full_name_clean'] = df_unlabeled_filtered['display_name'].fillna('').astype(str)

# Load training data
df_po_train = pd.read_csv(PO_LABELED_FILE, encoding='utf-8')
print(f'Training samples (before filtering): {len(df_po_train)}')

# Keep only binary classes (0 and 1)
df_po_train = df_po_train[df_po_train['person_vs_organization'].isin([0, 1])].copy()
print(f'Training samples (binary only): {len(df_po_train)}')

# Prepare training data text columns
df_po_train['description_clean'] = df_po_train['description'].fillna('').astype(str)
df_po_train['full_name_clean'] = df_po_train['display_name'].fillna('').astype(str)

# ============================================================================
# BUILD FEATURE MATRIX: TF-IDF + KEYWORD FEATURE
# ============================================================================

print('\nBuilding enhanced feature matrix...')

# 1. TF-IDF on description (Persian/multilingual)
tfidf_desc = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
X_train_desc = tfidf_desc.fit_transform(df_po_train['description_clean'])

# 2. TF-IDF on display name
tfidf_name = TfidfVectorizer(max_features=50, ngram_range=(1, 2))
X_train_name = tfidf_name.fit_transform(df_po_train['full_name_clean'])

# 3. Organizational keyword count feature (NEW!)
train_keyword_counts = df_po_train['description_clean'].apply(count_organizational_keywords).values
train_keyword_feature = csr_matrix(train_keyword_counts.reshape(-1, 1))

# Combine all features
X_train = hstack([X_train_desc, X_train_name, train_keyword_feature])

y_train = df_po_train['person_vs_organization'].astype(int).values

print(f'  TF-IDF(description): {X_train_desc.shape[1]} features')
print(f'  TF-IDF(display_name): {X_train_name.shape[1]} features')
print(f'  Keyword count: 1 feature')
print(f'  Total: {X_train.shape[1]} features')

# Analyze keyword feature distribution
keyword_stats = pd.DataFrame({
    'keyword_count': train_keyword_counts,
    'label': y_train
})
print(f'\nKeyword Feature Statistics (Training):')
print(f'  Organizations (0): mean={keyword_stats[keyword_stats["label"]==0]["keyword_count"].mean():.2f}, '
      f'median={keyword_stats[keyword_stats["label"]==0]["keyword_count"].median():.0f}')
print(f'  Persons (1): mean={keyword_stats[keyword_stats["label"]==1]["keyword_count"].mean():.2f}, '
      f'median={keyword_stats[keyword_stats["label"]==1]["keyword_count"].median():.0f}')

# Train RandomForest model
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', max_depth=10)
clf.fit(X_train, y_train)

print(f'\nRandomForest trained on {X_train.shape[0]} samples with {X_train.shape[1]} features')

# ============================================================================
# TRANSFORM UNLABELED DATA WITH SAME FEATURES
# ============================================================================

X_unlabeled_desc = tfidf_desc.transform(df_unlabeled_filtered['description_clean'])
X_unlabeled_name = tfidf_name.transform(df_unlabeled_filtered['full_name_clean'])

unlabeled_keyword_counts = df_unlabeled_filtered['description_clean'].apply(count_organizational_keywords).values
unlabeled_keyword_feature = csr_matrix(unlabeled_keyword_counts.reshape(-1, 1))

X_unlabeled = hstack([X_unlabeled_desc, X_unlabeled_name, unlabeled_keyword_feature])

# Predict
y_pred = clf.predict(X_unlabeled)
y_proba = clf.predict_proba(X_unlabeled)

# Calculate uncertainty and confidence
uncertainty = 1 - np.max(y_proba, axis=1)
confidence = 1 - uncertainty

# Create output dataframe
df_output = df_unlabeled_filtered.copy()
df_output['predicted_class'] = y_pred
df_output['probability_class_0'] = y_proba[:, 0]
df_output['probability_class_1'] = y_proba[:, 1]
df_output['uncertainty_score'] = uncertainty
df_output['confidence_level'] = confidence
df_output['org_keyword_count'] = unlabeled_keyword_counts  # Add keyword count for analysis

# Sort by uncertainty (highest first)
df_output = df_output.sort_values('uncertainty_score', ascending=False).reset_index(drop=True)

# Save
df_output.to_csv(PO_OUTPUT_FILE, index=False, encoding='utf-8')

print(f'\nPredictions saved to: {PO_OUTPUT_FILE.name}')
print(f'Total predictions: {len(df_output)}')

print('\n' + '='*80)
print('PREDICTIONS ANALYSIS')
print('='*80)

print(f'\nClass Distribution:')
print(f'  Person (class 1): {(y_pred == 1).sum()} ({100*(y_pred == 1).sum()/len(y_pred):.1f}%)')
print(f'  Organization (class 0): {(y_pred == 0).sum()} ({100*(y_pred == 0).sum()/len(y_pred):.1f}%)')

print(f'\nKeyword Feature Statistics (Predictions):')
print(f'  Organizations (0): mean={unlabeled_keyword_counts[y_pred==0].mean():.2f}, '
      f'median={np.median(unlabeled_keyword_counts[y_pred==0]):.0f}')
print(f'  Persons (1): mean={unlabeled_keyword_counts[y_pred==1].mean():.2f}, '
      f'median={np.median(unlabeled_keyword_counts[y_pred==1]):.0f}')
print(f'  Users with keywords>0: {(unlabeled_keyword_counts > 0).sum()} ({100*(unlabeled_keyword_counts > 0).sum()/len(unlabeled_keyword_counts):.1f}%)')

print(f'\nConfidence Distribution:')
print(f'  Min: {confidence.min():.4f}')
print(f'  Max: {confidence.max():.4f}')
print(f'  Mean: {confidence.mean():.4f}')
print(f'  Median: {np.median(confidence):.4f}')

print(f'\nUncertainty Distribution:')
print(f'  Min: {uncertainty.min():.4f}')
print(f'  Max: {uncertainty.max():.4f}')
print(f'  Mean: {uncertainty.mean():.4f}')
print(f'  Median: {np.median(uncertainty):.4f}')

print(f'\nConfidence Histogram:')
bins = [(0, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]
for low, high in bins:
    count = ((confidence >= low) & (confidence < high)).sum()
    pct = 100 * count / len(confidence)
    print(f'  {low:.1f}-{high:.1f}: {count} ({pct:.1f}%)')

print(f'\nHigh Confidence Predictions (>0.8):')
high_conf = df_output[df_output['confidence_level'] > 0.8]
print(f'  Total: {len(high_conf)} ({100*len(high_conf)/len(df_output):.1f}%)')
print(f'  Person: {(high_conf["predicted_class"] == 1).sum()}')
print(f'  Organization: {(high_conf["predicted_class"] == 0).sum()}')

print(f'\nTop 10 most uncertain predictions:')
print(df_output[['username', 'display_name', 'predicted_class', 'uncertainty_score', 'confidence_level']].head(10).to_string(index=False))

print('\n' + '='*80)
print('COMPARISON WITH PREVIOUS VERSION')
print('='*80)
print('Improvements with keyword feature:')
print('  - Added organizational keyword count (Persian + English)')
print('  - Total features increased from 150 to 151')
print('  - Keyword feature detects org-specific words in descriptions')
print('  - This is evidence-based feature engineering, not hard-coded rules')
print('\nExpected benefits:')
print('  - Fewer overconfident mistakes on organizations')
print('  - Better probability calibration')
print('  - More informative uncertainty scores for active learning')


**reate merged labeled file for ITERATION 3 (person_vs_organization)**


In [ ]:
# STAGE 15: Create merged labeled file for ITERATION 3 (person_vs_organization)

# This merges:

#   - labeled_data_iteration_2_person_vs_organization.csv

#   - iteration_3_manual_labels_person_vs_organization.csv

# Into:

#   - labeled_data_iteration_3_person_vs_organization.csv

#

# The merged file is deduplicated by username (keeping the latest iteration_source),

# and ensures the label column is manual_person_vs_organization.



import pandas as pd

from pathlib import Path



print("="*80)

print("STAGE 15: MERGE LABELED DATA FOR ITERATION 3 - PERSON VS ORGANIZATION")

print("="*80)



# Base classification directory (reuse if already defined)

try:

    CLASSIFICATION_DIR

except NameError:

    CLASSIFICATION_DIR = Path("POIs") / "Classification"



iter2_file = CLASSIFICATION_DIR / "labeled_data_iteration_2_person_vs_organization.csv"

iter3_file = CLASSIFICATION_DIR / "iteration_3_manual_labels_person_vs_organization.csv"

out_file = CLASSIFICATION_DIR / "labeled_data_iteration_3_person_vs_organization.csv"



print(f"Iteration 2 labeled file: {iter2_file}")

print(f"Iteration 3 manual labels: {iter3_file}")

print(f"Output (iteration 3 merged): {out_file}\n")



# Load iteration 2 labeled data

df_iter2 = pd.read_csv(iter2_file)

print(f"Loaded iteration 2 labeled data: {len(df_iter2)} rows")



# Ensure manual_person_vs_organization exists for iteration 2

if "manual_person_vs_organization" not in df_iter2.columns:

    if "person_vs_organization" in df_iter2.columns:

        df_iter2["manual_person_vs_organization"] = df_iter2["person_vs_organization"]

        print("Added manual_person_vs_organization column to iteration 2 from person_vs_organization")

    else:

        raise ValueError("Iteration 2 file must contain either 'manual_person_vs_organization' or 'person_vs_organization' column.")



df_iter2["iteration_source"] = 2



# Load iteration 3 manual labels

df_iter3 = pd.read_csv(iter3_file)

print(f"Loaded iteration 3 manual labels: {len(df_iter3)} rows")



if "manual_person_vs_organization" not in df_iter3.columns:

    raise ValueError("Iteration 3 file must contain 'manual_person_vs_organization' column.")



df_iter3["iteration_source"] = 3



# Align columns (union of both files)

all_columns = sorted(set(df_iter2.columns) | set(df_iter3.columns))

df_iter2_aligned = df_iter2.reindex(columns=all_columns)

df_iter3_aligned = df_iter3.reindex(columns=all_columns)



df_merged = pd.concat([df_iter2_aligned, df_iter3_aligned], ignore_index=True)

print(f"\nTotal rows before de-duplication: {len(df_merged)}")



# Deduplicate by username, keeping the latest iteration_source (3 over 2)

if "username" not in df_merged.columns:

    raise ValueError("Merged data must contain a 'username' column for de-duplication.")



df_merged = (

    df_merged

    .sort_values(by=["username", "iteration_source"])

    .drop_duplicates(subset=["username"], keep="last")

    .reset_index(drop=True)

)



print(f"Total rows after de-duplication by username: {len(df_merged)}")



# Report iteration_source distribution

print("\nIteration source distribution (after merge):")

print(df_merged["iteration_source"].value_counts(dropna=False))



# Report label distribution

if "manual_person_vs_organization" in df_merged.columns:

    print("\nLabel distribution (manual_person_vs_organization):")

    print(df_merged["manual_person_vs_organization"].value_counts(dropna=False))

    if df_merged["manual_person_vs_organization"].isna().any():

        print("WARNING: There are rows with NaN in manual_person_vs_organization.")

else:

    raise ValueError("Merged data must contain 'manual_person_vs_organization' column.")



# Save merged iteration 3 labeled data

df_merged.to_csv(out_file, index=False)



print("\nMerged labeled file for iteration 3 saved successfully.")

print(f"Output path: {out_file}")


**Create merged labeled file for ITERATION 3 (target_population)**

In [ ]:
# STAGE 15B: Create merged labeled file for ITERATION 3 (target_population)

# This merges:
#   - labeled_data_iteration_2_target_population.csv
#   - iteration_3_manual_labels_target_population.csv
# Into:
#   - labeled_data_iteration_3_target_population.csv
#
# The merged file is deduplicated by username (keeping the latest
# iteration_source), and ensures the label column is manual_target_population.

import pandas as pd
from pathlib import Path

print("=" * 80)
print("STAGE 15B: MERGE LABELED DATA FOR ITERATION 3 - TARGET POPULATION")
print("=" * 80)

# Base classification directory (reuse if already defined)
try:
    CLASSIFICATION_DIR
except NameError:
    CLASSIFICATION_DIR = Path("POIs") / "Classification"

iter2_tp_file = CLASSIFICATION_DIR / "labeled_data_iteration_2_target_population.csv"
iter3_tp_file = CLASSIFICATION_DIR / "iteration_3_manual_labels_target_population.csv"
out_tp_file = CLASSIFICATION_DIR / "labeled_data_iteration_3_target_population.csv"

print(f"Iteration 2 labeled file (TP): {iter2_tp_file}")
print(f"Iteration 3 manual labels (TP): {iter3_tp_file}")
print(f"Output (iteration 3 merged TP): {out_tp_file}\n")

# Load iteration 2 labeled data (target_population)
df_tp_iter2 = pd.read_csv(iter2_tp_file, encoding="utf-8")
print(f"Loaded iteration 2 TP labeled data: {len(df_tp_iter2)} rows")

# Ensure manual_target_population exists for iteration 2
if "manual_target_population" not in df_tp_iter2.columns:
    if "target_population" in df_tp_iter2.columns:
        df_tp_iter2["manual_target_population"] = df_tp_iter2["target_population"]
        print("Added manual_target_population column to iteration 2 from target_population")
    else:
        raise ValueError(
            "Iteration 2 TP file must contain either 'manual_target_population' or 'target_population' column."
        )

# Mark iteration source
df_tp_iter2["iteration_source"] = 2

# Load iteration 3 manual labels (target_population)
df_tp_iter3 = pd.read_csv(iter3_tp_file, encoding="utf-8")
print(f"Loaded iteration 3 TP manual labels: {len(df_tp_iter3)} rows")

if "manual_target_population" not in df_tp_iter3.columns:
    raise ValueError("Iteration 3 TP file must contain 'manual_target_population' column.")

# Mark iteration source
df_tp_iter3["iteration_source"] = 3

# Align columns (union of both files)
all_tp_columns = sorted(set(df_tp_iter2.columns) | set(df_tp_iter3.columns))

df_tp_iter2_aligned = df_tp_iter2.reindex(columns=all_tp_columns)
df_tp_iter3_aligned = df_tp_iter3.reindex(columns=all_tp_columns)

# Concatenate and de-duplicate by username (keep latest iteration_source)
df_tp_merged = pd.concat([df_tp_iter2_aligned, df_tp_iter3_aligned], ignore_index=True)
print(f"\nTotal TP rows before de-duplication: {len(df_tp_merged)}")

if "username" not in df_tp_merged.columns:
    raise ValueError("Merged TP data must contain a 'username' column for de-duplication.")

df_tp_merged = (
    df_tp_merged
    .sort_values(by=["username", "iteration_source"])
    .drop_duplicates(subset=["username"], keep="last")
    .reset_index(drop=True)
)

print(f"Total TP rows after de-duplication by username: {len(df_tp_merged)}")

# Report iteration_source distribution
print("\nIteration source distribution for TP (after merge):")
print(df_tp_merged["iteration_source"].value_counts(dropna=False))

# Report label distribution
if "manual_target_population" in df_tp_merged.columns:
    print("\nLabel distribution (manual_target_population):")
    print(df_tp_merged["manual_target_population"].value_counts(dropna=False))
    if df_tp_merged["manual_target_population"].isna().any():
        print("WARNING: There are rows with NaN in manual_target_population.")
else:
    raise ValueError("Merged TP data must contain 'manual_target_population' column.")

# Save merged iteration 3 TP labeled data
df_tp_merged.to_csv(out_tp_file, index=False, encoding="utf-8")

print("\nMerged TP labeled file for iteration 3 saved successfully.")
print(f"Output path: {out_tp_file}")


**Running all classifiers from step 14 on the person vs organization column**

In [ ]:
# STAGE 16 (ITERATION 3): FULL GRID FOR PERSON VS ORGANIZATION (WITH PERSON/ORG FEATURES)
# This cell runs the Stage 14-style experiment grid ONLY for the
# person_vs_organization task, using the merged labeled data from
# iteration 2 + iteration 3:
#   POIs/Classification/labeled_data_iteration_3_person_vs_organization.csv
#
# It uses:
#   - TF-IDF text features (username, display_name, description)
#   - Numeric profile features (followers, following, statuses, verified)
#   - PERSON_KEYWORDS / ORG_KEYWORDS indicator features
#
# IMPORTANT:
#   * It DOES NOT modify or delete any existing rows in
#       POIs/Classification/Experiments/experiments_results.csv
#   * It only APPENDS new rows for iteration=3.

import numpy as np
import pandas as pd
from pathlib import Path
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

print("=" * 90)
print("STAGE 16 (ITERATION 3): PERSON VS ORGANIZATION GRID WITH PERSON/ORG FEATURES")
print("=" * 90)

# Ensure PERSON_KEYWORDS / ORG_KEYWORDS are available (fallback if not executed)
try:
    PERSON_KEYWORDS
    ORG_KEYWORDS

except NameError:
    PERSON_KEYWORDS = [
        "doctor", "dr", "md", "prof", "professor", "researcher", "activist", "journalist",
        "writer", "author", "artist", "singer", "player", "athlete", "lawyer", "engineer",
    ]

    ORG_KEYWORDS = [
        "news", "agency", "organization", "foundation", "media", "company", "corp", "ltd",
        "network", "center", "centre", "ministry", "office", "club", "association", "ngo",
        "university", "hospital", "clinic",
    ]

    print("[INFO] PERSON_KEYWORDS / ORG_KEYWORDS were not defined in this kernel; using fallback lists.")

# Helper functions (self-contained copy from Stage 14 NEW)

def _safe_get_score_binary_new(estimator, X):
    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X)
        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
    if hasattr(estimator, "decision_function"):
        return estimator.decision_function(X)
    return None

def _manual_cv_predictions_new(estimator, X, y, cv, is_binary: bool):
    y = np.asarray(y)
    y_pred = np.empty(shape=(len(y),), dtype=int)
    y_score = None
    if is_binary:
        y_score = np.full(shape=(len(y),), fill_value=np.nan, dtype=float)
    for train_idx, test_idx in cv.split(X, y):
        model = clone(estimator)
        model.fit(X[train_idx], y[train_idx])
        y_pred[test_idx] = model.predict(X[test_idx])
        if is_binary and y_score is not None:
            scores = _safe_get_score_binary_new(model, X[test_idx])
            if scores is not None:
                y_score[test_idx] = np.asarray(scores, dtype=float).ravel()
    if is_binary and y_score is not None and np.isnan(y_score).all():
        y_score = None
    return y_pred, y_score

def _evaluate_predictions_new(y_true, y_pred, y_score=None, is_binary=False):
    avg = "binary" if is_binary else "weighted"
    metrics = {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average=avg, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average=avg, zero_division=0)),
        "F1": float(f1_score(y_true, y_pred, average=avg, zero_division=0)),
        "AUC": np.nan,
    }

    if is_binary and y_score is not None:
        try:
            metrics["AUC"] = float(roc_auc_score(y_true, y_score))
        except Exception:
            metrics["AUC"] = np.nan
    return metrics

def _balance_indices_new(y_codes, random_state=42):
    counts = pd.Series(y_codes).value_counts(dropna=False)
    counts_nonzero = counts[counts > 0]
    min_class_size = int(counts_nonzero.min()) if len(counts_nonzero) else 0
    balanced = []
    for cls in sorted(counts_nonzero.index.tolist()):
        cls_idx = np.where(y_codes == cls)[0]
        sampled = resample(cls_idx, n_samples=min_class_size, random_state=random_state, replace=False)
        balanced.extend(sampled.tolist())
    return sorted(balanced), min_class_size

def make_algorithm_factories_new(random_state=42):
    def lr_factory(is_binary, n_classes):
        return LogisticRegression(max_iter=2000, random_state=random_state)
    def dt_factory(is_binary, n_classes):
        return DecisionTreeClassifier(random_state=random_state)
    def rf_factory(is_binary, n_classes):
        return RandomForestClassifier(n_estimators=300, random_state=random_state, n_jobs=-1)
    def svm_factory(is_binary, n_classes):
        return SVC(kernel="linear", probability=False)
    def xgb_factory(is_binary, n_classes):
        common = dict(
            n_estimators=400,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=random_state,
            n_jobs=-1,
            tree_method="hist",
        )
        if is_binary:
            return XGBClassifier(**common, objective="binary:logistic", eval_metric="logloss")
        return XGBClassifier(**common, objective="multi:softprob", num_class=int(n_classes), eval_metric="mlogloss")
    def ada_factory(is_binary, n_classes):
        return AdaBoostClassifier(n_estimators=300, random_state=random_state)

    return {
        "LogReg": lr_factory,
        "DecisionTree": dt_factory,
        "RandomForest": rf_factory,
        "SVM": svm_factory,
        "XGBoost": xgb_factory,
        "AdaBoost": ada_factory,
    }

def run_experiment_new(
    *, iteration, target_column, y_codes, X, mapping, positive_class,
    training_type, K, algorithm, feature_set, balanced, multiclass, random_state=42,
):

    code_to_label = mapping
    label_to_code = {v: k for k, v in code_to_label.items()}
    unknown_code = label_to_code.get("unknown", 2)
    y_codes_work = np.asarray(y_codes)
    X_work = X

    if multiclass:
        is_binary = False

    else:
        is_binary = True
        keep = y_codes_work != unknown_code
        y_codes_work = y_codes_work[keep]
        X_work = X_work[keep]
    counts_before = pd.Series(y_codes_work).value_counts().to_dict()

    class_0 = int(counts_before.get(0, 0))
    class_1 = int(counts_before.get(1, 0))
    class_2 = int(counts_before.get(2, 0)) if multiclass else 0
    present_counts = [c for c in [class_0, class_1, class_2] if c > 0]
    min_class_size = int(min(present_counts)) if present_counts else 0
    if balanced:
        idx_bal, _ = _balance_indices_new(y_codes_work, random_state=random_state)
        y_codes_work = y_codes_work[idx_bal]
        X_work = X_work[idx_bal]
    counts_after = pd.Series(y_codes_work).value_counts().to_dict()
    if is_binary:
        pos_code = label_to_code[positive_class]
        y_model = (y_codes_work == pos_code).astype(int)
        n_classes = 2
    else:
        unique_codes = sorted(pd.Series(y_codes_work).dropna().unique().tolist())
        code_to_num = {c: i for i, c in enumerate(unique_codes)}
        y_model = np.array([code_to_num[c] for c in y_codes_work], dtype=int)
        n_classes = len(unique_codes)
    if training_type == "KFold":
        cv = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_state)
    elif training_type == "LOOCV":
        cv = LeaveOneOut()
    else:
        raise ValueError(f"Unknown training_type: {training_type}")
    estimator = algorithm(is_binary, n_classes)
    y_pred, y_score = _manual_cv_predictions_new(estimator, X_work, y_model, cv=cv, is_binary=is_binary)
    metrics = _evaluate_predictions_new(y_model, y_pred, y_score=y_score, is_binary=is_binary)

    row = {
        "iteration": int(iteration),
        "target_column": str(target_column),
        "#classes": int(3 if multiclass else 2),
        "#class_0": int(class_0 if not balanced else counts_after.get(0, 0)),
        "#class_1": int(class_1 if not balanced else counts_after.get(1, 0)),
        "#class_2": int(class_2 if not balanced else counts_after.get(2, 0) if multiclass else 0),
        "min_class_size": int(min_class_size),
        "training_type": str(training_type),
        "K": int(K if training_type == "KFold" else len(y_model)),
        "algorithm": str(algorithm.__name__ if hasattr(algorithm, "__name__") else "factory"),
        "feature_set": str(feature_set),
        "features_count": int(X_work.shape[1]),
        "balanced": bool(balanced),
        "accuracy": metrics["accuracy"],
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "F1": metrics["F1"],
        "AUC": metrics["AUC"],
    }
    return row
# Paths and basic config

CLASSIFICATION_DIR_3 = Path("POIs") / "Classification"
EXPERIMENTS_DIR_3 = CLASSIFICATION_DIR_3 / "Experiments"
EXPERIMENTS_DIR_3.mkdir(parents=True, exist_ok=True)
LABELED_PO_ITER3 = CLASSIFICATION_DIR_3 / "labeled_data_iteration_3_person_vs_organization.csv"
RESULTS_FILE_3 = EXPERIMENTS_DIR_3 / "experiments_results.csv"
ITERATION_3 = 3
TARGET_NAME_3 = "person_vs_organization"
print(f"Labeled data (iteration 3 merged): {LABELED_PO_ITER3}")
print(f"Results file (append only): {RESULTS_FILE_3}\n")

# Load merged iteration-3 labeled data

df_po_iter3 = pd.read_csv(LABELED_PO_ITER3, encoding="utf-8")
print(f"Loaded merged iteration-3 person_vs_organization data: {len(df_po_iter3)} rows")
if "manual_person_vs_organization" not in df_po_iter3.columns:
    raise ValueError("Expected 'manual_person_vs_organization' column in iteration 3 labeled file.")
y_full = df_po_iter3["manual_person_vs_organization"].astype(int).to_numpy()
mapping_po = {0: "organization", 1: "person", 2: "unknown"}
positive_class_po = "person"

# Build text features (TF-IDF)

desc_series = df_po_iter3.get("description_en", df_po_iter3.get("description", "")).fillna("").astype(str)
name_series = df_po_iter3.get("display_name_en", df_po_iter3.get("display_name", "")).fillna("").astype(str)
user_series = df_po_iter3["username"].fillna("").astype(str)
tfidf_params_desc = dict(max_features=500, min_df=2, max_df=0.8, ngram_range=(1, 2), strip_accents="unicode", lowercase=True)
vec_desc_3 = TfidfVectorizer(stop_words="english", **tfidf_params_desc)
X_desc_3 = vec_desc_3.fit_transform(desc_series)
vec_name_3 = TfidfVectorizer(max_features=200, min_df=1, ngram_range=(1, 2), strip_accents="unicode", lowercase=True, stop_words="english")
X_name_3 = vec_name_3.fit_transform(name_series)
vec_user_3 = TfidfVectorizer(max_features=200, min_df=1, ngram_range=(1, 2), strip_accents="unicode", lowercase=True)
X_user_3 = vec_user_3.fit_transform(user_series)

print("Text feature shapes:")
print(f"  description: {X_desc_3.shape}")
print(f"  full_name:   {X_name_3.shape}")
print(f"  username:    {X_user_3.shape}")

# Numeric + PERSON/ORG keyword indicator features

numeric_cols_3 = ["followers_count", "following_count", "statuses_count", "verified"]
for col in numeric_cols_3:

    if col not in df_po_iter3.columns:
        df_po_iter3[col] = 0

X_numeric_base_3 = df_po_iter3[numeric_cols_3].fillna(0).to_numpy(dtype=float)

if "verified" in df_po_iter3.columns:
    X_numeric_base_3[:, numeric_cols_3.index("verified")] = df_po_iter3["verified"].fillna(0).astype(int).to_numpy()

combined_text_3 = (
    df_po_iter3["username"].fillna("").astype(str)
    + " "
    + df_po_iter3.get("display_name_en", df_po_iter3.get("display_name", "")).fillna("").astype(str)
    + " "
    + df_po_iter3.get("description_en", df_po_iter3.get("description", "")).fillna("").astype(str)

)
combined_lower_3 = combined_text_3.str.lower().fillna("")

def _keyword_flag(series, keywords):
    flags = []
    for text in series:
        t = str(text)
        flags.append(1 if any(kw.lower() in t for kw in keywords) else 0)
    return np.array(flags, dtype=int)



person_kw_flag_3 = _keyword_flag(combined_lower_3, PERSON_KEYWORDS)

org_kw_flag_3 = _keyword_flag(combined_lower_3, ORG_KEYWORDS)
X_kw_3 = np.vstack([person_kw_flag_3, org_kw_flag_3]).T
X_numeric_all_3 = np.hstack([X_numeric_base_3, X_kw_3])
scaler_3 = StandardScaler()
X_numeric_scaled_3 = scaler_3.fit_transform(X_numeric_all_3)
X_num_sparse_3 = csr_matrix(X_numeric_scaled_3)
print("Numeric + keyword feature shape:")
print(f"  base numeric: {X_numeric_base_3.shape}")
print(f"  keyword flags: {X_kw_3.shape}")
print(f"  all numeric+kw (scaled): {X_num_sparse_3.shape}\n")



# Define feature sets (text-only and text+numeric+keyword) – no numeric-only

feature_sets_3 = {
    "TFIDF_description": X_desc_3,
    "TFIDF_username": X_user_3,
    "TFIDF_full_name": X_name_3,
    "TFIDF_description+username": hstack([X_desc_3, X_user_3]),
    "TFIDF_description+full_name": hstack([X_desc_3, X_name_3]),
    "TFIDF_username+full_name": hstack([X_user_3, X_name_3]),
    "TFIDF_description+username+full_name": hstack([X_desc_3, X_user_3, X_name_3]),
    "TFIDF_description + NUM": hstack([X_desc_3, X_num_sparse_3]),
    "TFIDF_username + NUM": hstack([X_user_3, X_num_sparse_3]),
    "TFIDF_full_name + NUM": hstack([X_name_3, X_num_sparse_3]),
    "TFIDF_description+username + NUM": hstack([X_desc_3, X_user_3, X_num_sparse_3]),
    "TFIDF_description+full_name + NUM": hstack([X_desc_3, X_name_3, X_num_sparse_3]),
    "TFIDF_username+full_name + NUM": hstack([X_user_3, X_name_3, X_num_sparse_3]),
    "TFIDF_description+username+full_name + NUM": hstack([X_desc_3, X_user_3, X_name_3, X_num_sparse_3]),

}

print("Defined feature sets (iteration 3, person_vs_organization):")
for name, Xmat in feature_sets_3.items():
    print(f"  {name}: shape={Xmat.shape}")

# Run grid and append to experiments_results.csv
TRAINING_TYPES_3 = [("KFold", 5), ("LOOCV", 5)]
ALGO_FACTORIES_NEW = make_algorithm_factories_new(random_state=42)
results_3 = []
total_configs_est_3 = len(feature_sets_3) * len(ALGO_FACTORIES_NEW) * len(TRAINING_TYPES_3) * 2 * 2
print(f"\nApproximate number of experiment rows to create: {total_configs_est_3}")
for feature_name, X_mat_3 in feature_sets_3.items():
    for algo_name, algo_factory in ALGO_FACTORIES_NEW.items():
        for training_type, K_val in TRAINING_TYPES_3:
            for multiclass_flag in [True, False]:
                for balanced_flag in [False, True]:
                    row = run_experiment_new(
                        iteration=ITERATION_3,
                        target_column=TARGET_NAME_3,
                        y_codes=y_full,
                        X=X_mat_3,
                        mapping=mapping_po,
                        positive_class=positive_class_po,
                        training_type=training_type,
                        K=K_val,
                        algorithm=algo_factory,
                        feature_set=feature_name,
                        balanced=balanced_flag,
                        multiclass=multiclass_flag,
                        random_state=42,
                    )
                    row["algorithm"] = algo_name
                    results_3.append(row)

print(f"\nFinished running grid. New rows generated: {len(results_3)}")
df_results_3 = pd.DataFrame(results_3)
if RESULTS_FILE_3.exists():
    df_results_3.to_csv(RESULTS_FILE_3, mode="a", header=False, index=False)
    print(f"✅ Appended {len(df_results_3)} rows to existing {RESULTS_FILE_3.name}")
else:
    df_results_3.to_csv(RESULTS_FILE_3, mode="w", header=True, index=False)
    print(f"✅ Created {RESULTS_FILE_3.name} with {len(df_results_3)} rows")

print("\nPreview of iteration-3 person_vs_organization experiments (first 20 rows):")
print(df_results_3.head(20).to_string(index=False))
print("\n" + "=" * 90)
print("✅ STAGE 16 COMPLETED: ITERATION 3 GRID FOR PERSON VS ORGANIZATION")
print("=" * 90)


**STAGE 15 – PREDICTION ON TRANSLATED UNLABELED PERSON VS ORGANIZATION (AFTER ITERATION 3)**

In [ ]:
# STAGE 15 – PREDICTION ON TRANSLATED UNLABELED (AFTER ITERATION 3)
# Same model/logic as Stage 15, but using:
#   POIs/Classification/unlabeled_users_after_iteration_3_person_vs_organization.csv
# which now has description_en and display_name_en.

import numpy as np
import pandas as pd
from pathlib import Path
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

print("=" * 90)
print("STAGE 15B – PREDICTION (ENGLISH TRANSLATED UNLABELED AFTER ITERATION 3)")
print("RandomForest + TFIDF(description_en/full_name_en) + numeric features")
print("=" * 90)

CLASSIFICATION_DIR_PO_2 = Path("POIs") / "Classification"
LABELED_FILE_PO_3_2 = CLASSIFICATION_DIR_PO_2 / "labeled_data_iteration_3_person_vs_organization.csv"
UNLABELED_FILE_TRANSLATED = CLASSIFICATION_DIR_PO_2 / "unlabeled_users_after_iteration_3_person_vs_organization.csv"
OUTPUT_PREDICTIONS_EN = CLASSIFICATION_DIR_PO_2 / "iteration_3_unlabeled_users_predictions_person_vs_organization_english.csv"
OUTPUT_TOP100_EN = CLASSIFICATION_DIR_PO_2 / "iteration_4_manual_labeling_candidates_person_vs_organization_english.csv"

print(f"Labeled file:          {LABELED_FILE_PO_3_2}")
print(f"Unlabeled (translted):{UNLABELED_FILE_TRANSLATED}")
print(f"Output predictions:    {OUTPUT_PREDICTIONS_EN}")
print(f"Output top-100:        {OUTPUT_TOP100_EN}\n")

# Load labeled data and keep labels {0,1}

df_labeled_po_2 = pd.read_csv(LABELED_FILE_PO_3_2, encoding="utf-8")
label_col_po_2 = "manual_person_vs_organization" if "manual_person_vs_organization" in df_labeled_po_2.columns else "person_vs_organization"

if label_col_po_2 not in df_labeled_po_2.columns:
    raise ValueError("No person_vs_organization label column found in labeled data.")

df_train_po_2 = df_labeled_po_2[df_labeled_po_2[label_col_po_2].isin([0, 1])].copy()
df_train_po_2[label_col_po_2] = df_train_po_2[label_col_po_2].astype(int)
y_train_po_2 = df_train_po_2[label_col_po_2].to_numpy()

print(f"Training rows used (labels 0/1 only): {len(df_train_po_2)}\n")

if len(df_train_po_2) == 0:
    raise ValueError("No training data with labels 0/1 available for person_vs_organization.")

# Load translated unlabeled pool and (optionally) exclude any labeled usernames
df_unlabeled_tr_2 = pd.read_csv(UNLABELED_FILE_TRANSLATED, encoding="utf-8")

if "username" not in df_train_po_2.columns or "username" not in df_unlabeled_tr_2.columns:
    raise ValueError("Both labeled and unlabeled data must contain a 'username' column.")

labeled_usernames_all_2 = set(df_train_po_2["username"].astype(str))
df_unlabeled_po_2 = df_unlabeled_tr_2[~df_unlabeled_tr_2["username"].astype(str).isin(labeled_usernames_all_2)].copy()

print(f"Unlabeled rows in file:          {len(df_unlabeled_tr_2)}")
print(f"After excluding labeled (if any): {len(df_unlabeled_po_2)}\n")

if len(df_unlabeled_po_2) == 0:
    raise ValueError("No remaining unlabeled users after excluding labeled usernames.")

def _get_text_series(df, primary, fallback):
    if primary in df.columns:

        return df[primary].fillna("").astype(str)
    if fallback in df.columns:

        return df[fallback].fillna("").astype(str)
    return pd.Series([""] * len(df), index=df.index)

desc_train_2 = _get_text_series(df_train_po_2, "description_en", "description")
name_train_2 = _get_text_series(df_train_po_2, "display_name_en", "display_name")
desc_unlabeled_2 = _get_text_series(df_unlabeled_po_2, "description_en", "description")
name_unlabeled_2 = _get_text_series(df_unlabeled_po_2, "display_name_en", "display_name")

tfidf_params_2 = dict(max_features=500, ngram_range=(1, 2), lowercase=True, strip_accents="unicode", stop_words="english")

tfidf_desc_po_2 = TfidfVectorizer(**tfidf_params_2)
X_train_desc_po_2 = tfidf_desc_po_2.fit_transform(desc_train_2)
X_unlabeled_desc_po_2 = tfidf_desc_po_2.transform(desc_unlabeled_2)

tfidf_name_po_2 = TfidfVectorizer(**tfidf_params_2)
X_train_name_po_2 = tfidf_name_po_2.fit_transform(name_train_2)
X_unlabeled_name_po_2 = tfidf_name_po_2.transform(name_unlabeled_2)

print("Text feature shapes (training):")
print(f"  description_en: {X_train_desc_po_2.shape}")
print(f"  full_name_en:   {X_train_name_po_2.shape}")
print("Text feature shapes (unlabeled):")
print(f"  description_en: {X_unlabeled_desc_po_2.shape}")
print(f"  full_name_en:   {X_unlabeled_name_po_2.shape}\n")

# Numeric features + ratio
num_cols_2 = ["followers_count", "following_count", "statuses_count"]

for col in num_cols_2:

    if col not in df_train_po_2.columns:
        df_train_po_2[col] = 0

    if col not in df_unlabeled_po_2.columns:
        df_unlabeled_po_2[col] = 0

def _compute_ratio(df):
    followers = df["followers_count"].fillna(0).astype(float)
    following = df["following_count"].fillna(0).astype(float)
    following_safe = following.where(following != 0, 1.0)
    return (followers / following_safe).astype(float)

df_train_po_2["ratio"] = _compute_ratio(df_train_po_2)
df_unlabeled_po_2["ratio"] = _compute_ratio(df_unlabeled_po_2)

num_cols_full_2 = ["followers_count", "following_count", "statuses_count", "ratio"]
X_num_train_2 = df_train_po_2[num_cols_full_2].fillna(0).to_numpy(dtype=float)
X_num_unlabeled_2 = df_unlabeled_po_2[num_cols_full_2].fillna(0).to_numpy(dtype=float)

scaler_po_2 = StandardScaler()

X_num_train_scaled_2 = scaler_po_2.fit_transform(X_num_train_2)
X_num_unlabeled_scaled_2 = scaler_po_2.transform(X_num_unlabeled_2)

X_num_train_sparse_2 = csr_matrix(X_num_train_scaled_2)
X_num_unlabeled_sparse_2 = csr_matrix(X_num_unlabeled_scaled_2)

print("Numeric feature shapes (training / unlabeled):")
print(f"  numeric: {X_num_train_sparse_2.shape} / {X_num_unlabeled_sparse_2.shape}\n")

X_train_po_2 = hstack([X_train_desc_po_2, X_train_name_po_2, X_num_train_sparse_2])
X_unlabeled_po_2 = hstack([X_unlabeled_desc_po_2, X_unlabeled_name_po_2, X_num_unlabeled_sparse_2])

print("Final feature shapes:")
print(f"  X_train_po_2:    {X_train_po_2.shape}")
print(f"  X_unlabeled_po_2:{X_unlabeled_po_2.shape}\n")

rf_po_2 = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
rf_po_2.fit(X_train_po_2, y_train_po_2)

print("Model trained (Stage 15B): RandomForestClassifier")
print(f"# training samples: {len(df_train_po_2)}\n")

proba_po_2 = rf_po_2.predict_proba(X_unlabeled_po_2)
class_to_index_po_2 = {int(cls): idx for idx, cls in enumerate(rf_po_2.classes_)}

if not set(class_to_index_po_2.keys()).issuperset({0, 1}):
    raise ValueError(f"RandomForest classes are {rf_po_2.classes_}, expected [0, 1].")

prob_0_2 = proba_po_2[:, class_to_index_po_2[0]]
prob_1_2 = proba_po_2[:, class_to_index_po_2[1]]
predicted_class_2 = (prob_1_2 >= prob_0_2).astype(int)
confidence_level_2 = np.maximum(prob_0_2, prob_1_2)
uncertainty_score_2 = 1.0 - confidence_level_2

df_pred_po_2 = df_unlabeled_po_2.copy()
df_pred_po_2["prob_0_organization"] = prob_0_2
df_pred_po_2["prob_1_person"] = prob_1_2
df_pred_po_2["predicted_class_person_vs_organization"] = predicted_class_2
df_pred_po_2["confidence_level_person_vs_organization"] = confidence_level_2
df_pred_po_2["uncertainty_score_person_vs_organization"] = uncertainty_score_2

df_pred_po_2.to_csv(OUTPUT_PREDICTIONS_EN, index=False, encoding="utf-8")

print(f"# unlabeled samples predicted: {len(df_pred_po_2)}")
print(f"Predictions saved to: {OUTPUT_PREDICTIONS_EN}\n")

class_counts_po_2 = pd.Series(predicted_class_2).value_counts().sort_index()
print("Predicted class distribution (0=organization, 1=person):")
for cls, cnt in class_counts_po_2.items():
    print(f"  class {cls}: {cnt}")
print()

df_top100_2 = df_pred_po_2.sort_values("uncertainty_score_person_vs_organization", ascending=False).head(100).copy()
df_top100_2.to_csv(OUTPUT_TOP100_EN, index=False, encoding="utf-8")

if len(df_top100_2) > 0:
    unc_min_2 = float(df_top100_2["uncertainty_score_person_vs_organization"].min())
    unc_max_2 = float(df_top100_2["uncertainty_score_person_vs_organization"].max())
    print(f"Top 100 uncertain users saved to: {OUTPUT_TOP100_EN}")
    print(f"Uncertainty range among top 100: min={unc_min_2:.4f}, max={unc_max_2:.4f}")
else:
    print("Fewer than 1 user available for uncertainty ranking; top-100 file may be small or empty.")

print("\n" + "=" * 90)
print("✅ STAGE 15B – PREDICTION COMPLETED (TRANSLATED UNLABELED, ITERATION 3)")
print("=" * 90)


### STAGE 16C: Run Iteration 3 Experiments for Target Population (merging all 3 iterations, filtering NaN labels, running full experiment grid with 672 configurations)

In [ ]:
print("="*80)
print("STAGE 16C: TARGET_POPULATION Iteration 3 - Complete Experiment Grid")
print("="*80)

# -------------------------------------------------------------------
# 1) LOAD MERGED ITERATION 3 DATA
# -------------------------------------------------------------------
print("\n[1/4] Loading merged iteration 3 labeled data...")

# Load the merged iteration 3 file (already contains all iterations merged)
LABELED_TP_ITER3 = CLASSIFICATION_DIR_3 / "labeled_data_iteration_3_target_population.csv"
df_tp_merged = pd.read_csv(LABELED_TP_ITER3, encoding="utf-8")

print(f"   Loaded {len(df_tp_merged)} rows from: {LABELED_TP_ITER3}")

# Rename column if needed
if "manual_target_population" not in df_tp_merged.columns:
    if "target_population" in df_tp_merged.columns:
        df_tp_merged = df_tp_merged.rename(columns={"target_population": "manual_target_population"})

# -------------------------------------------------------------------
# 2) FILTER NaN LABELS
# -------------------------------------------------------------------
print("\n[2/4] Filtering NaN labels...")
before_filter = len(df_tp_merged)
df_tp_merged = df_tp_merged[df_tp_merged["manual_target_population"].notna()].copy()
after_filter = len(df_tp_merged)
print(f"   Removed {before_filter - after_filter} rows with NaN labels")
print(f"   Remaining rows: {after_filter}")

# Convert to int safely
df_tp_merged["manual_target_population"] = df_tp_merged["manual_target_population"].astype(int)

# -------------------------------------------------------------------
# 3) PREPARE FEATURES
# -------------------------------------------------------------------
print("\n[3/4] Preparing features for target_population...")

# Map labels
mapping_tp = {0: 'non_target', 1: 'target', 2: 'unknown'}
positive_class_tp = 'target'

# Get label vector
y_tp_full = df_tp_merged["manual_target_population"].astype(int).to_numpy()
print(f"   Label distribution: {pd.Series(y_tp_full).value_counts().to_dict()}")

# Text features (data already in df_tp_merged from CSV files)
desc_series_tp = df_tp_merged["description"].fillna("").astype(str)
name_series_tp = df_tp_merged["display_name"].fillna("").astype(str)
user_series_tp = df_tp_merged["username"].fillna("").astype(str)
combined_text_tp = (name_series_tp + " " + desc_series_tp + " " + user_series_tp).str.lower()

# TF-IDF
vec_desc_tp = TfidfVectorizer(max_features=500, ngram_range=(1,2), min_df=1)
vec_name_tp = TfidfVectorizer(max_features=200, ngram_range=(1,2), min_df=1)
vec_user_tp = TfidfVectorizer(max_features=200, ngram_range=(1,2), min_df=1)

X_desc_tp = vec_desc_tp.fit_transform(desc_series_tp)
X_name_tp = vec_name_tp.fit_transform(name_series_tp)
X_user_tp = vec_user_tp.fit_transform(user_series_tp)
X_mat_tp = hstack([X_desc_tp, X_name_tp, X_user_tp])

print(f"   Text features shape: {X_mat_tp.shape}")

# Numeric features
followers = df_tp_merged["followers_count"].fillna(0).to_numpy()
following = df_tp_merged["following_count"].fillna(0).to_numpy()
statuses = df_tp_merged["statuses_count"].fillna(0).to_numpy()
verified = df_tp_merged["verified"].fillna(0).astype(int).to_numpy()

# Followers/following ratio
ratio = np.divide(followers, following, out=np.zeros_like(followers, dtype=float), where=following > 0)

# Bio length
bio_lengths = desc_series_tp.str.len().fillna(0).to_numpy().reshape(-1, 1)

# Combine numeric
X_numeric_all_tp = np.column_stack([followers, following, statuses, verified, ratio, bio_lengths])
scaler_tp = StandardScaler()
X_numeric_scaled_tp = scaler_tp.fit_transform(X_numeric_all_tp)
X_num_sparse_tp = csr_matrix(X_numeric_scaled_tp)

print(f"   Numeric features shape: {X_numeric_scaled_tp.shape}")

# -------------------------------------------------------------------
# 4) DEFINE FEATURE SETS (14 sets: 7 text-only + 7 text+numeric)
# -------------------------------------------------------------------
print("\n[4/4] Defining 14 feature sets...")

feature_sets_tp = {
    # Text-only
    "desc_only": X_desc_tp,
    "name_only": X_name_tp,
    "user_only": X_user_tp,
    "desc+name": hstack([X_desc_tp, X_name_tp]),
    "desc+user": hstack([X_desc_tp, X_user_tp]),
    "name+user": hstack([X_name_tp, X_user_tp]),
    "all_text": X_mat_tp,
    
    # Text + numeric
    "desc+num": hstack([X_desc_tp, X_num_sparse_tp]),
    "name+num": hstack([X_name_tp, X_num_sparse_tp]),
    "user+num": hstack([X_user_tp, X_num_sparse_tp]),
    "desc+name+num": hstack([X_desc_tp, X_name_tp, X_num_sparse_tp]),
    "desc+user+num": hstack([X_desc_tp, X_user_tp, X_num_sparse_tp]),
    "name+user+num": hstack([X_name_tp, X_user_tp, X_num_sparse_tp]),
    "all_text+num": hstack([X_mat_tp, X_num_sparse_tp]),
}

for fs_name, fs_mat in feature_sets_tp.items():
    print(f"   {fs_name}: {fs_mat.shape}")

# -------------------------------------------------------------------
# 5) RUN FULL EXPERIMENT GRID (672 experiments)
# -------------------------------------------------------------------
print("\n[5/5] Running 672 experiments (6 algorithms × 14 feature sets × 2 training types × 2 class modes × 2 balance modes)...")
print("   This will take some time. Progress updates every 50 experiments.\n")

results_tp = []
experiment_counter = 0
total_experiments = len(feature_sets_tp) * len(ALGO_FACTORIES_NEW) * len(TRAINING_TYPES_3) * 2 * 2

for feature_name, Xmat_tp in feature_sets_tp.items():
    for algo_name in ALGO_FACTORIES_NEW.keys():
        for training_type, K_val in TRAINING_TYPES_3:
            for multiclass_flag in [True, False]:
                for balanced_flag in [True, False]:
                    experiment_counter += 1
                    
                    # Progress update
                    if experiment_counter % 50 == 0:
                        print(f"   Progress: {experiment_counter}/{total_experiments} experiments completed...")
                    
                    # Run experiment
                    row = run_experiment_new(
                        iteration=ITERATION_3,
                        target_column=TARGET_NAME_3_TP,
                        y_codes=y_tp_full,
                        X=Xmat_tp,
                        mapping=mapping_tp,
                        positive_class=positive_class_tp,
                        training_type=training_type,
                        K=K_val,
                        algorithm=ALGO_FACTORIES_NEW[algo_name],
                        feature_set=feature_name,
                        balanced=balanced_flag,
                        multiclass=multiclass_flag
                    )
                    
                    results_tp.append(row)

# -------------------------------------------------------------------
# 6) SAVE RESULTS
# -------------------------------------------------------------------
print(f"\n{'='*80}")
print(f"COMPLETED! Total experiments run: {len(results_tp)}")
print(f"{'='*80}")

# Convert to DataFrame
df_results_tp = pd.DataFrame(results_tp)

# Reorder columns to match experiments_results.csv
col_order = ["iteration", "target_column", "#classes", "#class_0", "#class_1", "#class_2", "min_class_size",
             "training_type", "K", "algorithm", "feature_set", "features_count", "balanced",
             "accuracy", "precision", "recall", "F1", "AUC"]
df_results_tp = df_results_tp[col_order]

# Append to main results file
if RESULTS_FILE_3.exists():
    df_results_tp.to_csv(RESULTS_FILE_3, mode='a', header=False, index=False)
    print(f"\n✓ Appended {len(df_results_tp)} rows to: {RESULTS_FILE_3}")
else:
    df_results_tp.to_csv(RESULTS_FILE_3, index=False)
    print(f"\n✓ Created new file with {len(df_results_tp)} rows: {RESULTS_FILE_3}")

# Display summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"\nLabel distribution:")
for label_val, label_name in mapping_tp.items():
    count = (y_tp_full == label_val).sum()
    print(f"   {label_name} ({label_val}): {count} samples")

print(f"\nFeature matrix shapes:")
print(f"   Text-only: {X_mat_tp.shape}")
print(f"   Numeric: {X_numeric_scaled_tp.shape}")
print(f"   Combined: {hstack([X_mat_tp, X_num_sparse_tp]).shape}")

print(f"\nResults breakdown:")
print(f"   Total experiments: {len(df_results_tp)}")
print(f"   Feature sets: {len(feature_sets_tp)}")
print(f"   Algorithms: {len(ALGO_FACTORIES_NEW)}")
print(f"   Training types: {len(TRAINING_TYPES_3)}")
print(f"   Iterations merged: 3 (iteration 1 + 2 + 3)")
print(f"   Total labeled samples: {len(y_tp_full)}")

print("\n" + "="*80)
print("DONE!")
print("="*80)

**translate all the unlabled users descriptions an usernames**

In [ ]:
# translate all the unlabeled users descriptions and usernames after iteration 3 to English
# Uses deep-translator's GoogleTranslator

import time
from deep_translator import GoogleTranslator

print("=" * 80)
print("STAGE 16D: TRANSLATE UNLABELED USERS (AFTER ITERATION 3) TO ENGLISH")
print("=" * 80)

CLASSIFICATION_DIR_TP = Path("POIs") / "Classification"
UNLABELED_TP_FILE = CLASSIFICATION_DIR_TP / "unlabeled_users.csv"
OUTPUT_TRANSLATED_TP_FILE = CLASSIFICATION_DIR_TP / "unlabeled_users_after_iteration_3_target_population.csv"

print(f"Loading unlabeled users from: {UNLABELED_TP_FILE}")
df_unlabeled_tp = pd.read_csv(UNLABELED_TP_FILE, encoding="utf-8")
print(f"Loaded {len(df_unlabeled_tp)} unlabeled users.")

def translate_text(text, max_retries: int = 3):
    """Translate text to English using deep-translator (GoogleTranslator)."""
    if pd.isna(text) or text == "":
        return ""
    text = str(text).strip()
    if text == "":
        return ""
    for attempt in range(max_retries):
        try:
            translator = GoogleTranslator(source="auto", target="en")
            text_to_translate = text[:5000] if len(text) > 5000 else text
            translated = translator.translate(text_to_translate)
            if translated and str(translated).strip() != "":
                return str(translated).strip()
            return text
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(0.5 * (attempt + 1))
                continue
            print(f"   ⚠️ Translation failed after {max_retries} attempts: {str(e)[:50]}...")
            return text
    return text

print("Translating descriptions and display names to English...")
start_time = time.time()

df_unlabeled_tp["description_en"] = df_unlabeled_tp["description"].apply(translate_text)
df_unlabeled_tp["display_name_en"] = df_unlabeled_tp["display_name"].apply(translate_text)

end_time = time.time()
print(f"Translation completed in {end_time - start_time:.2f} seconds.")

df_unlabeled_tp.to_csv(OUTPUT_TRANSLATED_TP_FILE, index=False, encoding="utf-8")
print(f"Translated unlabeled users saved to: {OUTPUT_TRANSLATED_TP_FILE}")
# End of Stage translate


STAGE 16D: TRANSLATE UNLABELED USERS (AFTER ITERATION 3) TO ENGLISH
Loading unlabeled users from: POIs\Classification\unlabeled_users.csv
Loaded 846 unlabeled users.
Translating descriptions and display names to English...
Translation completed in 941.76 seconds.
Translated unlabeled users saved to: POIs\Classification\unlabeled_users_after_iteration_3_target_population.csv


**chhose the best classifier to predict target_population iteration 3**

In [ ]:
#show only the iteration-3 target_population experiment results : show the highest accuracy ones
#and the AUC ROC ones and print the first 20 rows

TARGET_NAME_3_TP = "target_population"  
import pandas as pd
print("\nLoading iteration-3 target_population experiment results...")

df_results_tp = pd.read_csv(RESULTS_FILE_3, encoding="utf-8", engine='python', on_bad_lines='skip')

# use the Python engine and skip malformed lines to avoid ParserError
df_tp_iter3 = df_results_tp[df_results_tp["iteration"] == 3]
# Filter for target_population experiments
df_tp_iter3 = df_tp_iter3[df_tp_iter3["target_column"] == TARGET_NAME_3_TP]
print(f"Loaded {len(df_tp_iter3)} iteration-3 target_population experiment results.\n")
df_tp_iter3_sorted_acc = df_tp_iter3.sort_values(by="accuracy", ascending=False).head(20)
df_tp_iter3_sorted_auc = df_tp_iter3.sort_values(by="AUC", ascending=False).head(20)
print("Top 20 experiments by Accuracy:")
print(df_tp_iter3_sorted_acc.to_string(index=False))
print("\nTop 20 experiments by AUC:")
print(df_tp_iter3_sorted_auc.to_string(index=False))

#make a csv file for the top 20 accuracy and AUC ones
TOP20_RESULTS_FILE_TP = EXPERIMENTS_DIR_3 / "top20_iteration3_target_population_experiments.csv"
df_top20_tp = pd.concat([df_tp_iter3_sorted_acc, df_tp_iter3_sorted_auc]).drop_duplicates().reset_index(drop=True)
df_top20_tp.to_csv(TOP20_RESULTS_FILE_TP, index=False, encoding="utf-8")    


**Generate Predictions for Target Population (Iteration 3) - Using Logistic Regression with desc_only features**

In [33]:
# STAGE 15C: GENERATE PREDICTIONS FOR TARGET_POPULATION (ITERATION 3)
# 
# This stage generates predictions on unlabeled users using:
# - Classifier: Logistic Regression
# - Features: desc_only (TF-IDF from description column only)
# - Model selected based on iteration-3 experimental results
#
# Input: unlabeled_users_after_iteration_3_target_population.csv
# Output: iteration_3_unlabeled_users_predictions_target_population.csv

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

print("=" * 80)
print("STAGE 15C: GENERATE PREDICTIONS - TARGET_POPULATION (ITERATION 3)")
print("=" * 80)

# ============================================================================
# PATHS
# ============================================================================
CLASSIFICATION_DIR = Path("POIs") / "Classification"
LABELED_FILE = CLASSIFICATION_DIR / "labeled_data_iteration_3_target_population.csv"
UNLABELED_FILE = CLASSIFICATION_DIR / "unlabeled_users_after_iteration_3_target_population.csv"
OUTPUT_FILE = CLASSIFICATION_DIR / "iteration_3_unlabeled_users_predictions_target_population.csv"

print(f"\nInput files:")
print(f"  Labeled training data: {LABELED_FILE}")
print(f"  Unlabeled data: {UNLABELED_FILE}")
print(f"  Output file: {OUTPUT_FILE}\n")

# ============================================================================
# STEP 1: LOAD LABELED TRAINING DATA
# ============================================================================
print("[1/6] Loading labeled training data (iteration 3)...")
df_labeled = pd.read_csv(LABELED_FILE, encoding="utf-8")
print(f"  Loaded {len(df_labeled)} labeled records")

# Ensure we have the right column name
if "manual_target_population" not in df_labeled.columns:
    if "target_population" in df_labeled.columns:
        df_labeled = df_labeled.rename(columns={"target_population": "manual_target_population"})
    else:
        raise ValueError("Labeled file must contain 'target_population' or 'manual_target_population' column")

# ============================================================================
# STEP 2: FILTER OUT "UNKNOWN" LABELS AND NaN VALUES
# ============================================================================
print("\n[2/6] Filtering training data (removing 'unknown' and NaN labels)...")
print(f"  Label distribution before filtering:")
print(f"    {df_labeled['manual_target_population'].value_counts(dropna=False).to_dict()}")

# Map labels: 0=non_target, 1=target, 2=unknown
# Filter out both unknown (2) AND NaN values
df_train = df_labeled[
    (df_labeled["manual_target_population"].notna()) &  # Remove NaN
    (df_labeled["manual_target_population"] != 2)        # Remove unknown
].copy()

print(f"\n  Filtered to {len(df_train)} training samples")
print(f"  Label distribution after filtering:")
print(f"    {df_train['manual_target_population'].value_counts().to_dict()}")

# ============================================================================
# STEP 3: PREPARE TEXT FEATURES (desc_only)
# ============================================================================
print("\n[3/6] Preparing text features (desc_only)...")

# Extract descriptions and fill missing values
desc_train = df_train["description"].fillna("").astype(str)

# TF-IDF Vectorizer (matching desc_only experiment configuration)
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words="english",
    max_features=500,
    ngram_range=(1, 2),
    min_df=1
)

X_train = vectorizer.fit_transform(desc_train)
y_train = df_train["manual_target_population"].astype(int).to_numpy()

print(f"  Training feature matrix shape: {X_train.shape}")
print(f"  Training labels shape: {y_train.shape}")
print(f"  Class distribution: {np.bincount(y_train)}")

# ============================================================================
# STEP 4: TRAIN LOGISTIC REGRESSION MODEL
# ============================================================================
print("\n[4/6] Training Logistic Regression model...")

model = LogisticRegression(
    solver="liblinear",
    max_iter=5000,
    class_weight=None,
    random_state=42
)

model.fit(X_train, y_train)
print(f"  Model trained successfully")
print(f"  Classes: {model.classes_}")

# ============================================================================
# STEP 5: LOAD UNLABELED DATA AND GENERATE PREDICTIONS
# ============================================================================
print("\n[5/6] Loading unlabeled data and generating predictions...")

df_unlabeled = pd.read_csv(UNLABELED_FILE, encoding="utf-8")
print(f"  Loaded {len(df_unlabeled)} unlabeled records")

# Prepare text features for unlabeled data
desc_unlabeled = df_unlabeled["description"].fillna("").astype(str)
X_unlabeled = vectorizer.transform(desc_unlabeled)

# Generate predictions
y_pred = model.predict(X_unlabeled)
y_proba = model.predict_proba(X_unlabeled)

# Extract probabilities
prob_0 = y_proba[:, 0]  # P(non_target)
prob_1 = y_proba[:, 1]  # P(target)

# Derive prediction columns
predicted_class = np.where(prob_1 >= prob_0, "1", "0")
confidence_level = np.maximum(prob_0, prob_1)
uncertainty_score = 1 - confidence_level
prob_2 = np.full(len(df_unlabeled), np.nan)  # Binary task, no prob_2

# Validation: Check probabilities sum to ~1
prob_sums = prob_0 + prob_1
assert np.allclose(prob_sums, 1.0, atol=1e-6), "Probabilities do not sum to 1"
print(f"  ✓ Validation passed: probabilities sum to 1")

# ============================================================================
# STEP 6: APPEND PREDICTIONS AND SAVE
# ============================================================================
print("\n[6/6] Appending predictions to unlabeled data...")

# Add prediction columns to the end
df_unlabeled["predicted_class"] = predicted_class
df_unlabeled["confidence_level"] = confidence_level
df_unlabeled["prob_0"] = prob_0
df_unlabeled["prob_1"] = prob_1
df_unlabeled["prob_2"] = prob_2
df_unlabeled["uncertainty_score"] = uncertainty_score

# Sort by uncertainty_score descending
df_unlabeled = df_unlabeled.sort_values(by="uncertainty_score", ascending=False)

# Validation: Check row count
assert len(df_unlabeled) == len(df_unlabeled), "Output row count mismatch"
print(f"  ✓ Validation passed: {len(df_unlabeled)} rows in output")

# Save to CSV
CLASSIFICATION_DIR.mkdir(parents=True, exist_ok=True)
df_unlabeled.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

print(f"\n{'=' * 80}")
print(f"SUCCESS! Predictions saved to:")
print(f"  {OUTPUT_FILE}")
print(f"\nPrediction summary:")
print(f"  Total predictions: {len(df_unlabeled)}")
print(f"  Predicted 'target': {(predicted_class == '1').sum()}")
print(f"  Predicted 'non_target': {(predicted_class == '0').sum()}")
print(f"  Mean confidence: {confidence_level.mean():.4f}")
print(f"  Mean uncertainty: {uncertainty_score.mean():.4f}")
print(f"\nTop 5 most uncertain predictions:")
print(df_unlabeled[["username", "predicted_class", "uncertainty_score", "confidence_level"]].head())
print("=" * 80)

STAGE 15C: GENERATE PREDICTIONS - TARGET_POPULATION (ITERATION 3)

Input files:
  Labeled training data: POIs\Classification\labeled_data_iteration_3_target_population.csv
  Unlabeled data: POIs\Classification\unlabeled_users_after_iteration_3_target_population.csv
  Output file: POIs\Classification\iteration_3_unlabeled_users_predictions_target_population.csv

[1/6] Loading labeled training data (iteration 3)...
  Loaded 300 labeled records

[2/6] Filtering training data (removing 'unknown' and NaN labels)...
  Label distribution before filtering:
    {0.0: 131, 1.0: 105, 2.0: 62, nan: 2}

  Filtered to 236 training samples
  Label distribution after filtering:
    {0.0: 131, 1.0: 105}

[3/6] Preparing text features (desc_only)...
  Training feature matrix shape: (236, 500)
  Training labels shape: (236,)
  Class distribution: [131 105]

[4/6] Training Logistic Regression model...
  Model trained successfully
  Classes: [0 1]

[5/6] Loading unlabeled data and generating predictions...

### STAGE 16E: Select Top 100 Most Uncertain Predictions for Manual Labeling (Iteration 4 Candidates)

In [34]:
# STAGE 16E: CREATE ITERATION 4 MANUAL LABELING FILE
# 
# This stage selects the top 100 most uncertain predictions from iteration 3
# and creates a file for manual labeling in iteration 4.
#
# Input: iteration_3_unlabeled_users_predictions_target_population.csv
# Output: iteration_4_manual_labels_target_population.csv

import pandas as pd
from pathlib import Path

print("=" * 80)
print("STAGE 16E: SELECT TOP 100 UNCERTAIN PREDICTIONS FOR ITERATION 4")
print("=" * 80)

# ============================================================================
# PATHS
# ============================================================================
CLASSIFICATION_DIR = Path("POIs") / "Classification"
PREDICTIONS_FILE = CLASSIFICATION_DIR / "iteration_3_unlabeled_users_predictions_target_population.csv"
OUTPUT_FILE = CLASSIFICATION_DIR / "iteration_4_manual_labels_target_population.csv"

print(f"\nInput file: {PREDICTIONS_FILE}")
print(f"Output file: {OUTPUT_FILE}\n")

# ============================================================================
# LOAD PREDICTIONS
# ============================================================================
print("[1/3] Loading predictions from iteration 3...")
df_predictions = pd.read_csv(PREDICTIONS_FILE, encoding="utf-8")
print(f"  Loaded {len(df_predictions)} predictions")
print(f"  Uncertainty score range: {df_predictions['uncertainty_score'].min():.4f} - {df_predictions['uncertainty_score'].max():.4f}")

# ============================================================================
# SELECT TOP 100 MOST UNCERTAIN
# ============================================================================
print("\n[2/3] Selecting top 100 most uncertain predictions...")

# The file is already sorted by uncertainty_score descending, but let's ensure it
df_top100 = df_predictions.nlargest(100, 'uncertainty_score').copy()

print(f"  Selected {len(df_top100)} candidates")
print(f"  Uncertainty score range in selection: {df_top100['uncertainty_score'].min():.4f} - {df_top100['uncertainty_score'].max():.4f}")
print(f"\n  Predicted class distribution in top 100:")
print(f"    {df_top100['predicted_class'].value_counts().to_dict()}")

# ============================================================================
# PREPARE MANUAL LABELING FILE
# ============================================================================
print("\n[3/3] Preparing manual labeling file...")

# Add a column for manual labels (empty, to be filled by annotators)
df_top100["manual_target_population"] = ""

# Select columns for manual labeling
# Include key information for annotators to make decisions
columns_to_keep = [
    "username",
    "display_name",
    "manual_target_population",  # Empty column for manual annotation
    "predicted_class",
    "uncertainty_score",
    "confidence_level",
    "description",
    "description_en",
    "display_name_en",
    "followers_count",
    "following_count",
    "statuses_count",
    "verified",
    "created_at",
    "location",
    "url",
    "profile_image_url",
    "profile_banner_url"
]

# Keep only columns that exist in the dataframe
columns_available = [col for col in columns_to_keep if col in df_top100.columns]
df_manual = df_top100[columns_available].copy()

# Save to CSV
df_manual.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

print(f"\n{'=' * 80}")
print(f"SUCCESS! Manual labeling file created:")
print(f"  {OUTPUT_FILE}")
print(f"\nFile details:")
print(f"  Total candidates: {len(df_manual)}")
print(f"  Columns included: {len(columns_available)}")
print(f"\nColumns in file:")
for col in columns_available:
    print(f"  - {col}")
print(f"\n{'=' * 80}")
print(f"📋 NEXT STEPS:")
print(f"  1. Open: {OUTPUT_FILE}")
print(f"  2. Manually label the 'manual_target_population' column")
print(f"  3. Use values: 0 (non_target), 1 (target), 2 (unknown)")
print(f"  4. Save the file when done")
print("=" * 80)

STAGE 16E: SELECT TOP 100 UNCERTAIN PREDICTIONS FOR ITERATION 4

Input file: POIs\Classification\iteration_3_unlabeled_users_predictions_target_population.csv
Output file: POIs\Classification\iteration_4_manual_labels_target_population.csv

[1/3] Loading predictions from iteration 3...
  Loaded 846 predictions
  Uncertainty score range: 0.2272 - 0.4990

[2/3] Selecting top 100 most uncertain predictions...
  Selected 100 candidates
  Uncertainty score range in selection: 0.4334 - 0.4990

  Predicted class distribution in top 100:
    {1: 53, 0: 47}

[3/3] Preparing manual labeling file...

SUCCESS! Manual labeling file created:
  POIs\Classification\iteration_4_manual_labels_target_population.csv

File details:
  Total candidates: 100
  Columns included: 18

Columns in file:
  - username
  - display_name
  - manual_target_population
  - predicted_class
  - uncertainty_score
  - confidence_level
  - description
  - description_en
  - display_name_en
  - followers_count
  - following_cou

In [11]:
#add menual column empty for iteration_4_manual_labeling_candidates_person_vs_organization_english
# add empty column for manual labels to the existing file
import pandas as pd
from pathlib import Path
print("=" * 80)
print("STAGE 16F: ADD MANUAL LABEL COLUMN TO ITERATION 4 PERSON_VS_ORGANIZATION CANDIDATES")
print("=" * 80)
CLASSIFICATION_DIR = Path("POIs") / "Classification"
INPUT_FILE = CLASSIFICATION_DIR / "iteration_4_manual_labeling_candidates_person_vs_organization_english.csv"
OUTPUT_FILE = CLASSIFICATION_DIR / "iteration_4_manual_labels_person_vs_organization_english.csv"
print(f"\nLoading candidates from: {INPUT_FILE}")
df_candidates = pd.read_csv(INPUT_FILE, encoding="utf-8")
print(f"Loaded {len(df_candidates)} candidates.")
df_candidates["manual_person_vs_organization"] = ""
df_candidates.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"Saved updated file with manual label column to: {OUTPUT_FILE}")
# ============================================================================

STAGE 16F: ADD MANUAL LABEL COLUMN TO ITERATION 4 PERSON_VS_ORGANIZATION CANDIDATES

Loading candidates from: POIs\Classification\iteration_4_manual_labeling_candidates_person_vs_organization_english.csv
Loaded 100 candidates.
Saved updated file with manual label column to: POIs\Classification\iteration_4_manual_labels_person_vs_organization_english.csv


In [12]:
# Create unlabeled users files after iteration 4 by excluding manually labeled users
import pandas as pd
from pathlib import Path

print("=" * 80)
print("STAGE 16G: CREATE UNLABELED USERS FILES AFTER ITERATION 4")
print("=" * 80)

CLASSIFICATION_DIR = Path("POIs") / "Classification"

# Process Person vs Organization
print(f"\nProcessing Person vs Organization...")
INPUT_FILE_PO = CLASSIFICATION_DIR / "unlabeled_users_after_iteration_3_person_vs_organization.csv"
MANUAL_LABELS_PO = CLASSIFICATION_DIR / "iteration_4_manual_labels_person_vs_organization_english.csv"
OUTPUT_FILE_PO = CLASSIFICATION_DIR / "unlabeled_users_after_iteration_4_person_vs_organization.csv"

print(f"Loading unlabeled users from: {INPUT_FILE_PO}")
df_candidates_po = pd.read_csv(INPUT_FILE_PO, encoding="utf-8")
print(f"Loaded {len(df_candidates_po)} candidate users.")

print(f"Loading manually labeled users from: {MANUAL_LABELS_PO}")
df_labeled_po = pd.read_csv(MANUAL_LABELS_PO, encoding="utf-8")
print(f"Loaded {len(df_labeled_po)} manually labeled users.")

# Exclude manually labeled users by username
df_unlabeled_po = df_candidates_po[~df_candidates_po["username"].isin(df_labeled_po["username"])]
print(f"After excluding manually labeled users: {len(df_unlabeled_po)} unlabeled users remain.")

df_unlabeled_po.to_csv(OUTPUT_FILE_PO, index=False, encoding="utf-8")
print(f"✓ Unlabeled users saved to: {OUTPUT_FILE_PO}")

# Process Target Population
print(f"\nProcessing Target Population...")
INPUT_FILE_TP = CLASSIFICATION_DIR / "unlabeled_users_after_iteration_3_target_population.csv"
MANUAL_LABELS_TP = CLASSIFICATION_DIR / "iteration_4_manual_labels_target_population.csv"
OUTPUT_FILE_TP = CLASSIFICATION_DIR / "unlabeled_users_after_iteration_4_target_population.csv"

print(f"Loading unlabeled users from: {INPUT_FILE_TP}")
df_candidates_tp = pd.read_csv(INPUT_FILE_TP, encoding="utf-8")
print(f"Loaded {len(df_candidates_tp)} candidate users.")

print(f"Loading manually labeled users from: {MANUAL_LABELS_TP}")
df_labeled_tp = pd.read_csv(MANUAL_LABELS_TP, encoding="utf-8")
print(f"Loaded {len(df_labeled_tp)} manually labeled users.")

# Exclude manually labeled users by username
df_unlabeled_tp = df_candidates_tp[~df_candidates_tp["username"].isin(df_labeled_tp["username"])]
print(f"After excluding manually labeled users: {len(df_unlabeled_tp)} unlabeled users remain.")

df_unlabeled_tp.to_csv(OUTPUT_FILE_TP, index=False, encoding="utf-8")
print(f"✓ Unlabeled users saved to: {OUTPUT_FILE_TP}")

print(f"\n{'=' * 80}")
print("PROCESSING COMPLETE")
print("=" * 80)

STAGE 16G: CREATE UNLABELED USERS FILES AFTER ITERATION 4

Processing Person vs Organization...
Loading unlabeled users from: POIs\Classification\unlabeled_users_after_iteration_3_person_vs_organization.csv
Loaded 648 candidate users.
Loading manually labeled users from: POIs\Classification\iteration_4_manual_labels_person_vs_organization_english.csv
Loaded 100 manually labeled users.
After excluding manually labeled users: 548 unlabeled users remain.
✓ Unlabeled users saved to: POIs\Classification\unlabeled_users_after_iteration_4_person_vs_organization.csv

Processing Target Population...
Loading unlabeled users from: POIs\Classification\unlabeled_users_after_iteration_3_target_population.csv
Loaded 846 candidate users.
Loading manually labeled users from: POIs\Classification\iteration_4_manual_labels_target_population.csv
Loaded 100 manually labeled users.
After excluding manually labeled users: 746 unlabeled users remain.
✓ Unlabeled users saved to: POIs\Classification\unlabeled_us